**sample data set downloaded from this **
[link text](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data)

**Metric names**

Here are the metric names extracted from the image:

Correlation

*   Completeness
*   Completeness
*    Uniqueness
*    Minimum
*    Mean
*    Maximum
*    Entropy
*    ApproxQuantiles
*    MaxLength
*    Size
*    MinLength
*    Sum
*    UniqueValueRatio
*    CountDistinct
*    Distinctness
*    StandardDeviation
*    negativeCount
*    Histogram
*    DataType
*    zeroCount
*    missingCount



**install required lib**

In [ ]:
gdrive_path="/content/drive/MyDrive/Anamolies/CATS_dataset"

In [ ]:
!pip install cupy-cuda12x -q
!pip install pyod -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from typing import Dict, List, Optional, Any, Union
import os
import matplotlib.pyplot as plt
import time

**Read the sample data**

In [ ]:
source_Df=pd.read_csv(gdrive_path+"/data.csv")
soure_columns=[c for c in source_Df.columns if c != 'timestamp']
print(f" list of source columns {soure_columns}")

 list of source columns ['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1', 'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1', 'y', 'category']


**generate historical data**

In [ ]:
import pandas as pd
import numpy as np
try:
    import cupy as cp
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False
    print("CuPy not available, falling back to CPU")

def generate_historical_data_gpu(df, historical_days=90, batch_size=100000):
    """
    GPU-accelerated historical data generation
    """
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')

    # Data characteristics
    min_date = df['timestamp'].min()
    max_date = df['timestamp'].max()
    total_records = len(df)
    total_days = (max_date - min_date).total_seconds() / (24 * 3600)
    records_per_day = total_records / total_days

    print(f"Original: {total_records:,} records, generating {historical_days} days")

    # Generate timestamps
    historical_records = int(historical_days * records_per_day)
    start_historical = min_date - pd.Timedelta(days=historical_days)

    historical_timestamps = pd.date_range(
        start=start_historical,
        end=min_date - pd.Timedelta(seconds=1),
        periods=historical_records
    )

    print(f"Creating {len(historical_timestamps):,} records using {'GPU' if GPU_AVAILABLE else 'CPU'}")

    # Prepare data for GPU processing
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = [col for col in df.columns if col not in numeric_cols and col != 'timestamp']

    # Sample for patterns
    sample_df = df.sample(n=min(50000, len(df)), random_state=42)

    # Process in batches
    all_data = []

    for batch_start in range(0, len(historical_timestamps), batch_size):
        batch_end = min(batch_start + batch_size, len(historical_timestamps))
        batch_timestamps = historical_timestamps[batch_start:batch_end]
        batch_size_actual = len(batch_timestamps)

        batch_data = {'timestamp': batch_timestamps}

        if GPU_AVAILABLE:
            # GPU processing for numeric columns
            for col in numeric_cols:
                if col in sample_df.columns:
                    col_values = sample_df[col].dropna().values

                    # Move to GPU
                    gpu_values = cp.array(col_values)

                    # Random sampling on GPU
                    indices = cp.random.choice(len(gpu_values), batch_size_actual, replace=True)
                    base_values = gpu_values[indices]

                    # Add variation on GPU
                    variations = base_values * cp.random.normal(0, 0.02, batch_size_actual)
                    final_values = base_values + variations

                    # Move back to CPU
                    batch_data[col] = cp.asnumpy(final_values)
        else:
            # CPU fallback
            for col in numeric_cols:
                if col in sample_df.columns:
                    col_values = sample_df[col].dropna().values
                    base_values = np.random.choice(col_values, batch_size_actual, replace=True)
                    variations = base_values * np.random.normal(0, 0.02, batch_size_actual)
                    batch_data[col] = base_values + variations

        # Categorical columns (CPU)
        for col in categorical_cols:
            if col in sample_df.columns and not sample_df[col].isna().all():
                unique_vals = sample_df[col].dropna().unique()
                batch_data[col] = np.random.choice(unique_vals, batch_size_actual, replace=True)

        all_data.append(pd.DataFrame(batch_data))

        if (batch_end) % 500000 == 0:
            print(f"Processed {batch_end:,} / {len(historical_timestamps):,} records")

    # Combine all batches
    historical_df = pd.concat(all_data, ignore_index=True)
    result = pd.concat([historical_df, df], ignore_index=True)
    result = result.sort_values('timestamp').reset_index(drop=True)

    print(f"Complete: {len(result):,} records from {result['timestamp'].min()} to {result['timestamp'].max()}")

    return result

def generate_historical_data_vectorized(df, historical_days=90):
    """
    Vectorized CPU version (faster than loop)
    """
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')

    # Data characteristics
    min_date = df['timestamp'].min()
    total_records = len(df)
    total_days = (df['timestamp'].max() - min_date).total_seconds() / (24 * 3600)
    records_per_day = total_records / total_days

    print(f"Vectorized generation: {historical_days} days")

    # Generate timestamps
    historical_records = int(historical_days * records_per_day)
    start_historical = min_date - pd.Timedelta(days=historical_days)

    historical_timestamps = pd.date_range(
        start=start_historical,
        end=min_date - pd.Timedelta(seconds=1),
        periods=historical_records
    )

    # Sample patterns
    sample_df = df.sample(n=min(100000, len(df)), random_state=42)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = [col for col in df.columns if col not in numeric_cols and col != 'timestamp']

    print(f"Creating {len(historical_timestamps):,} records (vectorized)")

    # Vectorized generation
    historical_data = {'timestamp': historical_timestamps}

    # Numeric columns - vectorized
    for col in numeric_cols:
        if col in sample_df.columns:
            col_values = sample_df[col].dropna().values
            base_values = np.random.choice(col_values, len(historical_timestamps), replace=True)
            variations = base_values * np.random.normal(0, 0.02, len(historical_timestamps))
            historical_data[col] = base_values + variations

    # Categorical columns - vectorized
    for col in categorical_cols:
        if col in sample_df.columns and not sample_df[col].isna().all():
            unique_vals = sample_df[col].dropna().unique()
            historical_data[col] = np.random.choice(unique_vals, len(historical_timestamps), replace=True)

    # Combine
    historical_df = pd.DataFrame(historical_data)
    result = pd.concat([historical_df, df], ignore_index=True)
    result = result.sort_values('timestamp').reset_index(drop=True)

    print(f"Complete: {len(result):,} records")
    return result

# Usage:
# Fast CPU version: df = generate_historical_data_vectorized(source_df, 120)
# GPU version: df = generate_historical_data_gpu(source_df, 120)

In [ ]:
df=generate_historical_data_gpu(source_Df, 45)

Original data: 5,000,000 records from 2023-01-01 00:00:00 to 2023-02-27 20:53:19
Spans 57.9 days, ~86400 records/day
Generating 120 days of historical data...
Creating 10,368,002 historical records
Historical range: 2022-09-03 00:00:00 to 2022-12-31 23:59:59
Generated 50,000 / 10,368,002 records
Generated 100,000 / 10,368,002 records
Generated 150,000 / 10,368,002 records
Generated 200,000 / 10,368,002 records
Generated 250,000 / 10,368,002 records
Generated 300,000 / 10,368,002 records
Generated 350,000 / 10,368,002 records
Generated 400,000 / 10,368,002 records
Generated 450,000 / 10,368,002 records


In [ ]:
print(f"extended: {df['timestamp'].max()} and min: {df['timestamp'].min()}")
print(f"\n extended: {source_Df['timestamp'].max()} and min: {source_Df['timestamp'].min()}")

extended: 2023-02-27 21:06:31 and min: 2023-01-01 00:00:00

 extended: 2023-02-27 20:53:19 and min: 2023-01-01 00:00:00


**Profiling**

In [ ]:
class DataQualityProfiler:
    """
    Dataset-level DQ metrics for anomaly detection
    Approach: Column-wise metrics per day → detect corrupt datasets (not individual accounts)
    """

    def __init__(self, df, numeric_cols=None, exclude_cols=None):
        print("Initializing DataQualityProfiler...")
        self.df = df.copy()
        self.exclude_cols = exclude_cols or []

        # Use provided numeric_cols or auto-detect
        if numeric_cols:
            self.numeric_cols = numeric_cols
        else:
            self.numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        # Filter out excluded columns
        self.numeric_cols = [col for col in self.numeric_cols if col not in self.exclude_cols]

        print(f"Initialized with {len(self.df)} records, {len(self.numeric_cols)} numeric columns")
        if self.exclude_cols:
            print(f"Excluded {len(self.exclude_cols)} columns from profiling")


    def prepare_timestamp(self, timestamp_col='timestamp'):
        """Convert timestamp to yyyy-mm-dd format"""
        print(f"Converting {timestamp_col} to yyyy-mm-dd format...")

        if timestamp_col in self.df.columns:
            self.df[timestamp_col] = pd.to_datetime(self.df[timestamp_col]).dt.date
            unique_dates = self.df[timestamp_col].nunique()
            print(f"Converted timestamps - Found {unique_dates} unique dates")
        else:
            print("No timestamp column found, creating synthetic dates")

        return self.df[timestamp_col].unique() if timestamp_col in self.df.columns else None

    def calculate_basic_metrics(self, series, col_name, date):
        """Calculate basic statistical metrics with logging"""
        print(f"Basic metrics for {col_name} on {date}")

        return {
            'Mean': series.mean(),
            'Minimum': series.min(),
            'Maximum': series.max(),
            'StandardDeviation': series.std(),
            'Sum': series.sum(),
            'count': len(series),
            'Completeness': series.notna().sum() / len(series),
            'missingCount': series.isna().sum(),
        }

    def calculate_distribution_metrics(self, series, col_name, date):
        """Calculate distribution-based metrics with logging"""
        print(f"Distribution metrics for {col_name} on {date}")

        clean_series = series.dropna()

        # Histogram calculation
        try:
            hist_counts, _ = np.histogram(clean_series, bins=10)
            histogram_str = ','.join(map(str, hist_counts))
        except:
            histogram_str = "0,0,0,0,0,0,0,0,0,0"

        # Data type inference
        if pd.api.types.is_numeric_dtype(series):
            data_type = "INTEGER" if pd.api.types.is_integer_dtype(series) else "DECIMAL"
        elif pd.api.types.is_datetime64_any_dtype(series):
            data_type = "TIMESTAMP"
        elif pd.api.types.is_bool_dtype(series):
            data_type = "BOOLEAN"
        else:
            data_type = "STRING"

        return {
            'zerocount': (clean_series == 0).sum(),
            'zeroCount': (clean_series == 0).sum(),
            'zerocountpercentage': ((clean_series == 0).sum() / len(clean_series)) * 100 if len(clean_series) > 0 else 0,
            'negativecount': (clean_series < 0).sum(),
            'negativeCount': (clean_series < 0).sum(),
            'negativecountpercentage': ((clean_series < 0).sum() / len(clean_series)) * 100 if len(clean_series) > 0 else 0,
            'CountDistinct': clean_series.nunique(),
            'Uniqueness': clean_series.nunique() / len(clean_series) if len(clean_series) > 0 else 0,
            'UniqueValueRatio': clean_series.nunique() / len(clean_series) if len(clean_series) > 0 else 0,
            'Distinctness': clean_series.nunique() / len(clean_series) if len(clean_series) > 0 else 0,
            'Histogram': histogram_str,
            'DataType': data_type,
        }

    def calculate_quantiles(self, series, col_name, date):
        """Calculate approximate quantiles with logging"""
        print(f"Quantile metrics for {col_name} on {date}")

        clean_series = series.dropna()
        if len(clean_series) == 0:
            return {
                'ApproxQuantiles_0.1': 0, 'ApproxQuantiles_0.25': 0, 'ApproxQuantiles_0.5': 0,
                'ApproxQuantiles_0.75': 0, 'ApproxQuantiles_0.9': 0
            }

        return {
            'ApproxQuantiles_0.1': clean_series.quantile(0.1),
            'ApproxQuantiles_0.25': clean_series.quantile(0.25),
            'ApproxQuantiles_0.5': clean_series.quantile(0.5),
            'ApproxQuantiles_0.75': clean_series.quantile(0.75),
            'ApproxQuantiles_0.9': clean_series.quantile(0.9),
        }

    def calculate_entropy(self, series, col_name, date):
        """Calculate Shannon entropy with logging"""
        print(f"Entropy calculation for {col_name} on {date}")

        clean_series = series.dropna()
        if len(clean_series) == 0:
            return 0

        try:
            bins = min(50, len(clean_series.unique()))
            hist, _ = np.histogram(clean_series, bins=bins)
            hist = hist[hist > 0]
            probs = hist / hist.sum()
            entropy = -np.sum(probs * np.log2(probs))
            print(f"Entropy calculated: {entropy:.3f}")
            return entropy
        except:
            print(f"Entropy calculation failed for {col_name}")
            return 0

    def calculate_correlation_matrix(self, day_data, date):
        """Calculate correlation matrix for the day with logging"""
        print(f"Correlation matrix for {date}")

        available_cols = [col for col in self.numeric_cols if col in day_data.columns]
        if len(available_cols) < 2:
            return pd.DataFrame()

        corr_matrix = day_data[available_cols].corr()
        print(f"Correlation matrix calculated for {len(available_cols)} columns")
        return corr_matrix

    def calculate_mutual_information(self, col1_data, col2_data, col1, col2, date):
        """Calculate mutual information with logging"""
        print(f"Mutual information: {col1} vs {col2} on {date}")

        try:
            x_binned = pd.cut(col1_data.dropna(), bins=10, labels=False)
            y_binned = pd.cut(col2_data.dropna(), bins=10, labels=False)

            # Align the series
            min_len = min(len(x_binned), len(y_binned))
            x_binned = x_binned[:min_len]
            y_binned = y_binned[:min_len]

            contingency = pd.crosstab(x_binned, y_binned)

            mi = 0
            total = contingency.sum().sum()

            for i in range(contingency.shape[0]):
                for j in range(contingency.shape[1]):
                    if contingency.iloc[i, j] > 0:
                        pxy = contingency.iloc[i, j] / total
                        px = contingency.iloc[i, :].sum() / total
                        py = contingency.iloc[:, j].sum() / total
                        mi += pxy * np.log2(pxy / (px * py))

            print(f"MI calculated: {mi:.3f}")
            return mi
        except Exception as e:
            print(f"MI calculation failed: {str(e)}")
            return 0

    def profile_dataset_daily(self, timestamp_col='timestamp'):
        """
        Profile dataset column-wise per day for dataset-level anomaly detection
        Returns: One row per date with aggregated column metrics
        """
        print("Starting dataset-level daily profiling...")

        # Prepare timestamps
        dates = self.prepare_timestamp(timestamp_col)

        if dates is None:
            print("No valid dates found")
            return pd.DataFrame()

        features_list = []

        for date_idx, date in enumerate(sorted(dates)):
            print(f"Processing date {date_idx + 1}/{len(dates)}: {date}")

            # Get all data for this date
            if timestamp_col in self.df.columns:
                day_data = self.df[self.df[timestamp_col] == date]
            else:
                # Fallback to chunking
                start_idx = date_idx * 1000
                end_idx = min(start_idx + 1000, len(self.df))
                day_data = self.df.iloc[start_idx:end_idx]

            if len(day_data) == 0:
                print(f"No data for {date}")
                continue

            print(f"Processing {len(day_data)} records")

            # Initialize daily features
            daily_features = {
                'date': str(date),
                'total_records': len(day_data),
                'dataset_completeness': day_data.notna().sum().sum() / (len(day_data) * len(self.numeric_cols))
            }

            # Calculate correlation matrix for the day
            corr_matrix = self.calculate_correlation_matrix(day_data, date)

            # Process each column
            for col in self.numeric_cols:
                if col not in day_data.columns:
                    print(f"Column {col} not found")
                    continue

                print(f"Processing column: {col}")
                series = day_data[col]

                # Basic metrics
                basic_metrics = self.calculate_basic_metrics(series, col, date)
                for metric, value in basic_metrics.items():
                    daily_features[f"{col}_{metric}"] = value

                # Distribution metrics
                dist_metrics = self.calculate_distribution_metrics(series, col, date)
                for metric, value in dist_metrics.items():
                    daily_features[f"{col}_{metric}"] = value

                # Quantiles
                quantile_metrics = self.calculate_quantiles(series, col, date)
                for metric, value in quantile_metrics.items():
                    daily_features[f"{col}_{metric}"] = value

                # Entropy
                daily_features[f"{col}_Entropy"] = self.calculate_entropy(series, col, date)

                # Size metrics
                daily_features[f"{col}_Size"] = len(series)
                if series.dtype == 'object':
                    daily_features[f"{col}_MaxLength"] = series.astype(str).str.len().max()
                    daily_features[f"{col}_MinLength"] = series.astype(str).str.len().min()
                else:
                    daily_features[f"{col}_MaxLength"] = 0
                    daily_features[f"{col}_MinLength"] = 0

                # Correlation with other columns
                if not corr_matrix.empty and col in corr_matrix.columns:
                    for other_col in self.numeric_cols[:5]:  # Limit correlations
                        if other_col != col and other_col in corr_matrix.columns:
                            daily_features[f"{col}_Correlation_{other_col}"] = corr_matrix.loc[col, other_col]

                # Mutual information with first column
                if col != self.numeric_cols[0] and self.numeric_cols[0] in day_data.columns:
                    mi_value = self.calculate_mutual_information(
                        day_data[col], day_data[self.numeric_cols[0]], col, self.numeric_cols[0], date
                    )
                    daily_features[f"{col}_MutualInformation"] = mi_value

            features_list.append(daily_features)
            print(f"Completed {date} - Generated {len(daily_features)} features")

            # Limit for demo
            if len(features_list) >= 180:
                break

        result_df = pd.DataFrame(features_list)
        print(f"Profiling complete! Generated {result_df.shape[0]} days × {result_df.shape[1]} features")
        return result_df

**Z score calculation**

In [ ]:
# def prepare_features_for_model(df, zscore_threshold=3.0, target_cols=None, enhanced=False):
def prepare_features_for_model(df, zscore_threshold=3.0, target_cols=None, enhanced=False, exclude_original_cols=None):
    """Main function to prepare dataset-level features for anomaly detection"""
    print("DATASET-LEVEL ANOMALY DETECTION FEATURE PREPARATION")
    print("=" * 60)
    original_cols_to_exclude = exclude_original_cols or []
    # Initialize profiler select only the numeric columns
    # profiler = DataQualityProfiler(df)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    profiler = DataQualityProfiler(df, numeric_cols=numeric_cols, exclude_cols=original_cols_to_exclude)

    # Generate base features (daily dataset profiles)
    print("\n STEP 1: Calculating daily DQ metrics...")
    feature_df = profiler.profile_dataset_daily()

    if feature_df.empty:
        print("No features generated")
        return pd.DataFrame()

    # Add rolling window features with z-scores
    print("\n  STEP 2: Creating rolling features...")
    final_features = create_rolling_features(feature_df)

    # Handle null values for ML models
    print("\n STEP 3: Handling null values...")
    final_features = handle_nulls_for_ml(final_features)

    if enhanced:
        # Enhanced feature engineering for LSTM
        print(f"\n STEP 4: Enhanced feature engineering (threshold={zscore_threshold})...")
        feature_engineer = FeatureEngineer(zscore_threshold=zscore_threshold)
        final_features = feature_engineer.create_features(
            final_features,
            process_date_col='date',
            target_cols=target_cols
        )

    print(f"\nFEATURE PREPARATION COMPLETE!")
    print(f" Final shape: {final_features.shape[0]} days × {final_features.shape[1]} features")
    print(f" Ready for dataset-level anomaly detection")

    return final_features

**Create Lag features**

In [ ]:
# def create_lag_features(feature_df, lag_features=[1, 2, 3], base_columns_only=True):
def create_lag_features(feature_df, lag_features=[1, 2, 3], base_columns_only=True, exclude_cols=None):
    """Create lag features on base columns only (not derived features)"""
    print(f" Creating lag features: {lag_features}")
    exclude_cols = exclude_cols or []
    print(f" exclude cols {exclude_cols}")
    feature_df = feature_df.sort_values('date').reset_index(drop=True)

    if base_columns_only:
      # Exclude derived features and excluded columns
      derived_suffixes = ['_rolling_', '_zscore_', '_anomaly_', '_diff_', '_ratio_', '_trend_', '_mean', '_std', '_min', '_max']
      base_columns = [col for col in feature_df.columns
                    if col not in ['date'] + exclude_cols
                    and not any(suffix in col for suffix in derived_suffixes)]
    else:
        base_columns = [col for col in feature_df.columns
                      if col not in ['date'] + exclude_cols]

    # if base_columns_only:
    #     # Exclude derived features - only base columns
    #     derived_suffixes = ['_rolling_', '_zscore_', '_anomaly_', '_diff_', '_ratio_', '_trend_', '_mean', '_std', '_min', '_max']
    #     base_columns = [col for col in feature_df.columns
    #                    if col != 'date' and not any(suffix in col for suffix in derived_suffixes)]
    # else:
    #     base_columns = [col for col in feature_df.columns if col != 'date']

    result_df = feature_df.copy()

    for lag in lag_features:
        for col in base_columns:
            if pd.api.types.is_numeric_dtype(feature_df[col]):
                # Create lag feature
                result_df[f"{col}_lag_{lag}d"] = feature_df[col].shift(lag)
                # Create difference from lag
                result_df[f"{col}_diff_lag_{lag}d"] = feature_df[col] - result_df[f"{col}_lag_{lag}d"]

    # Fill NaN values from shifting
    lag_cols = [col for col in result_df.columns if '_lag_' in col]
    result_df[lag_cols] = result_df[lag_cols].fillna(0)

    new_features = len(result_df.columns) - len(feature_df.columns)
    print(f" Added {new_features} lag features")

    return result_df

**Deseasonality**

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
from typing import List, Dict, Tuple, Optional, Union
import logging
import warnings
import multiprocessing as mp
from functools import partial
import psutil
import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
warnings.filterwarnings('ignore')

class SmartSTLDeseasonalizer:
    """
    Smart STL deseasonalization with automatic GPU/CPU detection and scaling
    Features: GPU preprocessing, CPU STL, parallel execution, automatic resource detection
    """

    def __init__(self,
                 frequency: str = 'daily',
                 custom_params: Optional[Dict] = None,
                 force_mode: Optional[str] = None):  # 'cpu', 'gpu', 'hybrid', None=auto
        """
        Initialize Smart STL deseasonalizer with automatic resource detection

        Args:
            frequency: 'daily', 'weekly', 'monthly'
            custom_params: Override default parameters
            force_mode: Force specific mode or None for auto-detection
        """
        self.frequency = frequency
        self.seasonal_components = {}
        self.decomposition_cache = {}
        self.force_mode = force_mode

        # Resource detection and mode selection
        print("Detecting system capabilities...")
        self.capabilities = self._detect_system_capabilities()
        self.execution_mode = self._select_optimal_mode()

        # Initialize based on available resources
        print("Initializing compute backend...")
        self._initialize_compute_backend()

        # Industry standard parameters for different frequencies
        # These parameters are tuned based on common time series characteristics:
        # - seasonal: controls smoothness of seasonal component (higher = smoother)
        # - period: expected seasonal cycle length
        # - min_periods: minimum data points required for reliable decomposition
        default_params = {
            'daily': {'seasonal': 15, 'period': 7, 'min_periods': 21},
            'weekly': {'seasonal': 27, 'period': 52, 'min_periods': 104},
            'monthly': {'seasonal': 13, 'period': 12, 'min_periods': 24}
        }

        self.params = default_params[frequency].copy()
        if custom_params:
            self.params.update(custom_params)

        print("Smart STL Deseasonalizer initialized successfully")
        print(f"Mode: {self.execution_mode}")
        print(f"Frequency: {frequency}")
        print(f"System capabilities: {self._format_capabilities()}")
        print(f"STL parameters: {self.params}")

    def _detect_system_capabilities(self) -> Dict:
        """
        Automatically detect available compute resources

        This function probes the system for:
        1. CPU cores and memory for parallel processing
        2. GPU libraries (cuDF, CuPy, PyTorch) for accelerated preprocessing
        3. Actual GPU availability and memory

        Returns a comprehensive capability dictionary for optimal mode selection.
        """
        capabilities = {
            'cpu_cores': psutil.cpu_count(logical=True),
            'memory_gb': round(psutil.virtual_memory().total / (1024**3), 1),
            'gpu_available': False,
            'cudf_available': False,
            'cupy_available': False,
            'torch_gpu_available': False,
            'gpu_memory_gb': 0
        }

        print(f"System specs - CPU: {capabilities['cpu_cores']} cores, RAM: {capabilities['memory_gb']}GB")

        # Test GPU libraries - each serves different purposes:
        # cuDF: GPU-accelerated DataFrame operations (like pandas but on GPU)
        # CuPy: GPU arrays and numerical operations (like NumPy but on GPU)
        # PyTorch: Deep learning framework with GPU tensor operations

        try:
            import cudf
            capabilities['cudf_available'] = True
            print("cuDF (GPU DataFrame operations) detected")
        except ImportError:
            print("cuDF not available - using pandas for DataFrame operations")

        try:
            import cupy as cp
            capabilities['cupy_available'] = True
            # Test if GPU is actually accessible (libraries can be installed but GPU unavailable)
            try:
                cp.cuda.Device(0).compute_capability
                capabilities['gpu_available'] = True
                # Get GPU memory for capacity planning
                mempool = cp.get_default_memory_pool()
                capabilities['gpu_memory_gb'] = round(mempool.total_bytes() / (1024**3), 1)
                print(f"CuPy (GPU arrays) detected and working - GPU memory: {capabilities['gpu_memory_gb']}GB")
            except:
                capabilities['cupy_available'] = False
                print("CuPy installed but GPU not accessible")
        except ImportError:
            print("CuPy not available - using NumPy for array operations")

        try:
            import torch
            if torch.cuda.is_available():
                capabilities['torch_gpu_available'] = True
                capabilities['gpu_memory_gb'] = round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1)
                print(f"PyTorch GPU detected - GPU memory: {capabilities['gpu_memory_gb']}GB")
            else:
                print("PyTorch available but no GPU support")
        except ImportError:
            print("PyTorch not available")

        return capabilities

    def _select_optimal_mode(self) -> str:
        """
        Intelligently select the best execution mode based on available resources

        Mode selection hierarchy:
        1. hybrid_gpu: Best performance - GPU preprocessing + parallel CPU STL
        2. hybrid_torch: Good for PyTorch users - PyTorch GPU preprocessing + parallel CPU STL
        3. parallel_cpu: Standard multi-core processing
        4. single_cpu: Fallback for limited resources

        The selection considers both availability and performance thresholds.
        """
        if self.force_mode:
            print(f"Using forced mode: {self.force_mode}")
            return self.force_mode

        print("Selecting optimal execution mode...")

        # Best case: Full GPU + CPU hybrid processing
        if (self.capabilities['cudf_available'] and
            self.capabilities['cupy_available'] and
            self.capabilities['gpu_available']):
            print("Selected hybrid_gpu mode - GPU preprocessing with parallel CPU STL")
            return 'hybrid_gpu'

        # Good case: PyTorch GPU with sufficient memory
        elif (self.capabilities['torch_gpu_available'] and
              self.capabilities['gpu_memory_gb'] > 2):
            print("Selected hybrid_torch mode - PyTorch GPU preprocessing with parallel CPU STL")
            return 'hybrid_torch'

        # Standard case: Multi-core CPU processing
        elif self.capabilities['cpu_cores'] >= 4:
            print("Selected parallel_cpu mode - multi-core CPU processing")
            return 'parallel_cpu'

        # Fallback: Single-threaded processing
        else:
            print("Selected single_cpu mode - single-threaded processing")
            return 'single_cpu'

    def _initialize_compute_backend(self):
        """
        Initialize the appropriate compute backend based on selected mode

        This sets up the preprocessing engines and determines optimal worker counts.
        GPU preprocessing is initialized only if the mode requires it, preventing
        unnecessary resource allocation.
        """
        self.gpu_preprocessing = None
        self.torch_preprocessing = None

        # Initialize GPU preprocessing if selected mode supports it
        if 'gpu' in self.execution_mode:
            try:
                if 'cudf' in self.execution_mode or self.capabilities['cudf_available']:
                    import cudf
                    import cupy as cp
                    self.gpu_preprocessing = self._GPUPreprocessor()
                    print("GPU preprocessing with cuDF/CuPy initialized successfully")
                elif 'torch' in self.execution_mode:
                    import torch
                    self.torch_preprocessing = self._TorchPreprocessor()
                    print("GPU preprocessing with PyTorch initialized successfully")
            except Exception as e:
                print(f"Warning: GPU initialization failed, falling back to CPU: {e}")
                self.execution_mode = 'parallel_cpu'

        # Set optimal worker count for parallel processing
        # Cap at 8 workers to prevent memory exhaustion on systems with many cores
        if 'parallel' in self.execution_mode:
            self.max_workers = min(self.capabilities['cpu_cores'], 8)
            print(f"Parallel processing configured with {self.max_workers} workers")
        else:
            self.max_workers = 1
            print("Single-threaded processing configured")

    def _format_capabilities(self) -> str:
        """Format capabilities for display"""
        cap = self.capabilities
        return (f"CPU: {cap['cpu_cores']} cores, "
                f"RAM: {cap['memory_gb']}GB" +
                (f", GPU: {cap['gpu_memory_gb']}GB" if cap['gpu_available'] else ""))

    class _GPUPreprocessor:
        """
        GPU-accelerated preprocessing using cuDF/CuPy

        This class handles data preprocessing operations on GPU, which can be
        significantly faster for large datasets. Operations include:
        - DateTime conversion
        - Missing value handling through interpolation
        - Zero/negative value replacement
        """

        def __init__(self):
            import cudf
            import cupy as cp
            self.cudf = cudf
            self.cp = cp
            print("GPU preprocessor initialized - operations will run on GPU")

        def preprocess_dataframe(self, df: pd.DataFrame, date_col: str, value_cols: List[str]) -> pd.DataFrame:
            """
            GPU-accelerated data preprocessing

            This method performs the following operations on GPU:
            1. Convert pandas DataFrame to cuDF (GPU DataFrame)
            2. Handle datetime conversion on GPU
            3. Interpolate missing values on GPU
            4. Replace zero/negative values with quantile-based substitution
            5. Transfer results back to CPU

            The GPU processing is most beneficial for large datasets (>10k rows).
            """
            start_time = time.time()
            print("Starting GPU preprocessing operations...")

            # Move DataFrame to GPU memory
            gpu_df = self.cudf.from_pandas(df)
            print(f"Transferred {len(df)} rows to GPU memory")

            # GPU-accelerated datetime conversion - much faster than pandas for large datasets
            gpu_df[date_col] = self.cudf.to_datetime(gpu_df[date_col])

            # Process each value column on GPU
            for col in value_cols:
                if col in gpu_df.columns:
                    print(f"Processing column '{col}' on GPU...")

                    # Handle missing values using GPU-accelerated interpolation
                    if gpu_df[col].isnull().any():
                        print(f"  Interpolating missing values in '{col}'")
                        gpu_df[col] = gpu_df[col].interpolate()
                        gpu_df[col] = gpu_df[col].fillna(method='ffill').fillna(method='bfill')

                    # Handle zeros/negatives - STL requires positive values
                    # Replace with 1st percentile of positive values (robust outlier handling)
                    mask = gpu_df[col] <= 0
                    if mask.any():
                        positive_vals = gpu_df[col][gpu_df[col] > 0]
                        if len(positive_vals) > 0:
                            replacement = positive_vals.quantile(0.01)
                            gpu_df[col] = gpu_df[col].where(~mask, replacement)
                            print(f"  Replaced {mask.sum()} zero/negative values in '{col}' with {replacement:.2f}")

            # Transfer processed data back to CPU
            result_df = gpu_df.to_pandas()

            preprocessing_time = time.time() - start_time
            print(f"GPU preprocessing completed in {preprocessing_time:.2f} seconds")

            return result_df

    class _TorchPreprocessor:
        """
        PyTorch-based preprocessing for numerical operations

        This provides an alternative GPU acceleration path using PyTorch tensors.
        Useful when cuDF/CuPy is not available but PyTorch GPU support exists.
        """

        def __init__(self):
            import torch
            self.torch = torch
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print(f"PyTorch preprocessor initialized - using device: {self.device}")

        def preprocess_series(self, series: pd.Series) -> pd.Series:
            """
            GPU-accelerated series preprocessing using PyTorch

            This method:
            1. Extracts non-null values and converts to PyTorch tensor
            2. Moves tensor to GPU
            3. Performs zero/negative value replacement using GPU operations
            4. Transfers results back to CPU and reconstructs pandas Series
            """
            print(f"Processing series with PyTorch GPU acceleration...")

            # Extract valid (non-null) values
            mask = ~series.isnull()
            values = series.values[mask]

            if len(values) == 0:
                return series

            # Move to GPU as tensor
            tensor_vals = self.torch.tensor(values, dtype=self.torch.float32, device=self.device)

            # Handle zeros/negatives using GPU tensor operations
            positive_mask = tensor_vals > 0
            if positive_mask.any():
                positive_vals = tensor_vals[positive_mask]
                if len(positive_vals) > 0:
                    replacement = self.torch.quantile(positive_vals, 0.01)
                    tensor_vals = self.torch.where(tensor_vals <= 0, replacement, tensor_vals)
                    print(f"  Replaced zero/negative values with {replacement.item():.2f}")

            # Move back to CPU and reconstruct series
            processed_values = tensor_vals.cpu().numpy()
            result = series.copy()
            result.iloc[mask] = processed_values

            # Handle missing values with pandas (more reliable for edge cases)
            if result.isnull().any():
                result = result.interpolate().fillna(method='ffill').fillna(method='bfill')

            return result

    def _preprocess_series_smart(self, series: pd.Series) -> pd.Series:
        """
        Smart preprocessing that chooses optimal method based on data size and available resources

        Decision logic:
        - Use PyTorch GPU for large series (>1000 points) when available
        - Fall back to CPU preprocessing for smaller datasets or when GPU unavailable
        """
        if self.torch_preprocessing and len(series) > 1000:
            print("Using PyTorch GPU preprocessing for large series")
            return self.torch_preprocessing.preprocess_series(series)
        else:
            # Fallback to CPU preprocessing
            return self._preprocess_series_cpu(series)

    def _preprocess_series_cpu(self, series: pd.Series) -> pd.Series:
        """
        CPU-based preprocessing (fallback method)

        Performs the same operations as GPU methods but using pandas/numpy on CPU.
        This ensures compatibility when GPU resources are unavailable.
        """
        print("Using CPU preprocessing...")

        # Handle missing values using linear interpolation
        if series.isnull().any():
            print("  Interpolating missing values")
            series = series.interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')

        # Replace zero/negative values for STL compatibility
        if (series <= 0).any():
            positive_values = series[series > 0]
            if len(positive_values) > 0:
                replacement = positive_values.quantile(0.01)
                series = series.mask(series <= 0, replacement)
                print(f"  Replaced zero/negative values with {replacement:.2f}")

        return series

    def _stl_decompose_single(self, args: Tuple) -> Tuple[str, Dict]:
        """
        Single STL decomposition worker function

        This function performs the core STL (Seasonal and Trend decomposition using Loess)
        decomposition on a single time series. It's designed to be used both in
        sequential and parallel processing contexts.

        STL decomposes a time series into:
        - Trend: Long-term movement/direction
        - Seasonal: Repeating patterns (daily, weekly, etc.)
        - Residual: Random noise/anomalies after removing trend and seasonality

        The 'robust' parameter makes STL resistant to outliers.
        """
        series, col_name, params = args

        try:
            print(f"Performing STL decomposition for '{col_name}'...")

            # STL decomposition - always performed on CPU for stability
            # Seasonal parameter is adjusted to prevent overfitting on small datasets
            stl = STL(series,
                     seasonal=min(params['seasonal'], len(series)//10),
                     period=params['period'],
                     robust=True)

            result = stl.fit()

            # Deseasonalized series = trend + residual (seasonal component removed)
            # This is what we use for anomaly detection and modeling
            deseasonalized = result.trend + result.resid

            components = {
                'trend': result.trend,
                'seasonal': result.seasonal,
                'residual': result.resid,
                'deseasonalized': deseasonalized,
                'success': True
            }

            print(f"STL decomposition completed for '{col_name}'")
            return col_name, components

        except Exception as e:
            print(f"STL decomposition failed for '{col_name}': {str(e)}")
            return col_name, {'success': False, 'error': str(e)}

    def _determine_batch_strategy(self, df_size: int, num_cols: int) -> str:
        """
        Determine optimal processing strategy based on data size

        Strategy selection based on total data points (rows × columns):
        - chunk_parallel: For very large datasets that might not fit in memory
        - parallel: For medium datasets that benefit from parallel processing
        - sequential: For small datasets where parallelization overhead isn't worth it

        These thresholds are tuned based on typical performance characteristics.
        """
        total_series = df_size * num_cols

        print(f"Determining processing strategy for {total_series:,} total data points...")

        if total_series > 1000000:  # Large dataset
            print("Large dataset detected - using chunk-based parallel processing")
            return 'chunk_parallel'
        elif total_series > 100000:  # Medium dataset
            print("Medium dataset detected - using parallel processing")
            return 'parallel'
        else:  # Small dataset
            print("Small dataset detected - using sequential processing")
            return 'sequential'

    def deseasonalize_dataframe(self,
                               df: pd.DataFrame,
                               date_col: str,
                               value_cols: List[str]) -> pd.DataFrame:
        """
        Smart deseasonalization with automatic scaling

        This is the main entry point that orchestrates the entire deseasonalization process:
        1. Smart preprocessing (GPU or CPU based on capabilities)
        2. Strategy determination based on data size
        3. STL decomposition (parallel or sequential)
        4. Result integration and validation

        The method automatically adapts to available resources and data characteristics.
        """
        print("=" * 60)
        print(f"SMART DESEASONALIZATION STARTING")
        print(f"Processing {len(value_cols)} columns with {len(df):,} rows")
        print(f"Execution mode: {self.execution_mode}")
        print("=" * 60)

        start_time = time.time()

        # Step 1: Smart preprocessing
        # Use GPU preprocessing for large datasets when available
        if self.gpu_preprocessing and len(df) > 10000:
            print("Using GPU-accelerated preprocessing for large dataset...")
            df_processed = self.gpu_preprocessing.preprocess_dataframe(df, date_col, value_cols)
        else:
            print("Using CPU preprocessing...")
            df_processed = df.copy()
            df_processed[date_col] = pd.to_datetime(df_processed[date_col])

        # Prepare DataFrame for time series analysis
        print("Preparing data for time series analysis...")
        df_processed.set_index(date_col, inplace=True)
        df_processed.sort_index(inplace=True)
        print(f"Data prepared - date range: {df_processed.index.min()} to {df_processed.index.max()}")

        # Step 2: Determine processing strategy based on data characteristics
        strategy = self._determine_batch_strategy(len(df_processed), len(value_cols))

        # Step 3: Process columns using selected strategy
        print(f"Executing {strategy} processing strategy...")
        if strategy == 'parallel' and self.max_workers > 1:
            results = self._parallel_stl_processing(df_processed, value_cols)
        else:
            results = self._sequential_stl_processing(df_processed, value_cols)

        # Step 4: Integrate results into the main DataFrame
        print("Integrating decomposition results...")
        successful_cols = []
        failed_cols = []

        for col, components in results.items():
            if components.get('success', False):
                # Add deseasonalized column to DataFrame
                df_processed[f'{col}_deseasonalized'] = components['deseasonalized']

                # Store components for later use (recomposition, explanation)
                self.seasonal_components[col] = components['seasonal']
                self.decomposition_cache[col] = components
                successful_cols.append(col)
                print(f"Successfully processed: {col}")
            else:
                failed_cols.append(col)
                print(f"Failed to process: {col} - {components.get('error', 'Unknown error')}")

        total_time = time.time() - start_time

        # Performance summary
        print("\n" + "=" * 60)
        print("PROCESSING COMPLETED")
        print(f"Successful columns: {len(successful_cols)}/{len(value_cols)}")
        if failed_cols:
            print(f"Failed columns: {failed_cols}")
        print(f"Total processing time: {total_time:.2f} seconds")
        print(f"Throughput: {len(df)*len(successful_cols)/total_time:,.0f} data points/second")
        print("=" * 60)

        return df_processed.reset_index()

    def _parallel_stl_processing(self, df: pd.DataFrame, value_cols: List[str]) -> Dict:
        """
        Parallel STL processing using multiprocessing

        This method distributes STL decomposition across multiple CPU cores.
        Each worker process handles one column independently, which works well
        because STL decomposition of different series is embarrassingly parallel.

        Uses ProcessPoolExecutor instead of ThreadPoolExecutor because:
        1. STL computation is CPU-intensive (benefits from true parallelism)
        2. No need for shared memory between decompositions
        3. Python's GIL would limit ThreadPoolExecutor effectiveness
        """
        print(f"Starting parallel processing with {self.max_workers} worker processes...")

        # Prepare work items - only process columns with sufficient data
        work_items = []
        skipped_cols = []

        for col in value_cols:
            if df[col].count() >= self.params['min_periods']:
                series = self._preprocess_series_smart(df[col])
                work_items.append((series, col, self.params))
            else:
                skipped_cols.append(col)
                print(f"Skipping '{col}' - insufficient data ({df[col].count()} < {self.params['min_periods']} required)")

        if skipped_cols:
            print(f"Skipped {len(skipped_cols)} columns due to insufficient data")

        # Execute STL decompositions in parallel
        results = {}
        print(f"Processing {len(work_items)} columns in parallel...")

        with ProcessPoolExecutor(max_workers=self.max_workers) as executor:
            future_results = executor.map(self._stl_decompose_single, work_items)

            for col_name, components in future_results:
                results[col_name] = components
                status = "SUCCESS" if components.get('success', False) else "FAILED"
                print(f"  {col_name}: {status}")

        return results

    def _sequential_stl_processing(self, df: pd.DataFrame, value_cols: List[str]) -> Dict:
        """
        Sequential STL processing

        Processes columns one by one. Used for:
        1. Small datasets where parallel overhead isn't worth it
        2. Systems with limited CPU cores
        3. Memory-constrained environments
        4. Debugging purposes (easier to trace issues)
        """
        print("Starting sequential processing...")

        results = {}
        successful_count = 0

        for i, col in enumerate(value_cols, 1):
            print(f"[{i}/{len(value_cols)}] Processing column: '{col}'")

            # Check data sufficiency
            valid_points = df[col].count()
            if valid_points < self.params['min_periods']:
                print(f"  Skipping - insufficient data points ({valid_points} < {self.params['min_periods']} required)")
                results[col] = {'success': False, 'error': 'Insufficient data'}
                continue

            # Preprocess and decompose
            series = self._preprocess_series_smart(df[col])
            col_name, components = self._stl_decompose_single((series, col, self.params))
            results[col_name] = components

            if components.get('success', False):
                successful_count += 1
                print(f"  SUCCESS - Decomposition completed")
            else:
                print(f"  FAILED - {components.get('error', 'Unknown error')}")

        print(f"Sequential processing completed: {successful_count}/{len(value_cols)} successful")
        return results

    def get_modeling_columns(self, df: pd.DataFrame) -> List[str]:
        """
        Get deseasonalized columns for LSTM/ECOD modeling

        Returns column names that have been successfully deseasonalized and are
        ready for anomaly detection or forecasting models. These columns have
        seasonal patterns removed, making them more suitable for ML algorithms.
        """
        modeling_cols = [col for col in df.columns if '_deseasonalized' in col]
        print(f"Available modeling columns: {len(modeling_cols)}")
        for col in modeling_cols:
            print(f"  - {col}")
        return modeling_cols

    def recompose_for_explanation(self,
                                 deseasonalized_values: pd.Series,
                                 original_column_name: str) -> pd.Series:
        """
        Recompose deseasonalized values back to original scale

        This function is crucial for interpreting anomaly detection results.
        When an anomaly is detected in deseasonalized data, we need to add back
        the seasonal component to understand what the original value would be.

        Formula: recomposed_value = deseasonalized_value + seasonal_component
        """
        if original_column_name not in self.seasonal_components:
            raise ValueError(f"No seasonal component available for '{original_column_name}'. "
                           f"Ensure the column was successfully processed.")

        seasonal = self.seasonal_components[original_column_name]
        min_length = min(len(deseasonalized_values), len(seasonal))

        print(f"Recomposing {min_length} values for '{original_column_name}'...")

        recomposed = deseasonalized_values.iloc[:min_length] + seasonal.iloc[:min_length]

        print(f"Recomposition completed - value range: {recomposed.min():.2f} to {recomposed.max():.2f}")
        return recomposed

    def explain_anomaly(self,
                       row_index: int,
                       column_name: str,
                       original_dataframe: pd.DataFrame) -> Dict:
        """
        Generate business-friendly anomaly explanation

        When an anomaly is detected, this function provides a comprehensive explanation
        that breaks down the original value into its components:

        1. Original value: What was actually observed
        2. Seasonal component: How much was due to expected seasonal patterns
        3. Deseasonalized value: The underlying value after removing seasonality
        4. Business interpretation: Whether this represents normal or anomalous behavior

        This helps business users understand why something was flagged as anomalous.
        """
        if column_name not in self.decomposition_cache:
            raise ValueError(f"No decomposition data available for '{column_name}'. "
                           f"Ensure the column was successfully processed.")

        components = self.decomposition_cache[column_name]

        # Extract values for the specific row
        original_value = original_dataframe.iloc[row_index][column_name]
        deseasonalized_value = components['deseasonalized'].iloc[row_index]
        seasonal_component = components['seasonal'].iloc[row_index]

        # Calculate seasonal impact as percentage
        seasonal_impact_pct = (seasonal_component / original_value * 100) if original_value != 0 else 0

        # Determine if the deseasonalized value is anomalous
        # Using 2 standard deviations as threshold (covers ~95% of normal data)
        deseason_mean = components['deseasonalized'].mean()
        deseason_std = components['deseasonalized'].std()
        is_anomalous = abs(deseasonalized_value - deseason_mean) > 2 * deseason_std

        explanation = {
            'original_value': original_value,
            'deseasonalized_value': deseasonalized_value,
            'seasonal_component': seasonal_component,
            'seasonal_impact_percent': seasonal_impact_pct,
            'is_anomalous': is_anomalous,
            'business_explanation': (
                f"Original {column_name}: {original_value:.2f}. "
                f"Seasonal effect: {seasonal_component:+.2f} ({seasonal_impact_pct:.1f}%). "
                f"After removing seasonal patterns, the underlying value is {deseasonalized_value:.2f}, "
                f"which indicates {'anomalous' if is_anomalous else 'normal'} behavior "
                f"(deviation: {abs(deseasonalized_value - deseason_mean)/deseason_std:.1f} standard deviations)."
            )
        }

        print("Anomaly explanation generated:")
        print(f"  Original value: {original_value:.2f}")
        print(f"  Seasonal impact: {seasonal_impact_pct:.1f}%")
        print(f"  Deseasonalized: {deseasonalized_value:.2f}")
        print(f"  Status: {'ANOMALOUS' if is_anomalous else 'NORMAL'}")

        return explanation

    def get_performance_stats(self) -> Dict:
        """
        Get performance and capability statistics

        Provides a comprehensive overview of:
        1. Current execution mode and system capabilities
        2. Processing performance metrics
        3. Success/failure statistics

        Useful for performance monitoring and optimization decisions.
        """
        stats = {
            'execution_mode': self.execution_mode,
            'system_capabilities': self.capabilities,
            'max_workers': self.max_workers,
            'processed_columns': len(self.decomposition_cache),
            'successful_columns': len([k for k in self.decomposition_cache.keys()
                                     if self.decomposition_cache[k].get('success', False)]),
            'available_for_modeling': len([k for k in self.decomposition_cache.keys()
                                         if self.decomposition_cache[k].get('success', False)])
        }

        print("Performance Statistics:")
        print(f"  Execution mode: {stats['execution_mode']}")
        print(f"  Worker processes: {stats['max_workers']}")
        print(f"  Successfully processed columns: {stats['successful_columns']}/{stats['processed_columns']}")
        print(f"  Columns ready for modeling: {stats['available_for_modeling']}")

        return stats

**Rolling feature**

In [ ]:
def create_rolling_features(feature_df, windows=[7, 14, 30, 90],exclude_cols=None):
    """Create rolling window features for temporal analysis"""
    print(f"\n Creating rolling features with windows: {windows}")
    exclude_cols = exclude_cols or []
    feature_df = feature_df.sort_values('date').reset_index(drop=True)
    numeric_features = [col for col in feature_df.columns
                       if col not in ['date'] + exclude_cols
                       and feature_df[col].dtype in ['float64', 'int64']]
    # numeric_features = [col for col in feature_df.columns if col not in ['date']]

    # Use list to collect DataFrames, then concat once
    rolling_dfs = [feature_df]

    for window in windows:
        print(f"   Processing {window}-day rolling window...")
        window_features = {}

        for feature in numeric_features:
            if feature_df[feature].dtype in ['float64', 'int64']:
                # Calculate all rolling stats at once
                rolling_data = feature_df[feature].rolling(window)
                rolling_mean = rolling_data.mean()
                rolling_std = rolling_data.std()

                window_features[f"{feature}_rolling_{window}d_mean"] = rolling_mean
                window_features[f"{feature}_rolling_{window}d_std"] = rolling_std
                window_features[f"{feature}_rolling_{window}d_min"] = rolling_data.min()
                window_features[f"{feature}_rolling_{window}d_max"] = rolling_data.max()

                # Z-score features (current vs rolling window)
                current_value = feature_df[feature]
                window_features[f"{feature}_zscore_{window}d"] = (
                    (current_value - rolling_mean) / (rolling_std + 1e-8)
                )

                # Anomaly flags based on z-score threshold
                zscore_col = f"{feature}_zscore_{window}d"
                window_features[f"{feature}_anomaly_{window}d"] = (
                    window_features[zscore_col].abs() > 3.0
                ).astype(int)

                # Trend features
                window_features[f"{feature}_trend_{window}d"] = rolling_data.apply(
                    lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) == window else 0
                )

                # Difference from rolling mean
                window_features[f"{feature}_diff_{window}d"] = current_value - rolling_mean

                # Ratio to rolling mean
                window_features[f"{feature}_ratio_{window}d"] = (
                    current_value / (rolling_mean + 1e-8)
                )

        # Add window features as DataFrame
        rolling_dfs.append(pd.DataFrame(window_features))

    # Concatenate all at once
    rolling_features = pd.concat(rolling_dfs, axis=1)
    print(f" Rolling features created: {rolling_features.shape[1]} total features")
    return rolling_features


**Enhanced feature engineering**

In [ ]:
class FeatureEngineer:
    """Enhanced feature engineering for LSTM anomaly detection"""

    def __init__(self, zscore_threshold=3.0):
        self.zscore_threshold = zscore_threshold
        print(f"[INIT] FeatureEngineer with threshold={zscore_threshold}")

    def create_features(self, profile_df, process_date_col='date', target_cols=None):
        """Create LSTM-optimized features"""
        print(f"[FEATURES] Enhancing {profile_df.shape[1]} features for LSTM")

        result_df = profile_df.copy()

        # Find profile columns
        profile_cols = [col for col in profile_df.columns if any(
            suffix in col for suffix in ['_zscore_', '_anomaly_', '_diff_', '_ratio_']
        )]

        # Create combined anomaly scores
        result_df = self._create_combined_scores(result_df, profile_cols)

        # Add time features
        result_df = self._add_time_features(result_df, process_date_col)

        # Target-specific features
        if target_cols:
            result_df = self._create_target_features(result_df, target_cols)

        print(f"[COMPLETE] Added {result_df.shape[1] - profile_df.shape[1]} enhanced features")
        return result_df

    def _create_combined_scores(self, df, profile_cols):
        """Combined anomaly scores across windows"""
        result_df = df.copy()

        # Get base feature names
        base_features = set()
        for col in profile_cols:
            if '_zscore_' in col:
                base_name = col.split('_zscore_')[0]
                base_features.add(base_name)

        print(f"[COMBINED] Creating scores for {len(base_features)} features")

        for base in list(base_features)[:10]:  # Limit for performance
            zscore_cols = [col for col in profile_cols if col.startswith(f"{base}_zscore_")]

            if len(zscore_cols) > 1:
                # Combined anomaly score (max absolute z-score across windows)
                result_df[f"{base}_max_zscore"] = df[zscore_cols].abs().max(axis=1)

                # Combined flag
                result_df[f"{base}_any_anomaly"] = (
                    result_df[f"{base}_max_zscore"] > self.zscore_threshold
                ).astype(int)

        return result_df

    def _add_time_features(self, df, date_col):
        """Calendar features for LSTM temporal patterns"""
        result_df = df.copy()

        if date_col in df.columns:
            result_df[date_col] = pd.to_datetime(result_df[date_col])
            result_df['day_of_week'] = result_df[date_col].dt.dayofweek
            result_df['is_month_end'] = result_df[date_col].dt.is_month_end.astype(int)
            result_df['is_quarter_end'] = result_df[date_col].dt.is_quarter_end.astype(int)

        return result_df

    def _create_target_features(self, df, target_cols):
        """Target-specific aggregated features"""
        result_df = df.copy()

        for target in target_cols:
            anomaly_flags = [col for col in df.columns if f"{target}_anomaly_" in col]

            if anomaly_flags:
                result_df[f"{target}_total_anomalies"] = result_df[anomaly_flags].sum(axis=1)
                result_df[f"{target}_any_anomaly"] = result_df[anomaly_flags].max(axis=1)

        return result_df

def handle_nulls_for_ml(df):
    """Handle null values for LSTM/PyOD models"""
    print(f"   Input shape: {df.shape}")

    # 1. Drop columns where ALL values are null
    all_null_cols = df.columns[df.isnull().all()].tolist()
    if all_null_cols:
        print(f"    Dropping {len(all_null_cols)} all-null columns: {all_null_cols[:5]}...")
        df = df.drop(columns=all_null_cols)

    # 2. Drop columns with >95% nulls as most of the population is null
    high_null_cols = df.columns[df.isnull().mean() > 0.95].tolist()
    if high_null_cols:
        print(f"    Dropping {len(high_null_cols)} high-null columns (>95%): {high_null_cols[:5]}...")
        df = df.drop(columns=high_null_cols)

    # 3. Fill remaining nulls
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    non_numeric_cols = [col for col in df.columns if col not in numeric_cols and col != 'date']

    print(f"   Filling nulls in {len(numeric_cols)} numeric columns...")
    df[numeric_cols] = df[numeric_cols].fillna(0)  # Fill with 0 for metrics

    if non_numeric_cols:
        print(f"   Filling nulls in {len(non_numeric_cols)} non-numeric columns...")
        df[non_numeric_cols] = df[non_numeric_cols].fillna('unknown')

    print(f"   Final shape: {df.shape}")
    print(f"   Remaining nulls: {df.isnull().sum().sum()}")

    return df

**Feature engineering main function**

In [ ]:
def prepare_features_for_model(df, zscore_threshold=3.0, target_cols=None,
                              enhanced=False, exclude_original_cols=None):
    """
    Main function to prepare dataset-level features for anomaly detection

    Args:
        exclude_original_cols: List of original columns to exclude from feature engineering
                              (typically the columns used for deseasonalization)
    """
    print("DATASET-LEVEL ANOMALY DETECTION FEATURE PREPARATION")
    print("=" * 60)

    # Track original columns to exclude
    original_cols_to_exclude = exclude_original_cols or []
    if original_cols_to_exclude:
        print(f"Excluding {len(original_cols_to_exclude)} original columns from feature engineering")

    # Initialize profiler with exclusions
    profiler = DataQualityProfiler(df, exclude_cols=original_cols_to_exclude)

    # Generate base features (daily dataset profiles)
    print("\nSTEP 1: Calculating daily DQ metrics...")
    feature_df = profiler.profile_dataset_daily()

    if feature_df.empty:
        print("No features generated")
        return pd.DataFrame()

    # Add rolling window features with z-scores
    print("\nSTEP 2: Creating rolling features...")
    final_features = create_rolling_features(feature_df, exclude_cols=original_cols_to_exclude)

    print("\nSTEP 2.5: Creating lag features...")
    final_features = create_lag_features(
        final_features,
        lag_features=[1, 2, 3],
        base_columns_only=True,
        exclude_cols=original_cols_to_exclude
    )

    # Handle null values for ML models
    print("\nSTEP 3: Handling null values...")
    final_features = handle_nulls_for_ml(final_features)

    if enhanced:
        print(f"\nSTEP 4: Enhanced feature engineering (threshold={zscore_threshold})...")
        feature_engineer = FeatureEngineer(
            zscore_threshold=zscore_threshold,
            exclude_original_cols=original_cols_to_exclude
        )
        final_features = feature_engineer.create_features(
            final_features,
            process_date_col='date',
            target_cols=target_cols
        )

    print(f"\nFEATURE PREPARATION COMPLETE!")
    print(f"Final shape: {final_features.shape[0]} days × {final_features.shape[1]} features")

    return final_features

**POst anamolies detected revert the changes back to original data**

In [ ]:
# # After anomaly detection
# anomaly_rows = detected_anomalies_df

# # For each anomaly, explain in original terms
# for idx, row in anomaly_rows.iterrows():
#     explanation = deseasonalizer.explain_anomaly(
#         row_index=idx,
#         column_name='transaction_amount',  # original column name
#         original_dataframe=original_df
#     )

#     # Report shows:
#     # - Original value: $52,000
#     # - Seasonal effect: +$4,000 (holiday boost)
#     # - Underlying anomaly: $48,000 (after removing seasonality)
#     print(explanation['business_explanation'])

In [ ]:
deseasonalizer = SmartSTLDeseasonalizer(frequency='daily')

Detecting system capabilities...
System specs - CPU: 12 cores, RAM: 83.5GB
cuDF (GPU DataFrame operations) detected
CuPy (GPU arrays) detected and working - GPU memory: 0.0GB
PyTorch GPU detected - GPU memory: 39.6GB
Selecting optimal execution mode...
Selected hybrid_gpu mode - GPU preprocessing with parallel CPU STL
Initializing compute backend...
GPU preprocessor initialized - operations will run on GPU
GPU preprocessing with cuDF/CuPy initialized successfully
Single-threaded processing configured
Smart STL Deseasonalizer initialized successfully
Mode: hybrid_gpu
Frequency: daily
System capabilities: CPU: 12 cores, RAM: 83.5GB, GPU: 39.6GB
STL parameters: {'seasonal': 15, 'period': 7, 'min_periods': 21}


In [ ]:
start=time.time()
target_columns=['aimp', 'amud', 'arnd']
processed_df = deseasonalizer.deseasonalize_dataframe(df, 'timestamp', target_columns)
print(f"Time: {(time.time()-start)/60:.6f} minutes")

SMART DESEASONALIZATION STARTING
Processing 3 columns with 15,368,002 rows
Execution mode: hybrid_gpu
Using GPU-accelerated preprocessing for large dataset...
Starting GPU preprocessing operations...
Transferred 15368002 rows to GPU memory
Processing column 'aimp' on GPU...
  Replaced 15212568 zero/negative values in 'aimp' with 0.96
Processing column 'amud' on GPU...
  Replaced 8972892 zero/negative values in 'amud' with 0.98
Processing column 'arnd' on GPU...
GPU preprocessing completed in 4.46 seconds
Preparing data for time series analysis...
Data prepared - date range: 2022-09-03 00:00:00 to 2023-02-27 20:53:19
Determining processing strategy for 46,104,006 total data points...
Large dataset detected - using chunk-based parallel processing
Executing chunk_parallel processing strategy...
Starting sequential processing...
[1/3] Processing column: 'aimp'
Using CPU preprocessing...
Performing STL decomposition for 'aimp'...
STL decomposition completed for 'aimp'
  SUCCESS - Decomposit

In [ ]:
processed_df[["arnd","arnd_deseasonalized"]].head(10)

,arnd,arnd_deseasonalized
0,20.000000,20.102121
1,20.080031,20.082107
2,20.276562,20.142370
3,20.730938,20.585734
4,21.118101,21.110047
5,20.911718,20.962739
6,21.110398,21.242095
7,21.078446,21.179786
8,21.056689,21.057175
9,21.143147,21.010024


In [ ]:
print(target_columns)
# features = prepare_features_for_model(processed_df,exclude_original_cols=target_columns)
features = prepare_features_for_model(processed_df)

Streaming output truncated to the last 5000 lines.
Distribution metrics for adbr on 2023-01-31
Quantile metrics for adbr on 2023-01-31
Entropy calculation for adbr on 2023-01-31
Entropy calculated: 0.999
Mutual information: adbr vs aimp on 2023-01-31
MI calculated: 0.000
Processing column: adfl
Basic metrics for adfl on 2023-01-31
Distribution metrics for adfl on 2023-01-31
Quantile metrics for adfl on 2023-01-31
Entropy calculation for adfl on 2023-01-31
Entropy calculated: 0.963
Mutual information: adfl vs aimp on 2023-01-31
MI calculated: 0.000
Processing column: bed1
Basic metrics for bed1 on 2023-01-31
Distribution metrics for bed1 on 2023-01-31
Quantile metrics for bed1 on 2023-01-31
Entropy calculation for bed1 on 2023-01-31
Entropy calculated: 4.560
Mutual information: bed1 vs aimp on 2023-01-31
MI calculated: 0.000
Processing column: bed2
Basic metrics for bed2 on 2023-01-31
Distribution metrics for bed2 on 2023-01-31
Quantile metrics for bed2 on 2023-01-31
Entropy calculation

In [ ]:
features.columns

Index(['date', 'total_records', 'dataset_completeness', 'aimp_Mean',
       'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum',
       'aimp_count', 'aimp_Completeness',
       ...
       'arnd_deseasonalized_Correlation_amud_lag_3d',
       'arnd_deseasonalized_Correlation_amud_diff_lag_3d',
       'arnd_deseasonalized_Correlation_arnd_lag_3d',
       'arnd_deseasonalized_Correlation_arnd_diff_lag_3d',
       'arnd_deseasonalized_Correlation_asin1_lag_3d',
       'arnd_deseasonalized_Correlation_asin1_diff_lag_3d',
       'arnd_deseasonalized_Correlation_asin2_lag_3d',
       'arnd_deseasonalized_Correlation_asin2_diff_lag_3d',
       'arnd_deseasonalized_MutualInformation_lag_3d',
       'arnd_deseasonalized_MutualInformation_diff_lag_3d'],
      dtype='object', length=24390)

**Data masssaging for Histogram columns**

In [ ]:
import pandas as pd
import numpy as np

def extract_histogram_columns(df):
    """Extract histogram column names from DataFrame"""
    histogram_cols = [col for col in df.columns if 'Histogram' in col]
    print(f"Found {len(histogram_cols)} histogram columns")
    return histogram_cols

def transform_histogram_columns(df, histogram_columns):
    """
    Transform histogram string columns into numeric features

    Args:
        df: DataFrame containing histogram columns
        histogram_columns: List of histogram column names to transform

    Returns:
        DataFrame with new numeric features (original columns kept)
    """
    print(f"Transforming {len(histogram_columns)} histogram columns...")

    result_df = df.copy()

    for col in histogram_columns:
        base_name = col.replace('_Histogram', '')
        print(f"Processing {col} -> {base_name}_hist_*")

        try:
            # Parse histogram strings to arrays
            hist_arrays = df[col].apply(
                lambda x: np.array([int(i) for i in str(x).split(',')]) if pd.notna(x) else np.array([0])
            )

            # Extract features
            result_df[f"{base_name}_hist_mean"] = hist_arrays.apply(np.mean)
            result_df[f"{base_name}_hist_std"] = hist_arrays.apply(np.std)
            result_df[f"{base_name}_hist_sum"] = hist_arrays.apply(np.sum)
            result_df[f"{base_name}_hist_max"] = hist_arrays.apply(np.max)
            result_df[f"{base_name}_hist_entropy"] = hist_arrays.apply(
                lambda x: -np.sum((p := x/x.sum()) * np.log2(p + 1e-12)) if x.sum() > 0 else 0
            )

        except Exception as e:
            print(f"Error transforming {col}: {e}")

    print(f"Created {len(histogram_columns) * 5} new features")
    return result_df

**extract histogram feature**

In [ ]:

# Method 1: Auto-extract histogram columns
histogram_cols = extract_histogram_columns(features)



Found 22 histogram columns


In [ ]:
features[histogram_cols].head(2)

,aimp_Histogram,amud_Histogram,arnd_Histogram,asin1_Histogram,asin2_Histogram,adbr_Histogram,adfl_Histogram,bed1_Histogram,bed2_Histogram,bfo1_Histogram,...,bso2_Histogram,bso3_Histogram,ced1_Histogram,cfo1_Histogram,cso1_Histogram,y_Histogram,category_Histogram,aimp_deseasonalized_Histogram,amud_deseasonalized_Histogram,arnd_deseasonalized_Histogram
0,"0,0,0,0,0,86400,0,0,0,0","41341,13035,13680,5170,3141,2945,5125,1256,0,707","36307,16980,12719,7956,3845,3206,1563,1698,132...","5009,5059,5167,5341,5604,5996,6594,7595,9624,3...","16093,7095,5800,5255,5034,6038,6306,6960,8514,...","43253,0,0,0,0,0,0,0,0,43147","33940,0,0,0,0,0,0,0,0,52460","26217,23378,18192,10911,5013,1829,627,167,49,17","65554,9395,6123,3669,996,438,163,33,19,10","35766,10761,17484,9289,6507,2809,1339,1237,717...",...,"6286,8736,7356,7053,7663,13380,12912,10971,718...","674,1370,5111,12523,15780,16136,17137,11979,48...","644,6736,16715,24449,18879,10295,5723,1968,560...","113,837,1615,3577,8959,14288,18164,27794,9310,...","165,247,48,1779,9459,16328,22018,21218,13059,2079","0,0,0,0,0,86400,0,0,0,0","0,0,0,0,0,86400,0,0,0,0","0,0,0,0,0,86400,0,0,0,0","41346,13053,13656,7270,1047,6581,1487,1253,3,704","36173,17189,12773,7939,3868,3170,1572,1659,134..."
1,"0,0,0,0,0,86400,0,0,0,0","76118,4993,0,1422,0,2152,1523,0,0,192","1100,4349,9538,9910,9225,11727,9599,9328,10077...","6694,6532,6490,6558,6750,7099,7684,8704,10775,...","16082,7095,5800,5252,5047,6042,6303,6960,8514,...","45701,0,0,0,0,0,0,0,0,40699","33610,0,0,0,0,0,0,0,0,52790","24543,23927,18509,10726,5363,2289,796,188,48,11","64695,9036,6135,4373,1258,554,243,76,23,7","33038,7158,6296,6520,7205,6192,6801,4745,4256,...",...,"6416,8806,7086,6931,7424,13243,13337,11082,735...","1930,6768,13521,15272,13037,11734,11289,7889,4...","1989,9963,14032,18919,17250,12626,7090,3045,10...","748,3126,5558,8520,8422,9707,10216,16443,19698...","1860,4017,6571,11412,15642,15028,12837,10501,6...","0,0,0,0,0,86400,0,0,0,0","0,0,0,0,0,86400,0,0,0,0","0,0,0,0,0,86400,0,0,0,0","76104,4993,18,1425,20,2129,1517,3,3,188","1139,4575,9608,10025,9552,11866,9490,9471,1038..."


In [ ]:
transformed_df = transform_histogram_columns(features, histogram_cols)
print(f"Before transfomration number of col {len(features.columns)} \n  transfomration number of col {len(transformed_df.columns)}+ \
\n number of new columns {len(transformed_df.columns)-len(features.columns) }")

Transforming 22 histogram columns...
Processing aimp_Histogram -> aimp_hist_*
Processing amud_Histogram -> amud_hist_*
Processing arnd_Histogram -> arnd_hist_*
Processing asin1_Histogram -> asin1_hist_*
Processing asin2_Histogram -> asin2_hist_*
Processing adbr_Histogram -> adbr_hist_*
Processing adfl_Histogram -> adfl_hist_*
Processing bed1_Histogram -> bed1_hist_*
Processing bed2_Histogram -> bed2_hist_*
Processing bfo1_Histogram -> bfo1_hist_*
Processing bfo2_Histogram -> bfo2_hist_*
Processing bso1_Histogram -> bso1_hist_*
Processing bso2_Histogram -> bso2_hist_*
Processing bso3_Histogram -> bso3_hist_*
Processing ced1_Histogram -> ced1_hist_*
Processing cfo1_Histogram -> cfo1_hist_*
Processing cso1_Histogram -> cso1_hist_*
Processing y_Histogram -> y_hist_*
Processing category_Histogram -> category_hist_*
Processing aimp_deseasonalized_Histogram -> aimp_deseasonalized_hist_*
Processing amud_deseasonalized_Histogram -> amud_deseasonalized_hist_*
Processing arnd_deseasonalized_Histo

In [ ]:
transformed_df.columns

Index(['date', 'total_records', 'dataset_completeness', 'aimp_Mean',
       'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum',
       'aimp_count', 'aimp_Completeness',
       ...
       'amud_deseasonalized_hist_mean', 'amud_deseasonalized_hist_std',
       'amud_deseasonalized_hist_sum', 'amud_deseasonalized_hist_max',
       'amud_deseasonalized_hist_entropy', 'arnd_deseasonalized_hist_mean',
       'arnd_deseasonalized_hist_std', 'arnd_deseasonalized_hist_sum',
       'arnd_deseasonalized_hist_max', 'arnd_deseasonalized_hist_entropy'],
      dtype='object', length=24500)

In [ ]:
gdrive_path

'/content/drive/MyDrive/Anamolies/CATS_dataset'

In [ ]:
transformed_dfs=pd.read_csv(gdrive_path+"/transformed_df.csv")
transformed_df=transformed_dfs

In [ ]:
transformed_df.columns

Index(['date', 'total_records', 'dataset_completeness', 'aimp_Mean',
       'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum',
       'aimp_count', 'aimp_Completeness',
       ...
       'amud_deseasonalized_hist_mean', 'amud_deseasonalized_hist_std',
       'amud_deseasonalized_hist_sum', 'amud_deseasonalized_hist_max',
       'amud_deseasonalized_hist_entropy', 'arnd_deseasonalized_hist_mean',
       'arnd_deseasonalized_hist_std', 'arnd_deseasonalized_hist_sum',
       'arnd_deseasonalized_hist_max', 'arnd_deseasonalized_hist_entropy'],
      dtype='object', length=30905)

**target_column profiling feature selection**

In [ ]:
def _get_target_features(df: pd.DataFrame, target_columns: List[str]) -> List[str]:
    """Get relevant features for target columns with fallbacks."""

    # Priority 1: Base profiling metrics
    base_suffixes = ['_Mean', '_StandardDeviation', '_ApproxQuantiles', '_CountDistinct']

    # Priority 2: Engineered features (fallback)
    engineered_suffixes = ['_Mean_zscore_7d', '_StandardDeviation_zscore_7d',
                          '_Completeness_anomaly_7d', '_CountDistinct_anomaly_7d']

    available_cols = set(df.columns)
    features = []

    for col in target_columns:
        # Try base metrics first
        base_features = [f"{col}{suffix}" for suffix in base_suffixes
                        if f"{col}{suffix}" in available_cols]

        if base_features:
            features.extend(base_features)
        else:
            # Fallback to engineered features
            engineered_features = [f"{col}{suffix}" for suffix in engineered_suffixes
                                 if f"{col}{suffix}" in available_cols]
            features.extend(engineered_features[:4])  # Max 4 per column

    return features

In [ ]:
# Given
target_columns = ['aimp', 'amud', 'arnd']

# Get matching columns that start with any of the target prefixes
matching_cols = [c for c in transformed_df.columns if any(c.startswith(p) for p in target_columns)]

# Example: subset the dataframe if needed
subset_df = transformed_df[matching_cols]
target_columns=matching_cols

print(target_columns)
print(f"source columns {soure_columns}")


['aimp_Mean', 'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum', 'aimp_count', 'aimp_Completeness', 'aimp_missingCount', 'aimp_zerocount', 'aimp_zeroCount', 'aimp_zerocountpercentage', 'aimp_negativecount', 'aimp_negativeCount', 'aimp_negativecountpercentage', 'aimp_CountDistinct', 'aimp_Uniqueness', 'aimp_UniqueValueRatio', 'aimp_Distinctness', 'aimp_Histogram', 'aimp_DataType', 'aimp_ApproxQuantiles_0.1', 'aimp_ApproxQuantiles_0.25', 'aimp_ApproxQuantiles_0.5', 'aimp_ApproxQuantiles_0.75', 'aimp_ApproxQuantiles_0.9', 'aimp_Entropy', 'aimp_Size', 'aimp_MaxLength', 'aimp_MinLength', 'aimp_Correlation_amud', 'aimp_Correlation_arnd', 'aimp_Correlation_asin1', 'aimp_Correlation_asin2', 'amud_Mean', 'amud_Minimum', 'amud_Maximum', 'amud_StandardDeviation', 'amud_Sum', 'amud_count', 'amud_Completeness', 'amud_missingCount', 'amud_zerocount', 'amud_zeroCount', 'amud_zerocountpercentage', 'amud_negativecount', 'amud_negativeCount', 'amud_negativecountpercentage', 'amud_Coun

**Drop all the base columns**

In [ ]:
# Excluding columns (ignores any that don't exist to avoid errors)
transformeddf = transformed_df.drop(columns=[c for c in soure_columns
                                             if c in transformed_df.columns])



**Fix the datatype**

In [ ]:
import pandas as pd
import numpy as np

def fix_dataframe_dtypes_enhanced(df):
    """
    Enhanced DataFrame type fixing that handles comma-separated values.
    """
    print("Fixing DataFrame data types (enhanced)...")

    fixed_df = df.copy()
    conversions = {'numeric': 0, 'comma_separated_fixed': 0, 'kept_string': 0, 'kept_numeric': 0, 'dropped': 0}
    dropped_columns = []

    for col in df.columns:
        if col == 'date':
            continue

        if pd.api.types.is_numeric_dtype(df[col]):
            conversions['kept_numeric'] += 1
            continue

        if df[col].dtype == 'object':
            # Sample first few non-null values to detect pattern
            sample_values = df[col].dropna().head(10)

            if len(sample_values) == 0:
                # Empty column - drop it
                fixed_df.drop(columns=[col], inplace=True)
                dropped_columns.append(col)
                conversions['dropped'] += 1
                continue

            # Check if values contain commas (comma-separated)
            has_commas = any(',' in str(val) for val in sample_values)

            if has_commas:
                # Extract first value from comma-separated strings
                def extract_first_numeric(x):
                    if pd.isna(x):
                        return np.nan
                    try:
                        first_val = str(x).split(',')[0]
                        return float(first_val)
                    except (ValueError, IndexError):
                        return np.nan

                fixed_df[col] = df[col].apply(extract_first_numeric)
                conversions['comma_separated_fixed'] += 1
                print(f"  {col}: comma-separated → numeric (using first value)")

            else:
                # Try regular numeric conversion
                try:
                    fixed_df[col] = pd.to_numeric(df[col], errors='raise')
                    conversions['numeric'] += 1
                    print(f"  {col}: object → numeric")
                except (ValueError, TypeError):
                    # Check if it's actually needed for analysis
                    unique_count = df[col].nunique()
                    if unique_count < 10 and col not in ['date']:  # Likely categorical
                        fixed_df.drop(columns=[col], inplace=True)
                        dropped_columns.append(col)
                        conversions['dropped'] += 1
                        print(f"  {col}: dropped (categorical/low variance)")
                    else:
                        conversions['kept_string'] += 1
                        print(f"  {col}: keeping as string")
        else:
            conversions['kept_numeric'] += 1

    print(f"Enhanced conversion summary:")
    print(f"  Converted to numeric: {conversions['numeric']}")
    print(f"  Fixed comma-separated: {conversions['comma_separated_fixed']}")
    print(f"  Already numeric: {conversions['kept_numeric']}")
    print(f"  Kept as string: {conversions['kept_string']}")
    print(f"  Dropped columns: {conversions['dropped']}")

    if dropped_columns:
        print(f"  Dropped: {dropped_columns[:5]}{'...' if len(dropped_columns) > 5 else ''}")

    return fixed_df

# Usage
def create_clean_transformed_df(original_df):
    """Create properly cleaned transformed DataFrame."""
    # Your existing transformation
    transformed_df = prepare_features_for_model(original_df, enhanced=True)

    # Enhanced cleaning
    cleaned_df = fix_dataframe_dtypes_enhanced(transformed_df)

    return cleaned_df

In [ ]:
transformed_datatype_df = fix_dataframe_dtypes_enhanced(transformeddf)

Fixing DataFrame data types (enhanced)...
  aimp_Histogram: comma-separated → numeric (using first value)
  aimp_DataType: dropped (categorical/low variance)
  amud_Histogram: comma-separated → numeric (using first value)
  amud_DataType: dropped (categorical/low variance)
  arnd_Histogram: comma-separated → numeric (using first value)
  arnd_DataType: dropped (categorical/low variance)
  asin1_Histogram: comma-separated → numeric (using first value)
  asin1_DataType: dropped (categorical/low variance)
  asin2_Histogram: comma-separated → numeric (using first value)
  asin2_DataType: dropped (categorical/low variance)
  adbr_Histogram: comma-separated → numeric (using first value)
  adbr_DataType: dropped (categorical/low variance)
  adfl_Histogram: comma-separated → numeric (using first value)
  adfl_DataType: dropped (categorical/low variance)
  bed1_Histogram: comma-separated → numeric (using first value)
  bed1_DataType: dropped (categorical/low variance)
  bed2_Histogram: comma-se

In [ ]:
# transformeddf[target_features]

# transformed_df[target_features].to_csv("/content/sample_datatarget_features.csv", index=False)




**Feature Selection**

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

class FeatureSelector:
    """
    Production-ready feature selector using PCA and optional Autoencoder.

    Selection Process:
    1. Variance filtering (removes low-variance features)
    2. Correlation filtering (removes highly correlated features)
    3. PCA analysis (finds optimal dimensionality)
    4. Autoencoder ranking (optional - ranks by reconstruction importance)
    5. Final selection (top N features by importance)

    Key Parameters:
    - target_features: Number of final features to select
    - use_autoencoder: Whether to use autoencoder for feature ranking
    - variance_threshold: Minimum variance threshold for filtering
    - correlation_threshold: Maximum correlation threshold for filtering
    """

    def __init__(
        self,
        target_features: Optional[int] = None,
        use_autoencoder: bool = True,
        variance_threshold: float = 0.01,
        correlation_threshold: float = 0.95,
        pca_variance_ratio: float = 0.95,
        random_state: int = 42
    ):
        """
        Initialize feature selector with configurable parameters.

        Args:
            target_features: Final number of features (None = auto-determine)
            use_autoencoder: Use autoencoder for feature ranking
            variance_threshold: Minimum variance for feature retention
            correlation_threshold: Maximum correlation for feature retention
            pca_variance_ratio: PCA variance ratio for optimal components
            random_state: Random seed for reproducibility
        """
        self.target_features = target_features
        self.use_autoencoder = use_autoencoder
        self.variance_threshold = variance_threshold
        self.correlation_threshold = correlation_threshold
        self.pca_variance_ratio = pca_variance_ratio
        self.random_state = random_state

        # Internal state
        self.selected_features_ = []
        self.feature_scores_ = {}
        self.selection_stats_ = {}
        self.scaler_ = StandardScaler()
        self.pca_ = None

        print("Production Feature Selector initialized")
        print(f"  Target features: {target_features or 'auto-determine'}")
        print(f"  Use autoencoder: {use_autoencoder}")
        print(f"  Variance threshold: {variance_threshold}")
        print(f"  Correlation threshold: {correlation_threshold}")

    def fit_transform(
        self,
        df: pd.DataFrame,
        target_columns: Optional[List[str]] = None,
        exclude_columns: Optional[List[str]] = None
    ) -> List[str]:
        """
        Main method to select optimal features from dataframe.

        Args:
            df: Input dataframe with all features
            target_columns: Base columns to focus on (None = use all numeric)
            exclude_columns: Columns to exclude from selection

        Returns:
            List of selected feature names for downstream models
        """
        print(f"\nStarting feature selection on dataframe: {df.shape}")

        # Step 1: Get candidate features
        candidate_features = self._extract_candidate_features(
            df, target_columns, exclude_columns or ['date']
        )
        print(f"Candidate features extracted: {len(candidate_features)}")

        # Step 2: Variance filtering
        variance_filtered = self._apply_variance_filtering(df, candidate_features)
        print(f"After variance filtering: {len(variance_filtered)}")

        # Step 3: Correlation filtering
        correlation_filtered = self._apply_correlation_filtering(df, variance_filtered)
        print(f"After correlation filtering: {len(correlation_filtered)}")

        # Step 4: PCA analysis for optimal feature count
        optimal_count = self._determine_optimal_feature_count(df, correlation_filtered)
        print(f"Optimal feature count determined: {optimal_count}")

        # Step 5: Feature importance ranking
        if self.use_autoencoder:
            feature_scores = self._rank_features_autoencoder(df, correlation_filtered)
            print("Feature ranking completed using autoencoder")
        else:
            feature_scores = self._rank_features_statistical(df, correlation_filtered)
            print("Feature ranking completed using statistical methods")

        # Step 6: Final feature selection
        final_features = self._select_top_features(
            correlation_filtered, feature_scores, optimal_count
        )

        # Store results
        self.selected_features_ = final_features
        self.feature_scores_ = feature_scores
        self._store_selection_statistics(candidate_features, final_features)

        print(f"Feature selection complete: {len(final_features)} features selected")
        print(f"Reduction ratio: {len(final_features)/len(candidate_features):.3f}")

        return final_features

    def _extract_candidate_features(
        self,
        df: pd.DataFrame,
        target_columns: Optional[List[str]],
        exclude_columns: List[str]
    ) -> List[str]:
        """Extract candidate features based on target columns or all numeric."""

        # Get all numeric columns
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

        # Remove excluded columns
        for col in exclude_columns:
            if col in numeric_columns:
                numeric_columns.remove(col)

        # Filter by target columns if specified
        if target_columns:
            candidate_features = []
            for target_col in target_columns:
                # Get features starting with target column name
                related_features = [
                    col for col in numeric_columns
                    if col.startswith(f"{target_col}_")
                ]
                candidate_features.extend(related_features)

            candidate_features = list(set(candidate_features))
            print(f"Target-based selection for: {target_columns}")
        else:
            candidate_features = numeric_columns
            print("Using all numeric features")

        return candidate_features

    def _apply_variance_filtering(
        self,
        df: pd.DataFrame,
        features: List[str]
    ) -> List[str]:
        """Remove features with low variance."""

        try:
            # Prepare data
            feature_data = df[features].fillna(0)

            # Apply variance threshold
            variance_selector = VarianceThreshold(threshold=self.variance_threshold)
            variance_selector.fit(feature_data)

            # Get selected features
            selected_mask = variance_selector.get_support()
            filtered_features = [features[i] for i, keep in enumerate(selected_mask) if keep]

            removed_count = len(features) - len(filtered_features)
            print(f"  Removed {removed_count} low-variance features")

            return filtered_features

        except Exception as e:
            print(f"  Variance filtering failed: {e}, keeping all features")
            return features

    def _apply_correlation_filtering(
        self,
        df: pd.DataFrame,
        features: List[str]
    ) -> List[str]:
        """Remove highly correlated features."""

        try:
            # Prepare data
            feature_data = df[features].fillna(0)

            # Calculate correlation matrix
            correlation_matrix = feature_data.corr().abs()

            # Find features to drop (upper triangle)
            upper_triangle = correlation_matrix.where(
                np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
            )

            # Identify highly correlated features
            to_drop = [
                column for column in upper_triangle.columns
                if any(upper_triangle[column] > self.correlation_threshold)
            ]

            # Keep features not in drop list
            filtered_features = [f for f in features if f not in to_drop]

            removed_count = len(features) - len(filtered_features)
            print(f"  Removed {removed_count} highly correlated features")

            return filtered_features

        except Exception as e:
            print(f"  Correlation filtering failed: {e}, keeping all features")
            return features

    def _determine_optimal_feature_count(
        self,
        df: pd.DataFrame,
        features: List[str]
    ) -> int:
        """Determine optimal number of features using PCA analysis."""

        # If target_features specified, use it
        if self.target_features is not None:
            return min(self.target_features, len(features))

        try:
            # Prepare data for PCA
            feature_data = df[features].fillna(0)
            scaled_data = self.scaler_.fit_transform(feature_data)

            # Fit PCA
            self.pca_ = PCA(random_state=self.random_state)
            self.pca_.fit(scaled_data)

            # Find number of components for target variance
            cumulative_variance = np.cumsum(self.pca_.explained_variance_ratio_)
            optimal_components = np.argmax(cumulative_variance >= self.pca_variance_ratio) + 1

            # Cap at reasonable limits
            min_features = max(10, len(features) // 20)  # At least 10, max 5% of total
            max_features = min(len(features) // 2, 500)   # At most 50% or 500

            optimal_count = np.clip(optimal_components, min_features, max_features)

            print(f"  PCA analysis: {cumulative_variance[optimal_components-1]:.3f} variance with {optimal_count} components")

            return optimal_count

        except Exception as e:
            print(f"  PCA analysis failed: {e}, using heuristic")
            # Fallback heuristic: 10% of features, capped between 50-200
            return np.clip(len(features) // 10, 50, 200)

    def _rank_features_autoencoder(
        self,
        df: pd.DataFrame,
        features: List[str]
    ) -> Dict[str, float]:
        """Rank features using autoencoder reconstruction error."""

        try:
            # Prepare data
            feature_data = df[features].fillna(0)
            scaled_data = self.scaler_.fit_transform(feature_data)

            # Autoencoder architecture
            n_features = scaled_data.shape[1]
            encoding_dim = max(min(n_features // 4, 100), 5)

            print(f"  Training autoencoder: {n_features} -> {encoding_dim} -> {n_features}")

            # Train autoencoder using MLPRegressor
            autoencoder = MLPRegressor(
                hidden_layer_sizes=(encoding_dim,),
                max_iter=300,
                early_stopping=True,
                validation_fraction=0.1,
                random_state=self.random_state,
                learning_rate_init=0.001
            )

            autoencoder.fit(scaled_data, scaled_data)

            # Calculate reconstruction errors
            reconstructed = autoencoder.predict(scaled_data)
            reconstruction_errors = np.mean((scaled_data - reconstructed) ** 2, axis=0)

            # Create feature importance scores (higher error = more important)
            feature_scores = {
                features[i]: float(reconstruction_errors[i])
                for i in range(len(features))
            }

            print(f"  Autoencoder training completed successfully")
            return feature_scores

        except Exception as e:
            print(f"  Autoencoder training failed: {e}, using fallback")
            return self._rank_features_statistical(df, features)

    def _rank_features_statistical(
        self,
        df: pd.DataFrame,
        features: List[str]
    ) -> Dict[str, float]:
        """Rank features using statistical methods (Random Forest)."""

        try:
            # Prepare data
            feature_data = df[features].fillna(0)

            # Create synthetic target using first principal component
            scaled_data = self.scaler_.fit_transform(feature_data)
            pca_temp = PCA(n_components=1, random_state=self.random_state)
            synthetic_target = pca_temp.fit_transform(scaled_data).flatten()

            # Train Random Forest for feature importance
            rf = RandomForestRegressor(
                n_estimators=100,
                random_state=self.random_state,
                n_jobs=-1
            )
            rf.fit(feature_data, synthetic_target)

            # Create feature importance scores
            feature_scores = {
                features[i]: float(rf.feature_importances_[i])
                for i in range(len(features))
            }

            print(f"  Statistical ranking completed using Random Forest")
            return feature_scores

        except Exception as e:
            print(f"  Statistical ranking failed: {e}, using uniform scores")
            return {feature: 1.0 for feature in features}

    def _select_top_features(
        self,
        features: List[str],
        feature_scores: Dict[str, float],
        target_count: int
    ) -> List[str]:
        """Select top N features based on importance scores."""

        # Sort features by score (descending)
        sorted_features = sorted(
            features,
            key=lambda x: feature_scores.get(x, 0),
            reverse=True
        )

        # Select top N features
        selected_features = sorted_features[:target_count]

        print(f"  Selected top {len(selected_features)} features")

        return selected_features

    def _store_selection_statistics(
        self,
        original_features: List[str],
        selected_features: List[str]
    ) -> None:
        """Store statistics about the selection process."""

        self.selection_stats_ = {
            'original_count': len(original_features),
            'selected_count': len(selected_features),
            'reduction_ratio': len(selected_features) / len(original_features),
            'use_autoencoder': self.use_autoencoder,
            'variance_threshold': self.variance_threshold,
            'correlation_threshold': self.correlation_threshold
        }

    def get_selected_features(self) -> List[str]:
        """Get list of selected features."""
        return self.selected_features_

    def get_feature_scores(self) -> Dict[str, float]:
        """Get feature importance scores."""
        return self.feature_scores_

    def get_selection_stats(self) -> Dict:
        """Get selection statistics."""
        return self.selection_stats_

    def create_feature_report(self) -> pd.DataFrame:
        """Create detailed feature selection report."""

        if not self.selected_features_:
            print("No features selected yet. Run fit_transform first.")
            return pd.DataFrame()

        # Create report data
        report_data = []
        for i, feature in enumerate(self.selected_features_):
            base_column = feature.split('_')[0] if '_' in feature else feature
            metric_type = feature.split('_')[-1] if '_' in feature else 'base'

            report_data.append({
                'rank': i + 1,
                'feature_name': feature,
                'base_column': base_column,
                'metric_type': metric_type,
                'importance_score': self.feature_scores_.get(feature, 0)
            })

        report_df = pd.DataFrame(report_data)
        return report_df


# **Feaure calling**
**Modes**

# FeatureSelector Usage Examples

## Mode 1: Intelligent Auto-Selection with Deep Learning

```python
# Automatically determines optimal feature count using autoencoder compression
print("Mode 1: Intelligent auto-selection with autoencoder")

selector = FeatureSelector(
    target_features=None,        # Let autoencoder determine optimal count
    use_autoencoder=True         # Enable deep learning feature optimization
)

selected_features = selector.fit_transform(
    df=your_dataframe,
    target_columns=['aimp', 'amud', 'arnd']  # Focus on specific business metrics
)

print(f"Auto-selected {len(selected_features)} optimal features")
```

**Best for:** Production systems where you want AI to optimize feature selection automatically.

---

## Mode 2: Fixed Count with Statistical Selection

```python
# Selects exactly N features using statistical importance ranking
print("Mode 2: Fixed feature count with statistical methods")

selector = FeatureSelector(
    target_features=100,         # Exactly 100 most important features
    use_autoencoder=False        # Use statistical ranking only
)

selected_features = selector.fit_transform(
    df=your_dataframe,
    target_columns=['aimp', 'amud', 'arnd']
)

print(f"Selected top {len(selected_features)} features by statistical importance")
```

**Best for:** When you need predictable feature count for memory/performance constraints.

---

## Mode 3: Maximum Features with Deep Learning Enhancement

```python
# Uses all available features enhanced by autoencoder insights
print("Mode 3: All features with autoencoder enhancement")

selector = FeatureSelector(
    target_features=200,         # Use up to 200 features (or all available)
    use_autoencoder=True         # Enhance with deep learning insights
)

selected_features = selector.fit_transform(
    df=your_dataframe,
    target_columns=None          # Analyze all numeric columns
)

print(f"Enhanced selection: {len(selected_features)} features")
```

**Best for:** High-accuracy models where you want maximum feature coverage with AI optimization.

---

## Results Analysis & Implementation

```python
# Create filtered dataset for anomaly detection
filtered_df = your_dataframe[['date'] + selected_features]
print(f"Final dataset shape: {filtered_df.shape}")

# Get detailed feature importance analysis
feature_report = selector.create_feature_report()
print("\nTop 10 most important features:")
print(feature_report[['feature_name', 'importance_score', 'selection_reason']].head(10))

# Use with anomaly detector
detector = EnhancedDatasetQualityAnomalyDetector()
results = detector.detect_dataset_anomalies(
    df=filtered_df,
    target_dates=['2023-02-27']
)
```

---

## Quick Selection Guide

| Use Case | Mode | Target Features | Autoencoder |
|----------|------|----------------|-------------|
| **Production AI** | Mode 1 | `None` | `True` |
| **Resource-constrained** | Mode 2 | `50-100` | `False` |
| **Maximum accuracy** | Mode 3 | `200+` | `True` |
| **Fast prototyping** | Mode 2 | `30-50` | `False` |

---

## Performance Expectations

- **Mode 1**: Optimal balance, ~60-120 features typically
- **Mode 2**: Predictable count, fastest execution
- **Mode 3**: Highest accuracy, longer processing time

**method1**

In [ ]:
method1_selector = FeatureSelector(
    target_features=None,        # Let autoencoder determine optimal count
    use_autoencoder=True         # Enable deep learning feature optimization
)

method1_selected_features = method1_selector.fit_transform(
    df=transformed_datatype_df,
    target_columns=target_columns  # Focus on specific business metrics
)

print(f" Numberof features before {len(transformed_datatype_df.columns)} After {len(method1_selected_features)} ")


Production Feature Selector initialized
  Target features: auto-determine
  Use autoencoder: True
  Variance threshold: 0.01
  Correlation threshold: 0.95

Starting feature selection on dataframe: (103, 30883)
Target-based selection for: ['aimp_Mean', 'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum', 'aimp_count', 'aimp_Completeness', 'aimp_missingCount', 'aimp_zerocount', 'aimp_zeroCount', 'aimp_zerocountpercentage', 'aimp_negativecount', 'aimp_negativeCount', 'aimp_negativecountpercentage', 'aimp_CountDistinct', 'aimp_Uniqueness', 'aimp_UniqueValueRatio', 'aimp_Distinctness', 'aimp_Histogram', 'aimp_DataType', 'aimp_ApproxQuantiles_0.1', 'aimp_ApproxQuantiles_0.25', 'aimp_ApproxQuantiles_0.5', 'aimp_ApproxQuantiles_0.75', 'aimp_ApproxQuantiles_0.9', 'aimp_Entropy', 'aimp_Size', 'aimp_MaxLength', 'aimp_MinLength', 'aimp_Correlation_amud', 'aimp_Correlation_arnd', 'aimp_Correlation_asin1', 'aimp_Correlation_asin2', 'amud_Mean', 'amud_Minimum', 'amud_Maximum', 'amud_

**method 2**

In [ ]:
# Selects exactly N features using statistical importance ranking
print("Mode 2: Fixed feature count with statistical methods")

method2_selector = FeatureSelector(
    target_features=100,         # Exactly 100 most important features
    use_autoencoder=False        # Use statistical ranking only
)

selected2_features = method2_selector.fit_transform(
    df=transformed_datatype_df,
    target_columns=target_columns
)

print(f" Numberof features before {len(transformed_datatype_df.columns)} After {len(selected2_features)} ")

Mode 2: Fixed feature count with statistical methods
Production Feature Selector initialized
  Target features: 100
  Use autoencoder: False
  Variance threshold: 0.01
  Correlation threshold: 0.95

Starting feature selection on dataframe: (103, 30883)
Target-based selection for: ['aimp_Mean', 'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum', 'aimp_count', 'aimp_Completeness', 'aimp_missingCount', 'aimp_zerocount', 'aimp_zeroCount', 'aimp_zerocountpercentage', 'aimp_negativecount', 'aimp_negativeCount', 'aimp_negativecountpercentage', 'aimp_CountDistinct', 'aimp_Uniqueness', 'aimp_UniqueValueRatio', 'aimp_Distinctness', 'aimp_Histogram', 'aimp_DataType', 'aimp_ApproxQuantiles_0.1', 'aimp_ApproxQuantiles_0.25', 'aimp_ApproxQuantiles_0.5', 'aimp_ApproxQuantiles_0.75', 'aimp_ApproxQuantiles_0.9', 'aimp_Entropy', 'aimp_Size', 'aimp_MaxLength', 'aimp_MinLength', 'aimp_Correlation_amud', 'aimp_Correlation_arnd', 'aimp_Correlation_asin1', 'aimp_Correlation_asin2', 'amud_Me

**Method3**

In [ ]:
# Uses all available features enhanced by autoencoder insights
print("Mode 3: All features with autoencoder enhancement")

method3_selector = FeatureSelector(
    target_features=200,         # Use up to 200 features (or all available)
    use_autoencoder=True         # Enhance with deep learning insights
)

M3selected_features = method3_selector.fit_transform(
    df=transformed_datatype_df,
    target_columns=None          # Analyze all numeric columns
)

print(f" Numberof features before {len(transformed_datatype_df.columns)} After {len(M3selected_features)} ")

Mode 3: All features with autoencoder enhancement
Production Feature Selector initialized
  Target features: 200
  Use autoencoder: True
  Variance threshold: 0.01
  Correlation threshold: 0.95

Starting feature selection on dataframe: (103, 30883)
Using all numeric features
Candidate features extracted: 30882
  Removed 14701 low-variance features
After variance filtering: 16181
  Removed 12424 highly correlated features
After correlation filtering: 3757
Optimal feature count determined: 200
  Training autoencoder: 3757 -> 100 -> 3757
  Autoencoder training completed successfully
Feature ranking completed using autoencoder
  Selected top 200 features
Feature selection complete: 200 features selected
Reduction ratio: 0.006
 Numberof features before 30883 After 200 


**Method4**

In [ ]:
# Uses all available features enhanced by autoencoder insights
print("Mode 4: All features with autoencoder enhancement")

method4_selector = FeatureSelector(
    target_features=1000,         # Use up to 200 features (or all available)
    use_autoencoder=True         # Enhance with deep learning insights
)

M3selected_features = method4_selector.fit_transform(
    df=transformed_datatype_df,
    target_columns=target_columns          # Analyze all numeric columns
)

print(f" Numberof features before {len(transformed_datatype_df.columns)} After {len(M3selected_features)} ")

Mode 4: All features with autoencoder enhancement
Production Feature Selector initialized
  Target features: 1000
  Use autoencoder: True
  Variance threshold: 0.01
  Correlation threshold: 0.95

Starting feature selection on dataframe: (103, 30883)
Target-based selection for: ['aimp_Mean', 'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum', 'aimp_count', 'aimp_Completeness', 'aimp_missingCount', 'aimp_zerocount', 'aimp_zeroCount', 'aimp_zerocountpercentage', 'aimp_negativecount', 'aimp_negativeCount', 'aimp_negativecountpercentage', 'aimp_CountDistinct', 'aimp_Uniqueness', 'aimp_UniqueValueRatio', 'aimp_Distinctness', 'aimp_Histogram', 'aimp_DataType', 'aimp_ApproxQuantiles_0.1', 'aimp_ApproxQuantiles_0.25', 'aimp_ApproxQuantiles_0.5', 'aimp_ApproxQuantiles_0.75', 'aimp_ApproxQuantiles_0.9', 'aimp_Entropy', 'aimp_Size', 'aimp_MaxLength', 'aimp_MinLength', 'aimp_Correlation_amud', 'aimp_Correlation_arnd', 'aimp_Correlation_asin1', 'aimp_Correlation_asin2', 'amud_Mean'

In [ ]:
transformed_datatype_df=transformed_datatype_df[M3selected_features+["date"]]

**Cross validation feature selection**

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
from collections import defaultdict
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

try:
    import cupy as cp
    import cudf
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False
    raise ImportError("PyTorch is required for this implementation")

class PyTorchAutoencoder:
    """PyTorch-only autoencoder for CPU and GPU."""

    def __init__(self, encoding_dim_ratio=0.5, epochs=50, batch_size=32,
                 learning_rate=0.001, use_gpu=False, patience=10):
        self.encoding_dim_ratio = encoding_dim_ratio
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.use_gpu = use_gpu and GPU_AVAILABLE and torch.cuda.is_available()
        self.patience = patience
        self.model = None
        self.scaler = MinMaxScaler()
        self.reconstruction_errors_ = None
        self.device = torch.device("cuda" if self.use_gpu else "cpu")

    def _create_autoencoder_model(self, input_dim, encoding_dim):
        """Create PyTorch autoencoder."""
        class AutoencoderNet(nn.Module):
            def __init__(self):
                super(AutoencoderNet, self).__init__()
                # Encoder
                self.encoder = nn.Sequential(
                    nn.Linear(input_dim, encoding_dim * 2),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(encoding_dim * 2, encoding_dim),
                    nn.ReLU()
                )
                # Decoder
                self.decoder = nn.Sequential(
                    nn.Linear(encoding_dim, encoding_dim * 2),
                    nn.ReLU(),
                    nn.Dropout(0.1),
                    nn.Linear(encoding_dim * 2, input_dim)
                )

            def forward(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded, encoded

        model = AutoencoderNet()
        model = model.to(self.device)
        return model

    def fit_transform(self, X):
        """Fit autoencoder and return feature importance ranking."""
        X = np.array(X)

        if X.shape[1] < 2:
            return np.arange(X.shape[1])

        # Scale data
        X_scaled = self.scaler.fit_transform(X)

        input_dim = X_scaled.shape[1]
        encoding_dim = max(1, int(input_dim * self.encoding_dim_ratio))

        try:
            return self._fit_pytorch_model(X_scaled, input_dim, encoding_dim)
        except Exception as e:
            print(f"PyTorch autoencoder training failed: {e}")
            # Simple variance-based fallback
            variances = np.var(X_scaled, axis=0)
            return np.argsort(variances)[::-1]

    def _fit_pytorch_model(self, X_scaled, input_dim, encoding_dim):
        """Fit using PyTorch with optimized training."""
        model = self._create_autoencoder_model(input_dim, encoding_dim)

        # Convert to tensor
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)

        # Optimizer and loss
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=1e-5)
        criterion = nn.MSELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

        # Training loop with batching for large datasets
        model.train()
        best_loss = float('inf')
        patience_counter = 0

        dataset_size = X_tensor.shape[0]

        for epoch in range(self.epochs):
            epoch_loss = 0.0
            num_batches = 0

            # Mini-batch training for memory efficiency
            for i in range(0, dataset_size, self.batch_size):
                batch_end = min(i + self.batch_size, dataset_size)
                batch_X = X_tensor[i:batch_end]

                optimizer.zero_grad()

                # Forward pass
                reconstructed, encoded = model(batch_X)
                loss = criterion(reconstructed, batch_X)

                # Backward pass
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                num_batches += 1

            avg_loss = epoch_loss / num_batches
            scheduler.step(avg_loss)

            # Early stopping
            if avg_loss < best_loss:
                best_loss = avg_loss
                patience_counter = 0
                # Save best model state
                best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        # Load best model
        model.load_state_dict(best_model_state)

        # Calculate reconstruction errors
        model.eval()
        with torch.no_grad():
            # Process in batches to avoid memory issues
            all_errors = []
            for i in range(0, dataset_size, self.batch_size):
                batch_end = min(i + self.batch_size, dataset_size)
                batch_X = X_tensor[i:batch_end]
                reconstructed, _ = model(batch_X)
                batch_errors = torch.mean((batch_X - reconstructed) ** 2, dim=0)
                all_errors.append(batch_errors)

            # Combine all batch errors
            feature_errors = torch.mean(torch.stack(all_errors), dim=0).cpu().numpy()

        self.reconstruction_errors_ = feature_errors
        self.model = model

        return self._rank_features_by_importance(feature_errors)

    def _rank_features_by_importance(self, reconstruction_errors):
        """Rank features by reconstruction error (lower error = more important)."""
        # Features with lower reconstruction error are more important
        feature_importance = 1 / (reconstruction_errors + 1e-8)

        # Get indices sorted by importance (descending)
        important_features = np.argsort(feature_importance)[::-1]

        return important_features


class CrossValidationFeatureSelector:
    """PyTorch-only cross-validation feature selector."""

    def __init__(
        self,
        base_selector_class,
        target_features: Optional[int] = None,
        cv_folds: int = 3,
        stability_threshold: float = 0.7,
        parameter_search: bool = False,
        use_autoencoder: bool = True,
        autoencoder_ratio: float = 0.5,
        autoencoder_epochs: int = 50,
        batch_size: int = 1024,  # Reduced for better memory management
        gpu_memory_limit: float = 0.8,
        random_state: int = 42
    ):
        if not TORCH_AVAILABLE:
            raise ImportError("PyTorch is required for this implementation")

        self.base_selector_class = base_selector_class
        self.target_features = target_features
        self.cv_folds = cv_folds
        self.stability_threshold = stability_threshold
        self.parameter_search = parameter_search
        self.use_autoencoder = use_autoencoder
        self.autoencoder_ratio = autoencoder_ratio
        self.autoencoder_epochs = autoencoder_epochs
        self.batch_size = batch_size
        self.gpu_memory_limit = gpu_memory_limit
        self.random_state = random_state

        # Results
        self.selected_features_ = []
        self.feature_stability_scores_ = {}
        self.best_params_ = {}
        self.autoencoder_importance_ = {}

        # GPU setup
        self.use_gpu = GPU_AVAILABLE and torch.cuda.is_available()
        if self.use_gpu:
            print(f"PyTorch GPU acceleration enabled - Autoencoder: {use_autoencoder}")
            print(f"GPU: {torch.cuda.get_device_name()}")
        else:
            print(f"PyTorch CPU mode - Autoencoder: {use_autoencoder}")

    def fit_transform(
        self,
        df: pd.DataFrame,
        target_columns: Optional[List[str]] = None,
        exclude_columns: Optional[List[str]] = None,
        date_column: str = 'timestamp'
    ) -> List[str]:
        """PyTorch-based feature selection with unified autoencoder."""

        print(f"PyTorch CV feature selection: {df.shape}")

        # Fast data prep
        feature_data = self._fast_data_prep(df, target_columns, exclude_columns)
        print(f"Feature candidates: {feature_data.shape[1]}")

        # Enhanced CV selection with PyTorch autoencoder
        stable_features = self._pytorch_cv_selection(feature_data, date_column)

        self.selected_features_ = stable_features
        print(f"Selected {len(stable_features)} stable features")

        return stable_features

    def _fast_data_prep(self, df, target_columns, exclude_columns):
        """Optimized data preparation."""
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

        if exclude_columns:
            numeric_cols = [c for c in numeric_cols if c not in exclude_columns]

        if target_columns:
            candidates = []
            for target in target_columns:
                related = [c for c in numeric_cols if c.startswith(f"{target}_")]
                candidates.extend(related)
            return df[list(set(candidates))].fillna(0)

        return df[numeric_cols].fillna(0)

    def _pytorch_cv_selection(self, feature_data, date_column):
        """Cross-validation with PyTorch autoencoder."""
        print("Using PyTorch autoencoder for CV feature selection")

        feature_frequency = defaultdict(int)

        # Time-based splits
        if len(feature_data) > 10000:
            n_samples = len(feature_data)
            fold_size = n_samples // self.cv_folds
            splits = []
            for fold in range(self.cv_folds):
                val_start = fold * fold_size
                val_end = min((fold + 1) * fold_size, n_samples)
                train_indices = list(range(0, val_start)) + list(range(val_end, n_samples))
                splits.append((train_indices, list(range(val_start, val_end))))
        else:
            tscv = TimeSeriesSplit(n_splits=self.cv_folds)
            splits = list(tscv.split(feature_data))

        for fold, (train_idx, val_idx) in enumerate(splits):
            print(f"Processing fold {fold+1}/{self.cv_folds}")

            train_data = feature_data.iloc[train_idx]
            fold_features = self._pytorch_fold_selection(train_data)

            for feature in fold_features:
                feature_frequency[feature] += 1

        # Select stable features
        min_appearances = int(self.cv_folds * self.stability_threshold)
        stable_features = [
            f for f, count in feature_frequency.items()
            if count >= min_appearances
        ]

        # Store stability scores
        self.feature_stability_scores_ = {
            f: {'stability_score': feature_frequency[f] / self.cv_folds}
            for f in stable_features
        }

        return stable_features

    def _pytorch_fold_selection(self, train_data):
        """PyTorch autoencoder feature selection for single fold."""

        if len(train_data) < 10 or train_data.shape[1] < 2:
            return list(train_data.columns)[:min(5, train_data.shape[1])]

        # Step 1: Basic filtering (variance + correlation)
        basic_features = self._basic_feature_filter(train_data)

        if not basic_features:
            return list(train_data.columns)[:min(10, train_data.shape[1])]

        # Step 2: PyTorch autoencoder selection if enabled
        if self.use_autoencoder and len(basic_features) > 2:
            final_features = self._pytorch_autoencoder_selection(train_data[basic_features])
        else:
            # Use basic features if autoencoder disabled
            target_count = self.target_features or min(len(basic_features), 25)
            final_features = basic_features[:target_count]

        return final_features

    def _basic_feature_filter(self, data):
        """Basic variance and correlation filtering."""
        # Variance filter
        variances = data.var()
        high_var_features = variances[variances > 0.001].index.tolist()

        if len(high_var_features) <= 1:
            return high_var_features

        # Correlation filter
        corr_matrix = data[high_var_features].corr().abs()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
        high_corr_pairs = np.where((corr_matrix > 0.95) & mask)
        to_drop = set(corr_matrix.columns[high_corr_pairs[1]])

        return [f for f in high_var_features if f not in to_drop]

    def _pytorch_autoencoder_selection(self, data):
        """PyTorch autoencoder-based feature selection."""
        try:
            # Create PyTorch autoencoder
            autoencoder = PyTorchAutoencoder(
                encoding_dim_ratio=self.autoencoder_ratio,
                epochs=self.autoencoder_epochs,
                batch_size=self.batch_size,
                use_gpu=self.use_gpu,
                patience=10
            )

            # Get feature importance ranking
            important_feature_indices = autoencoder.fit_transform(data.values)

            # Convert indices to feature names
            feature_names = list(data.columns)
            important_features = [feature_names[i] for i in important_feature_indices]

            # Take top features
            target_count = self.target_features or min(len(important_features), 25)
            selected_features = important_features[:target_count]

            # Store autoencoder importance scores
            if hasattr(autoencoder, 'reconstruction_errors_'):
                for i, feature in enumerate(feature_names):
                    if feature in selected_features:
                        if feature not in self.autoencoder_importance_:
                            self.autoencoder_importance_[feature] = []
                        self.autoencoder_importance_[feature].append(
                            float(autoencoder.reconstruction_errors_[i])
                        )

            return selected_features

        except Exception as e:
            print(f"PyTorch autoencoder selection failed: {e}, using fallback")
            target_count = self.target_features or min(len(data.columns), 25)
            return list(data.columns)[:target_count]

    def get_feature_importance_summary(self):
        """Get summary of feature importance from PyTorch autoencoders."""
        summary = {}
        for feature, errors in self.autoencoder_importance_.items():
            summary[feature] = {
                'mean_reconstruction_error': np.mean(errors),
                'std_reconstruction_error': np.std(errors),
                'importance_score': 1 / (np.mean(errors) + 1e-8)
            }
        return summary

    def get_selected_features(self):
        return self.selected_features_

    def get_best_params(self):
        """Return best parameters found."""
        params = {
            'use_autoencoder': self.use_autoencoder,
            'autoencoder_ratio': self.autoencoder_ratio,
            'autoencoder_epochs': self.autoencoder_epochs,
            'framework': 'pytorch'
        }
        if hasattr(self, 'best_params_') and self.best_params_:
            params.update(self.best_params_)
        return params

**simple version**

In [ ]:
# cv_selector = CrossValidationFeatureSelector(
#     base_selector_class=FeatureSelector,
#     target_features=100,
#     parameter_search=True,  # Now supported
#     use_autoencoder=True,   # Default method
#     random_state=42
# )
# OPTION A: Full auto-optimization
cvstart = time.time()
cv_selector = CrossValidationFeatureSelector(
    base_selector_class=FeatureSelector,
    target_features=100,
    parameter_search=True,
    use_autoencoder=True,
    random_state=42,
    cv_folds=3,
    autoencoder_ratio=0.5,
    autoencoder_epochs=30
)
features = cv_selector.fit_transform(transformed_datatype_df, target_columns=target_columns)
# Check results
best_params = cv_selector.get_best_params()
print(f"Best method: {'Autoencoder' if best_params.get('use_autoencoder') else 'Statistical'}")
print(f"Time: {(time.time()-cvstart)/60:.6f} minutes")

PyTorch CPU mode - Autoencoder: True
PyTorch CV feature selection: (103, 820)
Feature candidates: 819
Using PyTorch autoencoder for CV feature selection
Processing fold 1/3
Processing fold 2/3
Processing fold 3/3
Selected 56 stable features
Best method: Autoencoder
Time: 0.148854 minutes


In [ ]:
transformed_cv_df=transformed_datatype_df[features+['date']]

**LSTM & PYOD MODEL**

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Any, Tuple
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

class RealisticLSTMAnomalyScorer:
    """
    LSTM-based anomaly detection using sequence reconstruction errors.

    Simulates LSTM behavior by:
    1. Using sliding windows of historical data (14 days default)
    2. Predicting next day's values using sequence averages + noise
    3. Computing reconstruction error (MSE) between predicted vs actual
    4. Flagging samples where error exceeds learned threshold (95th percentile)

    Higher scores = more anomalous (threshold = 1.0)
    """

    def __init__(self, seq_length: int = 14, threshold_percentile: float = 95.0):
        self.seq_length = seq_length
        self.threshold_percentile = threshold_percentile
        self.baseline_errors = []
        self.threshold = None
        print(f"LSTM Scorer initialized: seq_length={seq_length}, threshold_percentile={threshold_percentile}")

    def fit_baseline(self, X_train: np.ndarray) -> None:
        """Learn normal reconstruction error patterns from training data."""
        print(f"Learning LSTM baseline from {len(X_train)} training samples...")

        self.baseline_errors = []

        for i in range(len(X_train) - self.seq_length):
            # Extract sequence (14 days) and target (next day)
            sequence = X_train[i:i+self.seq_length]
            target = X_train[i+self.seq_length]

            # Simulate LSTM prediction
            predicted = np.mean(sequence, axis=0) + np.random.normal(0, 0.1, sequence.shape[1])

            # Calculate reconstruction error
            mse = mean_squared_error(target, predicted)
            self.baseline_errors.append(mse)

        # Set threshold at 95th percentile
        self.threshold = np.percentile(self.baseline_errors, self.threshold_percentile)
        print(f"LSTM baseline threshold: {self.threshold:.4f}")

    def score_sequences(self, X_test: np.ndarray) -> np.ndarray:
        """Score sequences and return anomaly scores (>1.0 = anomalous)."""
        if self.threshold is None:
            raise ValueError("Must call fit_baseline() first")

        print(f"Scoring {len(X_test)} sequences with LSTM...")
        scores = []

        for i in range(len(X_test)):
            if i < self.seq_length:
                score = np.mean(self.baseline_errors) / self.threshold
            else:
                sequence = X_test[i-self.seq_length:i]
                target = X_test[i]
                predicted = np.mean(sequence, axis=0) + np.random.normal(0, 0.1, sequence.shape[1])
                mse = mean_squared_error(target, predicted)
                score = mse / self.threshold

            scores.append(score)

        lstm_anomalies = sum(1 for s in scores if s >= 1.0)
        print(f"LSTM scoring complete. Anomalies detected: {lstm_anomalies}")
        return np.array(scores)

class EnhancedDatasetQualityAnomalyDetector:
    """
    Multi-algorithm anomaly detector combining LSTM + PyOD methods.

    Detection pipeline:
    1. LSTM: Sequence-based reconstruction error analysis
    2. PyOD: Statistical outlier detection (ECOD, COPOD, LOF)
    3. Consensus: Weighted combination of both approaches
    4. Thresholding: Multiple severity levels (Critical/High/Medium/Low)

    Best for: Dataset-level quality monitoring, time-series anomalies
    """

    def __init__(self, default_config: Dict[str, Any] = None):
        print("Initializing Enhanced Dataset Quality Anomaly Detector...")

        self.default_config = default_config or {
            'seq_length': 14,
            'contamination': 0.02,
            'pyod_method': 'ecod',
            'use_autoencoder': False,
            'lstm_score_threshold': 1.0,
            'pyod_score_threshold': 0.7,
            'consensus_weight_lstm': 0.6,
            'consensus_weight_pyod': 0.4,
            'consensus_threshold': 0.75,
            'hidden_dim': 64,
            'epochs': 50
        }

        print(f"Configuration:")
        print(f"   • LSTM sequence length: {self.default_config['seq_length']} days")
        print(f"   • PyOD algorithm: {self.default_config['pyod_method'].upper()}")
        print(f"   • Use autoencoder: {self.default_config['use_autoencoder']}")
        print(f"   • Thresholds: LSTM={self.default_config['lstm_score_threshold']}, PyOD={self.default_config['pyod_score_threshold']}")
        print(f"   • Consensus weights: LSTM={self.default_config['consensus_weight_lstm']}, PyOD={self.default_config['consensus_weight_pyod']}")
        print("Detector ready")

    def detect_dataset_anomalies(
        self,
        df: pd.DataFrame,
        timestamp_col: str = 'date',
        target_columns: Optional[List[str]] = None,
        target_dates: Optional[List[str]] = None,
        mode: str = 'specific_dates',
        config: Dict[str, Any] = None
    ) -> Dict[str, Any]:
        """
        Main detection function combining LSTM + PyOD algorithms.

        Args:
            df: DataFrame with date column + numeric features
            timestamp_col: Date column name
            target_columns: Columns to analyze (auto-detected if None)
            target_dates: Specific dates to check (uses latest 7 if None)
            mode: 'specific_dates' or 'rolling_window'
            config: Override default detection parameters

        Returns:
            Dict with anomaly scores, dates, severity levels, and model results
        """

        print(f"\nENHANCED DATASET ANOMALY DETECTION")
        print(f"Input shape: {df.shape}")
        print(f"Mode: {mode}")

        # Configuration
        detection_config = self.default_config.copy()
        if config:
            detection_config.update(config)

        # Get dates and features
        all_dates = sorted(df[timestamp_col].unique())
        print(f"Date range: {all_dates[0]} to {all_dates[-1]} ({len(all_dates)} days)")

        if mode == 'specific_dates' and target_dates:
            print(f"Target dates: {target_dates}")
        else:
            target_dates = all_dates[-7:]
            print(f"Using latest 7 days as targets")

        # Handle target columns
        if target_columns is None:
            target_columns = [col for col in df.select_dtypes(include=[np.number]).columns
                            if col != timestamp_col]
        print(f"Target columns: {target_columns}")

        # Get features
        available_features = self._get_target_features(df, target_columns)
        print(f"Found {len(available_features)} relevant features")

        # Prepare data
        X, dates, feature_names = self._prepare_model_features(df, timestamp_col, available_features)

        # Split data
        train_indices = [i for i, date in enumerate(dates) if date not in target_dates]
        test_indices = [i for i, date in enumerate(dates) if date in target_dates]

        print(f"Training: {len(train_indices)} days, Testing: {len(test_indices)} days")

        # Run detection
        results = self._run_enhanced_detection(
            X, dates, detection_config, train_indices, test_indices
        )

        # Analyze results
        detailed_results = self._analyze_enhanced_anomalies(
            results, dates, target_columns, test_indices, detection_config
        )

        print(f"\nDETECTION COMPLETE!")
        print(f"Anomalies detected: {len(detailed_results['anomalous_dates'])}")
        return detailed_results

    def _run_enhanced_detection(
        self,
        X: np.ndarray,
        dates: np.ndarray,
        config: Dict[str, Any],
        train_indices: List[int],
        test_indices: List[int]
    ) -> Dict[str, Any]:
        """
        Core detection pipeline running LSTM + PyOD.

        Pipeline steps:
        1. Train LSTM on historical data, score all sequences
        2. Train PyOD model, score all samples
        3. Apply individual thresholds to get binary predictions
        4. Compute consensus scores using weighted combination
        5. Return comprehensive results for analysis
        """

        print(f"\nEnhanced Detection Pipeline Starting...")

        X_train = X[train_indices]
        X_test = X[test_indices]

        # 1. LSTM Scoring
        print(f"\n1. LSTM SEQUENCE ANOMALY DETECTION")
        lstm_scorer = RealisticLSTMAnomalyScorer(
            seq_length=config['seq_length'],
            threshold_percentile=95.0
        )
        lstm_scorer.fit_baseline(X_train)
        all_lstm_scores = lstm_scorer.score_sequences(X)

        # 2. PyOD Scoring
        print(f"\n2. PyOD STATISTICAL ANOMALY DETECTION")
        pyod_results = self._run_real_pyod_detection(
            X, train_indices, test_indices,
            config['contamination'],
            config.get('pyod_method', 'ecod')
        )
        all_pyod_scores = pyod_results['scores']
        pyod_threshold_learned = pyod_results['threshold']

        # 3. Apply thresholds
        print(f"\n3. APPLYING THRESHOLDS")
        lstm_threshold = config['lstm_score_threshold']
        pyod_threshold = config.get('pyod_score_threshold', pyod_threshold_learned)

        lstm_anomalies = (all_lstm_scores >= lstm_threshold).astype(int)
        pyod_anomalies = (all_pyod_scores >= pyod_threshold).astype(int)

        print(f"LSTM anomalies: {sum(lstm_anomalies)} (threshold: {lstm_threshold})")
        print(f"PyOD anomalies: {sum(pyod_anomalies)} (threshold: {pyod_threshold:.6f})")

        # 4. Consensus scoring
        print(f"\n4. CONSENSUS ENSEMBLE SCORING")
        consensus_scores = (
            config['consensus_weight_lstm'] * all_lstm_scores +
            config['consensus_weight_pyod'] * all_pyod_scores
        )

        consensus_anomalies = (consensus_scores >= config['consensus_threshold']).astype(int)
        combined_anomalies = np.logical_or(lstm_anomalies, pyod_anomalies).astype(int)

        print(f"Consensus anomalies: {sum(consensus_anomalies)} (threshold: {config['consensus_threshold']})")
        print(f"Combined anomalies: {sum(combined_anomalies)}")

        return {
            'lstm_anomalies': lstm_anomalies,
            'pyod_anomalies': pyod_anomalies,
            'consensus_anomalies': consensus_anomalies,
            'combined_anomalies': combined_anomalies,
            'lstm_scores': all_lstm_scores,
            'pyod_scores': all_pyod_scores,
            'consensus_scores': consensus_scores,
            'pyod_threshold': pyod_threshold_learned,
            'pyod_model': pyod_results.get('model'),
            'model_config': config,
            'train_indices': train_indices,
            'test_indices': test_indices
        }

    def _run_real_pyod_detection(
        self,
        X: np.ndarray,
        train_indices: List[int],
        test_indices: List[int],
        contamination: float,
        method: str = 'ecod'
    ) -> Dict[str, Any]:
        """
        Run PyOD statistical outlier detection.

        Supported methods:
        - ECOD: Empirical Cumulative Distribution Outlier Detection
        - COPOD: Copula-based Outlier Detection
        - LOF: Local Outlier Factor

        Falls back to statistical distance method if PyOD unavailable.
        """

        print(f"Running REAL PyOD detection with {method.upper()}...")

        X_train = X[train_indices]
        X_test = X[test_indices]

        try:
            # Import and initialize PyOD model
            if method == 'ecod':
                from pyod.models.ecod import ECOD
                model = ECOD(contamination=contamination)
                print(f"   ECOD: Empirical Cumulative Distribution Outlier Detection")
            elif method == 'copod':
                from pyod.models.copod import COPOD
                model = COPOD(contamination=contamination)
                print(f"   COPOD: Copula-based Outlier Detection")
            elif method == 'lof':
                from pyod.models.lof import LOF
                model = LOF(contamination=contamination)
                print(f"   LOF: Local Outlier Factor")
            else:
                from pyod.models.ecod import ECOD
                model = ECOD(contamination=contamination)
                print(f"   Unknown method '{method}', using ECOD as fallback")

            print(f"   Training {method.upper()} on {len(train_indices)} historical samples...")
            model.fit(X_train)

            print(f"   Scoring {len(test_indices)} test samples...")
            test_scores = model.decision_function(X_test)
            threshold = model.threshold_

            # Create full score array
            all_scores = np.zeros(len(X))

            # Map test scores back to full array
            for i, test_idx in enumerate(test_indices):
                all_scores[test_idx] = test_scores[i]

            print(f"   PyOD threshold: {threshold:.6f}")
            print(f"   Score range: [{test_scores.min():.4f}, {test_scores.max():.4f}]")

            return {
                'scores': all_scores,
                'threshold': threshold,
                'test_scores': test_scores,
                'model': model
            }

        except ImportError as e:
            print(f"   PyOD not available, using fallback: {e}")
            return self._generate_fallback_pyod_scores(X, train_indices, test_indices)

    def _generate_fallback_pyod_scores(
        self,
        X: np.ndarray,
        train_indices: List[int],
        test_indices: List[int]
    ) -> Dict[str, Any]:
        """
        Fallback statistical outlier detection when PyOD unavailable.

        Uses standardized distance from training mean as anomaly score.
        """

        print(f"   Using statistical fallback for PyOD scores...")

        X_train = X[train_indices]
        feature_means = np.mean(X_train, axis=0)
        feature_stds = np.std(X_train, axis=0)

        scores = []
        for i in range(len(X)):
            if i in train_indices:
                base_score = np.random.beta(2, 5)  # Lower scores for training
            else:
                base_score = np.random.beta(3, 3)  # Higher for test

            distances = np.abs(X[i] - feature_means) / (feature_stds + 1e-8)
            distance_score = np.mean(distances) / 10
            final_score = min(base_score + distance_score, 1.0)
            scores.append(final_score)

        all_scores = np.array(scores)
        test_scores = all_scores[test_indices]
        threshold = np.percentile(all_scores[train_indices], 95)

        return {
            'scores': all_scores,
            'threshold': threshold,
            'test_scores': test_scores,
            'model': None
        }

    def _analyze_enhanced_anomalies(
        self,
        results: Dict[str, Any],
        dates: np.ndarray,
        target_columns: List[str],
        test_indices: List[int],
        config: Dict[str, Any]
    ) -> Dict[str, Any]:
        """
        Analyze detection results with detailed scoring and severity classification.

        Severity levels:
        - CRITICAL: High LSTM + PyOD scores (lstm>=1.5, pyod>=0.8)
        - HIGH: High consensus score (>=0.8)
        - MEDIUM: Medium consensus score (>=0.6)
        - LOW: Low consensus score but above threshold
        """

        print(f"Analyzing enhanced anomaly results...")

        # Filter to test dates only
        test_dates = [str(dates[i]) for i in test_indices]

        anomaly_analysis = {
            'total_days_analyzed': len(test_indices),
            'anomalous_dates': [],
            'anomaly_scores': {},
            'anomaly_severity': {},
            'score_statistics': {},
            'model_results': results,
            'summary_stats': {}
        }

        # Analyze each test date
        consensus_anomalies = results['consensus_anomalies']

        for i, date_idx in enumerate(test_indices):
            date_str = str(dates[date_idx])

            # Get scores for this date
            lstm_score = results['lstm_scores'][date_idx]
            pyod_score = results['pyod_scores'][date_idx]
            consensus_score = results['consensus_scores'][date_idx]

            # Store detailed scores
            anomaly_analysis['anomaly_scores'][date_str] = {
                'lstm_score': float(lstm_score),
                'pyod_score': float(pyod_score),
                'consensus_score': float(consensus_score),
                'lstm_detected': bool(results['lstm_anomalies'][date_idx]),
                'pyod_detected': bool(results['pyod_anomalies'][date_idx]),
                'consensus_detected': bool(consensus_anomalies[date_idx])
            }

            # Determine severity
            if consensus_anomalies[date_idx]:
                anomaly_analysis['anomalous_dates'].append(date_str)

                if lstm_score >= 1.5 and pyod_score >= 0.8:
                    severity = 'CRITICAL'
                elif consensus_score >= 0.8:
                    severity = 'HIGH'
                elif consensus_score >= 0.6:
                    severity = 'MEDIUM'
                else:
                    severity = 'LOW'

                anomaly_analysis['anomaly_severity'][date_str] = severity
                print(f"   {date_str}: {severity} anomaly (consensus: {consensus_score:.3f})")
            else:
                print(f"   {date_str}: Normal (consensus: {consensus_score:.3f})")

        # Calculate statistics
        test_lstm_scores = [results['lstm_scores'][i] for i in test_indices]
        test_pyod_scores = [results['pyod_scores'][i] for i in test_indices]
        test_consensus_scores = [results['consensus_scores'][i] for i in test_indices]

        anomaly_analysis['score_statistics'] = {
            'lstm_scores': {
                'min': float(np.min(test_lstm_scores)),
                'max': float(np.max(test_lstm_scores)),
                'mean': float(np.mean(test_lstm_scores)),
                'std': float(np.std(test_lstm_scores))
            },
            'pyod_scores': {
                'min': float(np.min(test_pyod_scores)),
                'max': float(np.max(test_pyod_scores)),
                'mean': float(np.mean(test_pyod_scores)),
                'std': float(np.std(test_pyod_scores))
            },
            'consensus_scores': {
                'min': float(np.min(test_consensus_scores)),
                'max': float(np.max(test_consensus_scores)),
                'mean': float(np.mean(test_consensus_scores)),
                'std': float(np.std(test_consensus_scores))
            }
        }

        # Summary stats
        anomaly_analysis['summary_stats'] = {
            'total_anomalies': len(anomaly_analysis['anomalous_dates']),
            'anomaly_rate': len(anomaly_analysis['anomalous_dates']) / len(test_indices) if test_indices else 0,
            'critical_count': sum(1 for s in anomaly_analysis['anomaly_severity'].values() if s == 'CRITICAL'),
            'high_severity_count': sum(1 for s in anomaly_analysis['anomaly_severity'].values() if s == 'HIGH'),
            'target_columns_analyzed': target_columns,
            'thresholds_used': {
                'lstm_threshold': config['lstm_score_threshold'],
                'pyod_threshold': config['pyod_score_threshold'],
                'consensus_threshold': config['consensus_threshold']
            }
        }

        return anomaly_analysis

    def _get_target_features(self, df: pd.DataFrame, target_columns: List[str]) -> List[str]:
        """Get relevant features for target columns."""
        relevant_features = []

        for target_col in target_columns:
            col_features = [col for col in df.columns
                          if col.startswith(f"{target_col}_") and col != 'date']

            numeric_features = [f for f in col_features
                              if pd.api.types.is_numeric_dtype(df[f])]
            relevant_features.extend(numeric_features)

        return sorted(list(set(relevant_features)))

    def _prepare_model_features(self, df: pd.DataFrame, timestamp_col: str,
                               feature_columns: List[str]) -> Tuple[np.ndarray, np.ndarray, List[str]]:
        """Prepare features for models."""

        # Get numeric features
        numeric_features = []
        for col in feature_columns:
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
                numeric_features.append(col)

        if not numeric_features:
            raise ValueError("No numeric features found")

        # Extract and clean data
        X = df[numeric_features].values.astype(np.float64)
        dates = df[timestamp_col].values

        # Handle NaN/inf
        X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)

        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        print(f"Prepared {X_scaled.shape[0]} samples with {X_scaled.shape[1]} features")

        return X_scaled, dates, numeric_features

**Instruction**

# Enhanced Dataset Quality Anomaly Detector - Usage Guide

## Overview
Multi-algorithm anomaly detection system combining LSTM sequence analysis with PyOD statistical methods for dataset quality monitoring.

---

## 1. Initialize Detector

```python
detector = EnhancedDatasetQualityAnomalyDetector(
    default_config={
        'seq_length': 14,                    # LSTM lookback window (days)
        'contamination': 0.02,               # Expected anomaly rate (2%)
        'pyod_method': 'ecod',               # PyOD algorithm: 'ecod', 'copod', 'lof'
        'use_autoencoder': False,            # Enable deep learning (optional)
        'lstm_score_threshold': 1.0,         # LSTM anomaly threshold
        'pyod_score_threshold': 0.7,         # PyOD anomaly threshold
        'consensus_weight_lstm': 0.6,        # LSTM weight in consensus (60%)
        'consensus_weight_pyod': 0.4,        # PyOD weight in consensus (40%)
        'consensus_threshold': 0.75,         # Final anomaly threshold
        'hidden_dim': 64,                    # Neural network size
        'epochs': 50                         # Training iterations
    }
)
```

---

## 2. Run Detection

```python
results = detector.detect_dataset_anomalies(
    df=transformed_datatype_df,              # Your prepared DataFrame
    mode='specific_dates',                   # Detection mode
    target_dates=['2023-02-27'],             # Dates to analyze
    target_columns=['aimp', 'amud', 'arnd']  # Columns to monitor
)
```

### Parameters:
- **df**: DataFrame with date column + numeric features
- **mode**: `'specific_dates'` or `'rolling_window'`
- **target_dates**: List of dates to check (uses latest 7 if None)
- **target_columns**: Columns to analyze (auto-detected if None)

---

## 3. Access Results

### Individual Date Scores
```python
# Get anomaly scores for specific date
if '2023-02-27' in results.get('anomaly_scores', {}):
    scores = results['anomaly_scores']['2023-02-27']
    print(f'LSTM Score: {scores["lstm_score"]:.3f}')
    print(f'PyOD Score: {scores["pyod_score"]:.3f}')
    print(f'Consensus Score: {scores["consensus_score"]:.3f}')
    print(f'Final Decision: {"ANOMALY" if scores["consensus_detected"] else "NORMAL"}')
```

### Summary Statistics
```python
# Get summary statistics
print(f'Total anomalies: {results["summary_stats"]["total_anomalies"]}')
print(f'Anomaly rate: {results["summary_stats"]["anomaly_rate"]*100:.1f}%')
print(f'Critical alerts: {results["summary_stats"]["critical_count"]}')
```

### Severity Levels
```python
# Check severity classifications
for date, severity in results['anomaly_severity'].items():
    print(f'{date}: {severity}')
    # CRITICAL: lstm>=1.5 + pyod>=0.8
    # HIGH: consensus>=0.8
    # MEDIUM: consensus>=0.6
    # LOW: above threshold
```

---

## 4. Result Structure

```python
results = {
    'anomalous_dates': ['2023-02-27'],           # List of anomalous dates
    'anomaly_scores': {                          # Detailed scores per date
        '2023-02-27': {
            'lstm_score': 1.45,
            'pyod_score': 0.82,
            'consensus_score': 0.89,
            'consensus_detected': True
        }
    },
    'anomaly_severity': {                        # Severity classification
        '2023-02-27': 'HIGH'
    },
    'summary_stats': {                           # Overall statistics
        'total_anomalies': 1,
        'anomaly_rate': 0.14,
        'critical_count': 0,
        'high_severity_count': 1
    },
    'score_statistics': {                        # Score distributions
        'lstm_scores': {'min': 0.45, 'max': 1.45, 'mean': 0.82},
        'pyod_scores': {'min': 0.12, 'max': 0.82, 'mean': 0.41}
    }
}
```

---

## 5. Configuration Tips

### For Sensitive Detection (catch more anomalies):
```python
config = {
    'lstm_score_threshold': 0.8,        # Lower threshold
    'consensus_threshold': 0.6,         # Lower consensus
    'contamination': 0.05               # Higher expected rate
}
```

### For Conservative Detection (fewer false alarms):
```python
config = {
    'lstm_score_threshold': 1.2,        # Higher threshold
    'consensus_threshold': 0.8,         # Higher consensus
    'contamination': 0.01               # Lower expected rate
}
```

---


## 7. Quick Start Example

```python
# Initialize with default settings
detector = EnhancedDatasetQualityAnomalyDetector()

# Run detection on latest data
results = detector.detect_dataset_anomalies(
    df=your_dataframe,
    target_dates=['2023-02-27']
)

# Get simple yes/no answer
is_anomaly = '2023-02-27' in results['anomalous_dates']
print(f"2023-02-27 is {'ANOMALOUS' if is_anomaly else 'NORMAL'}")
```


**LSTM Execution**

# Parameter Configuration Guide

## LSTM Parameters

### `seq_length: 14`
**Purpose**: Historical window size for sequence analysis  
**How it works**: LSTM examines 14 consecutive days to predict the next day's pattern  
**Trade-offs**:
- Higher values (21-30) = More context, better trend detection, slower processing
- Lower values (7-10) = Faster processing, may miss long-term patterns
- Recommended: 14 days captures 2-week business cycles

### `lstm_score_threshold: 1.0`
**Purpose**: Anomaly detection cutoff for LSTM predictions  
**How it works**: Reconstruction error ≥ 1.0 flags as anomaly (1.0 = baseline threshold)  
**Sensitivity tuning**:
- **More sensitive** (0.8): Catches subtle anomalies, more false positives
- **Less sensitive** (1.2): Only major anomalies, fewer false positives
- **Default** (1.0): Balanced detection based on 95th percentile training errors

---

## PyOD Statistical Parameters

### `contamination: 0.02`
**Purpose**: Expected percentage of anomalous data points  
**How it works**: Tells PyOD algorithms that ~2% of data should be outliers  
**Guidelines**:
- **High quality data** (0.01): Very few expected anomalies
- **Moderate quality** (0.02): Standard business data assumption
- **Poor quality data** (0.05): Higher anomaly rate expected

### `pyod_score_threshold: 0.7`
**Purpose**: Statistical outlier detection cutoff  
**How it works**: PyOD scores ≥ 0.7 considered anomalous  
**Sensitivity tuning**:
- **More sensitive** (0.5): Detects borderline statistical outliers
- **Less sensitive** (0.8): Only clear statistical anomalies
- **Auto-learned**: Can use PyOD's built-in threshold instead

---

## Consensus Ensemble Scoring

### `consensus_weight_lstm: 0.6` & `consensus_weight_pyod: 0.4`
**Purpose**: Relative importance of each detection method  
**How it works**: Final score = (0.6 × LSTM) + (0.4 × PyOD)  
**When to adjust**:
- **Time-series focus** (0.7/0.3): Emphasize sequence patterns
- **Statistical focus** (0.4/0.6): Emphasize outlier detection
- **Balanced** (0.5/0.5): Equal weight to both approaches

### `consensus_threshold: 0.75`
**Purpose**: Final decision boundary for anomaly classification  
**How it works**: Combined weighted score ≥ 0.75 = anomaly  
**Decision impact**:
- **Conservative** (0.8): Requires strong agreement between methods
- **Moderate** (0.75): Standard threshold for business applications
- **Aggressive** (0.6): Flags more potential issues for investigation

---

## Model Architecture

### `hidden_dim: 64`
**Purpose**: LSTM neural network complexity  
**How it works**: Number of neurons in hidden layers  
**Resource trade-offs**:
- **Lightweight** (32): Faster training, simpler patterns
- **Standard** (64): Good balance for most datasets
- **Complex** (128): Captures intricate patterns, slower training

### `epochs: 50`
**Purpose**: Training iteration count  
**How it works**: Number of complete passes through training data  
**Training balance**:
- **Quick** (25): Fast training, may underfit
- **Standard** (50): Adequate learning for most cases
- **Thorough** (100): Better convergence, risk of overfitting

---

## Tuning Recommendations

### For High-Sensitivity Detection
```python
config = {
    'lstm_score_threshold': 0.8,      # Lower threshold
    'pyod_score_threshold': 0.6,      # More sensitive
    'consensus_threshold': 0.65,      # Easier to trigger
    'contamination': 0.03             # Expect more anomalies
}
```

### For Conservative Detection  
```python
config = {
    'lstm_score_threshold': 1.2,      # Higher threshold
    'pyod_score_threshold': 0.8,      # Less sensitive
    'consensus_threshold': 0.8,       # Require strong signal
    'contamination': 0.01             # Expect fewer anomalies
}
```

### For Performance Optimization
```python
config = {
    'seq_length': 10,                 # Shorter sequences
    'hidden_dim': 32,                 # Smaller network
    'epochs': 25,                     # Faster training
    'pyod_method': 'ecod'            # Fastest PyOD algorithm
}
```

**Performance**

**More Sensitive Detection:**

*  'lstm_score_threshold': 0.8,     # Lower threshold
*   'pyod_score_threshold': 0.5,     # Lower threshold
*   'contamination': 0.05,           # Expect more anomalies
*   'consensus_threshold': 0.6       # Lower combined threshold

**More Conservative Detection:**

*   'lstm_score_threshold': 1.5,     # Higher threshold
*   'contamination': 0.01,           # Expect fewer anomalies
*   'consensus_threshold': 0.9       # Higher combined threshold

**Performance vs Accuracy:**

*   'seq_length': 7,        # Faster but less context
*   'hidden_dim': 32,       # Faster but simpler model
*   'epochs': 25            # Faster training

**Trust LSTM more:**
*   'consensus_weight_lstm': 0.8,
*   'consensus_weight_pyod': 0.2





In [ ]:
detector = EnhancedDatasetQualityAnomalyDetector(
    default_config={
        'seq_length': 14,
        'contamination': 0.02,
        'pyod_method': 'ecod',
        'use_autoencoder': True,
        'lstm_score_threshold': 1.2, #1.0
        'pyod_score_threshold': 0.95,
        'consensus_weight_lstm': 0.6,
        'consensus_weight_pyod': 0.4,
        'consensus_threshold': 0.5,
        'hidden_dim': 64,
        'epochs': 50
    }
)

Initializing Enhanced Dataset Quality Anomaly Detector...
Configuration:
   • LSTM sequence length: 14 days
   • PyOD algorithm: ECOD
   • Use autoencoder: True
   • Thresholds: LSTM=1.2, PyOD=0.95
   • Consensus weights: LSTM=0.6, PyOD=0.4
Detector ready


In [ ]:
lstm_pyod_results = detector.detect_dataset_anomalies(
    df=transformed_cv_df, # cross validation feature df
    mode='specific_dates',
    target_dates=['2023-02-27'],
    target_columns=target_columns
    # target_columns=['aimp', 'amud', 'arnd']
)


ENHANCED DATASET ANOMALY DETECTION
Input shape: (103, 56)
Mode: specific_dates
Date range: 2022-11-17 to 2023-02-27 (103 days)
Target dates: ['2023-02-27']
Target columns: ['aimp_Mean', 'aimp_Minimum', 'aimp_Maximum', 'aimp_StandardDeviation', 'aimp_Sum', 'aimp_count', 'aimp_Completeness', 'aimp_missingCount', 'aimp_zerocount', 'aimp_zeroCount', 'aimp_zerocountpercentage', 'aimp_negativecount', 'aimp_negativeCount', 'aimp_negativecountpercentage', 'aimp_CountDistinct', 'aimp_Uniqueness', 'aimp_UniqueValueRatio', 'aimp_Distinctness', 'aimp_Histogram', 'aimp_DataType', 'aimp_ApproxQuantiles_0.1', 'aimp_ApproxQuantiles_0.25', 'aimp_ApproxQuantiles_0.5', 'aimp_ApproxQuantiles_0.75', 'aimp_ApproxQuantiles_0.9', 'aimp_Entropy', 'aimp_Size', 'aimp_MaxLength', 'aimp_MinLength', 'aimp_Correlation_amud', 'aimp_Correlation_arnd', 'aimp_Correlation_asin1', 'aimp_Correlation_asin2', 'amud_Mean', 'amud_Minimum', 'amud_Maximum', 'amud_StandardDeviation', 'amud_Sum', 'amud_count', 'amud_Completeness'

**save model result dict**

In [ ]:
# import pandas as pd

# # Extract the anomaly scores from the dictionary
# anomaly_scores_data = lstm_pyod_results.get('anomaly_scores', {})

# # Convert the scores to a DataFrame
# if anomaly_scores_data:
#     anomaly_scores_df = pd.DataFrame.from_dict(anomaly_scores_data, orient='index')
#     # Add a 'date' column from the index
#     anomaly_scores_df['date'] = anomaly_scores_df.index
#     # Reset the index
#     anomaly_scores_df = anomaly_scores_df.reset_index(drop=True)

#     # Save the DataFrame to a CSV file
#     csv_filename = 'lstm_pyod_anomaly_scores.csv'
#     anomaly_scores_df.to_csv(csv_filename, index=False)

#     print(f"Anomaly scores saved to {csv_filename}")
# else:
#     print("No anomaly scores found in the results dictionary.")

**Explainable DataFrame**

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Tuple, Optional
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from openpyxl.styles import PatternFill, Font, Border, Side
import warnings
warnings.filterwarnings('ignore')

class StreamlinedAnomalyExplainer:
    """Simplified anomaly explainer with DQ dimensions and enhanced reporting."""

    def __init__(self, model_results: Dict[str, Any], original_df: pd.DataFrame):
        self.model_results = model_results
        self.original_df = original_df.copy()

        if 'date' in self.original_df.columns:
            self.original_df['date'] = pd.to_datetime(self.original_df['date']).dt.strftime('%Y-%m-%d')

        self.feature_columns = [col for col in original_df.columns if col != 'date']
        self.dq_patterns = {
            'Completeness': ['completeness', 'missing', 'null', 'isnotnull'],
            'Accuracy': ['accuracy', 'entropy', 'zscore', 'outlier', 'anomaly', 'mean', 'sum', 'count'],
            'Consistency': ['consistency', 'standarddeviation', 'variance', 'correlation'],
            'Validity': ['validity', 'minimum', 'maximum', 'datatype', 'pattern', 'regex'],
            'Uniqueness': ['uniqueness', 'countdistinct', 'distinctness', 'duplicate'],
            'Timeliness': ['timeliness', 'freshness', 'lag', 'delay', 'timestamp']
        }

    def analyze_anomaly(self, target_date: str, top_n: int = 10) -> Dict[str, Any]:
        """Main analysis function."""
        target_date = pd.to_datetime(target_date).strftime('%Y-%m-%d')

        # Validate input
        anomaly_scores = self.model_results.get('anomaly_scores', {})
        if target_date not in anomaly_scores:
            return {'error': f"No anomaly detected for {target_date}"}

        target_row = self.original_df[self.original_df['date'] == target_date].iloc[0]
        historical_df = self.original_df[self.original_df['date'] < target_date]

        # Calculate feature contributions
        contributions = self._calculate_contributions(target_date, target_row, historical_df)
        top_features = sorted(contributions.items(), key=lambda x: x[1], reverse=True)[:top_n]

        # Generate explanations
        explanations = [self._explain_feature(f, score, target_row[f], historical_df[f].dropna())
                       for f, score in top_features]

        # Create DQ summary
        dq_summary = self._summarize_dq_dimensions(explanations)

        return {
            'date': target_date,
            'severity': self.model_results.get('anomaly_severity', {}).get(target_date, 'UNKNOWN'),
            'consensus_score': round(anomaly_scores[target_date].get('consensus_score', 0), 4),
            'explanations': explanations,
            'dq_summary': dq_summary,
            'fixes': self._generate_fixes(target_date, target_row, [f for f, _ in top_features[:5]]),
            'executive_summary': self._create_summary(explanations, dq_summary, target_date)
        }

    def _calculate_contributions(self, target_date: str, target_row: pd.Series,
                               historical_df: pd.DataFrame) -> Dict[str, float]:
        """Calculate feature importance scores using ensemble approach."""
        contributions = {}

        for feature in self.feature_columns:
            current_val = target_row[feature]
            hist_values = historical_df[feature].dropna()

            if len(hist_values) < 3:
                contributions[feature] = 0
                continue

            # Statistical deviation score
            z_score = abs((current_val - hist_values.mean()) / (hist_values.std() + 1e-8))

            # Pattern deviation score
            recent_trend = hist_values.tail(7).mean()
            expected = 0.7 * recent_trend + 0.3 * hist_values.median()
            pattern_score = abs(current_val - expected) / (hist_values.std() + 1e-8)

            # Model perturbation score (if available)
            model_score = 0
            model = self.model_results.get('statistical_model')
            if model is not None:
                baseline_data = target_row[self.feature_columns].values.reshape(1, -1)
                baseline_anomaly = abs(model.decision_function(baseline_data)[0])

                perturbed_data = baseline_data.copy()
                perturbed_data[0, self.feature_columns.index(feature)] = hist_values.median()
                perturbed_anomaly = abs(model.decision_function(perturbed_data)[0])

                model_score = max(0, baseline_anomaly - perturbed_anomaly)

            # Ensemble score
            contributions[feature] = 0.4 * z_score + 0.4 * pattern_score + 0.2 * model_score

        return contributions

    def _explain_feature(self, feature: str, importance: float, current_val: float,
                        hist_values: pd.Series) -> Dict[str, Any]:
        """Generate explanation for a single feature."""
        if len(hist_values) == 0:
            return {
                'feature': feature,
                'dq_dimension': self._map_to_dq_dimension(feature),
                'importance': round(importance, 4),
                'severity': 'UNKNOWN',
                'explanation': f"No historical data for {feature}",
                'current_value': current_val,
                'baseline': None
            }

        hist_mean = hist_values.mean()
        hist_std = hist_values.std()
        z_score = (current_val - hist_mean) / (hist_std + 1e-8)
        deviation_pct = ((current_val - hist_mean) / (abs(hist_mean) + 1e-8)) * 100

        # Classify severity
        severity = ('CRITICAL' if abs(z_score) >= 4 or abs(deviation_pct) >= 300 else
                   'HIGH' if abs(z_score) >= 3 or abs(deviation_pct) >= 150 else
                   'MEDIUM' if abs(z_score) >= 2 or abs(deviation_pct) >= 75 else 'LOW')

        # Generate explanation
        direction = "increased" if current_val > hist_mean else "decreased"
        magnitude = ("drastically" if abs(deviation_pct) >= 200 else
                    "significantly" if abs(deviation_pct) >= 100 else
                    "moderately" if abs(deviation_pct) >= 50 else "slightly")

        explanation = f"{feature} has {magnitude} {direction} by {abs(deviation_pct):.1f}% from baseline"

        return {
            'feature': feature,
            'base_column': feature.split('_')[0],
            'dq_dimension': self._map_to_dq_dimension(feature),
            'importance': round(importance, 4),
            'severity': severity,
            'current_value': round(current_val, 6),
            'baseline': round(hist_mean, 6),
            'z_score': round(z_score, 3),
            'deviation_pct': round(deviation_pct, 2),
            'explanation': explanation,
            'recommendation': self._get_recommendation(severity, feature.split('_')[0])
        }

    def _map_to_dq_dimension(self, feature: str) -> str:
        """Map feature to DQ dimension."""
        feature_lower = feature.lower()
        for dimension, patterns in self.dq_patterns.items():
            if any(pattern in feature_lower for pattern in patterns):
                return dimension
        return 'Accuracy'

    def _summarize_dq_dimensions(self, explanations: List[Dict]) -> Dict[str, Dict]:
        """Summarize by DQ dimension."""
        summary = {}
        for exp in explanations:
            dq_dim = exp['dq_dimension']
            if dq_dim not in summary:
                summary[dq_dim] = {'count': 0, 'total_impact': 0, 'severities': []}

            summary[dq_dim]['count'] += 1
            summary[dq_dim]['total_impact'] += exp['importance']
            summary[dq_dim]['severities'].append(exp['severity'])

        # Calculate metrics
        for dq_dim in summary:
            summary[dq_dim]['avg_impact'] = summary[dq_dim]['total_impact'] / summary[dq_dim]['count']
            severity_counts = {s: summary[dq_dim]['severities'].count(s) for s in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']}
            summary[dq_dim]['severity_breakdown'] = severity_counts
            summary[dq_dim]['risk_level'] = self._assess_risk_level(severity_counts)

        return summary

    def _assess_risk_level(self, severity_counts: Dict[str, int]) -> str:
        """Assess overall risk level for DQ dimension."""
        if severity_counts.get('CRITICAL', 0) > 0:
            return 'EXTREME'
        elif severity_counts.get('HIGH', 0) >= 2:
            return 'HIGH'
        elif severity_counts.get('HIGH', 0) > 0 or severity_counts.get('MEDIUM', 0) >= 3:
            return 'MODERATE'
        return 'LOW'

    def _handle_pyod_fallback(self, X: np.ndarray, train_indices: List[int], test_indices: List[int]) -> Dict[str, Any]:
        """Enhanced statistical fallback aligned with your detector's logic."""
        from sklearn.covariance import EllipticEnvelope

        X_train = X[train_indices]

        # Use Isolation Forest-like approach as fallback
        try:
            model = EllipticEnvelope(contamination=0.02, random_state=42)
            model.fit(X_train)

            all_scores = np.abs(model.decision_function(X))
            # Normalize to 0-1 range like PyOD
            all_scores = (all_scores - all_scores.min()) / (all_scores.max() - all_scores.min() + 1e-8)
            threshold = np.percentile(all_scores[train_indices], 95)

            return {
                'scores': all_scores,
                'threshold': threshold,
                'test_scores': all_scores[test_indices],
                'model': model
            }
        except:
            # Ultimate fallback with Z-score approach
            return self._statistical_zscore_fallback(X, train_indices, test_indices)

    def _statistical_zscore_fallback(self, X: np.ndarray, train_indices: List[int], test_indices: List[int]) -> Dict[str, Any]:
        """Z-score based fallback matching your parameter ranges."""
        X_train = X[train_indices]
        train_mean = np.mean(X_train, axis=0)
        train_std = np.std(X_train, axis=0) + 1e-8

        # Calculate multivariate Z-scores
        z_scores = np.sqrt(np.sum(((X - train_mean) / train_std) ** 2, axis=1))
        # Normalize to PyOD-like range (0-1)
        normalized_scores = z_scores / (z_scores.max() + 1e-8)

        threshold = 0.95  # Align with your pyod_score_threshold

        return {
            'scores': normalized_scores,
            'threshold': threshold,
            'test_scores': normalized_scores[test_indices],
            'model': None
        }

    def _generate_fixes(self, target_date: str, target_row: pd.Series, features: List[str]) -> List[Dict]:
        """Generate actionable model parameter adjustments and acceptable ranges."""
        historical_df = self.original_df[self.original_df['date'] < target_date]
        fixes = []

        for feature in features:
            if feature in historical_df.columns:
                hist_values = historical_df[feature].dropna()
                current = target_row[feature]

                # Calculate acceptable range (median ± 2 std)
                median_val = hist_values.median()
                std_val = hist_values.std()
                acceptable_min = median_val - (2 * std_val)
                acceptable_max = median_val + (2 * std_val)

                # Determine severity and parameter adjustments
                z_score = abs((current - median_val) / (std_val + 1e-8))

                # Model parameter recommendations based on severity
                if z_score >= 4:
                    sensitivity = "Decrease contamination to 0.01, increase consensus_threshold to 0.7"
                elif z_score >= 3:
                    sensitivity = "Decrease contamination to 0.015, increase consensus_threshold to 0.6"
                elif z_score >= 2:
                    sensitivity = "Use default contamination 0.02, consensus_threshold 0.5"
                else:
                    sensitivity = "Increase contamination to 0.03, decrease consensus_threshold to 0.4"

                fixes.append({
                    'feature': feature,
                    'current_value': round(current, 6),
                    'acceptable_range_min': round(acceptable_min, 6),
                    'acceptable_range_max': round(acceptable_max, 6),
                    'baseline_median': round(median_val, 6),
                    'z_score': round(z_score, 3),
                    'dq_dimension': self._map_to_dq_dimension(feature),
                    'model_parameter_adjustment': sensitivity,
                    'data_quality_action': self._get_dq_action(feature, z_score),
                    'acceptable_deviation': f"±{round(2 * std_val, 3)} from baseline"
                })

        return fixes

    def _get_dq_action(self, feature: str, z_score: float) -> str:
        """Get data quality improvement action."""
        dq_dim = self._map_to_dq_dimension(feature)

        actions = {
            'Completeness': f"Review data ingestion pipeline for {feature.split('_')[0]}",
            'Accuracy': f"Validate calculation logic for {feature.split('_')[0]} metrics",
            'Consistency': f"Standardize data formats in {feature.split('_')[0]} processing",
            'Validity': f"Check business rules compliance for {feature.split('_')[0]}",
            'Uniqueness': f"Investigate duplicate handling in {feature.split('_')[0]}",
            'Timeliness': f"Review data refresh schedules for {feature.split('_')[0]}"
        }

        severity_prefix = "URGENT: " if z_score >= 3 else "Monitor: "
        return severity_prefix + actions.get(dq_dim, f"Review {feature.split('_')[0]} data quality")

    def _get_recommendation(self, severity: str, column: str) -> str:
        """Get actionable recommendation."""
        recommendations = {
            'CRITICAL': f"IMMEDIATE: Investigate {column} data pipeline and source systems",
            'HIGH': f"URGENT: Review {column} data collection within 4 hours",
            'MEDIUM': f"MONITOR: Schedule {column} review within 24 hours",
            'LOW': f"TRACK: Include {column} in next routine review"
        }
        return recommendations.get(severity, f"Review {column} data quality")

    def _create_summary(self, explanations: List[Dict], dq_summary: Dict, date: str) -> str:
        """Create executive summary."""
        if not explanations:
            return f"No significant anomalies detected for {date}"

        critical_count = sum(1 for exp in explanations if exp['severity'] == 'CRITICAL')
        high_count = sum(1 for exp in explanations if exp['severity'] == 'HIGH')

        summary = f"Anomaly Alert {date}: "

        if critical_count > 0:
            summary += f"{critical_count} critical issues require immediate attention. "
        if high_count > 0:
            summary += f"{high_count} high-priority issues need 4-hour review. "

        # Top DQ dimensions
        if dq_summary:
            top_dq = max(dq_summary.items(), key=lambda x: x[1]['total_impact'])
            summary += f"Primary concern: {top_dq[0]} dimension ({top_dq[1]['count']} features affected)."

        return summary


def export_comprehensive_report(analysis_results: Dict[str, Dict],
                              filename: str = 'anomaly_intelligence_report.xlsx') -> str:
    """Export enhanced report with better formatting and user-friendly tabs."""

    # Color schemes
    severity_colors = {
        'CRITICAL': PatternFill(start_color='DC143C', end_color='DC143C', fill_type='solid'),
        'HIGH': PatternFill(start_color='FF8C00', end_color='FF8C00', fill_type='solid'),
        'MEDIUM': PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid'),
        'LOW': PatternFill(start_color='98FB98', end_color='98FB98', fill_type='solid')
    }

    severity_fonts = {
        'CRITICAL': Font(color='FFFFFF', bold=True),
        'HIGH': Font(color='FFFFFF', bold=True),
        'MEDIUM': Font(color='000000', bold=True),
        'LOW': Font(color='000000', bold=False)
    }

    with pd.ExcelWriter(filename, engine='openpyxl') as writer:

        # Tab 1: Executive Dashboard
        dashboard_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                critical_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'CRITICAL')
                high_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'HIGH')

                dashboard_data.append({
                    'Alert_Date': date,
                    'Overall_Severity': result['severity'],
                    'Consensus_Score': result['consensus_score'],
                    'Critical_Issues': critical_count,
                    'High_Priority_Issues': high_count,
                    'DQ_Dimensions_Affected': len(result['dq_summary']),
                    'Action_Items': critical_count + high_count,
                    'Executive_Summary': result['executive_summary']
                })

        if dashboard_data:
            dashboard_df = pd.DataFrame(dashboard_data)
            dashboard_df.to_excel(writer, sheet_name='Executive_Dashboard', index=False)

            # Apply severity coloring
            ws = writer.sheets['Executive_Dashboard']
            for row in range(2, len(dashboard_df) + 2):
                severity = ws[f'B{row}'].value
                if severity in severity_colors:
                    ws[f'B{row}'].fill = severity_colors[severity]
                    ws[f'B{row}'].font = severity_fonts[severity]

        # Tab 2: Immediate Actions Required
        action_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    if exp['severity'] in ['CRITICAL', 'HIGH']:
                        sla = '2 hours' if exp['severity'] == 'CRITICAL' else '4 hours'
                        action_data.append({
                            'Date': date,
                            'Priority': exp['severity'],
                            'Feature': exp['feature'],
                            'Data_Source': exp['base_column'],
                            'DQ_Issue_Type': exp['dq_dimension'],
                            'Impact_Score': exp['importance'],
                            'Issue_Description': exp['explanation'],
                            'Action_Required': exp['recommendation'],
                            'SLA': sla,
                            'Owner': 'Data Engineering' if exp['dq_dimension'] == 'Completeness' else 'Data Quality Team'
                        })

        if action_data:
            action_df = pd.DataFrame(action_data)
            action_df = action_df.sort_values(['Priority', 'Impact_Score'], ascending=[True, False])
            action_df.to_excel(writer, sheet_name='Immediate_Actions', index=False)

            # Color code by priority
            ws = writer.sheets['Immediate_Actions']
            for row in range(2, len(action_df) + 2):
                priority = ws[f'B{row}'].value
                if priority in severity_colors:
                    for col in range(1, len(action_df.columns) + 1):
                        ws.cell(row=row, column=col).fill = severity_colors[priority]
                        if col == 2:  # Priority column
                            ws.cell(row=row, column=col).font = severity_fonts[priority]

        # Tab 3: Data Quality Heatmap
        heatmap_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for dq_dim, stats in result['dq_summary'].items():
                    risk_score = stats['total_impact']
                    heatmap_data.append({
                        'Date': date,
                        'DQ_Dimension': dq_dim,
                        'Risk_Score': round(risk_score, 2),
                        'Risk_Level': stats['risk_level'],
                        'Features_Affected': stats['count'],
                        'Critical_Issues': stats['severity_breakdown'].get('CRITICAL', 0),
                        'High_Issues': stats['severity_breakdown'].get('HIGH', 0),
                        'Average_Impact': round(stats['avg_impact'], 3)
                    })

        if heatmap_data:
            heatmap_df = pd.DataFrame(heatmap_data)
            heatmap_df.to_excel(writer, sheet_name='DQ_Risk_Heatmap', index=False)

            # Apply risk level coloring
            ws = writer.sheets['DQ_Risk_Heatmap']
            risk_colors = {
                'EXTREME': severity_colors['CRITICAL'],
                'HIGH': severity_colors['HIGH'],
                'MODERATE': severity_colors['MEDIUM'],
                'LOW': severity_colors['LOW']
            }

            for row in range(2, len(heatmap_df) + 2):
                risk_level = ws[f'D{row}'].value
                if risk_level in risk_colors:
                    ws[f'D{row}'].fill = risk_colors[risk_level]

        # Tab 4: Feature Analysis Details
        detail_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    detail_data.append({
                        'Date': date,
                        'Feature': exp['feature'],
                        'Data_Source': exp['base_column'],
                        'DQ_Dimension': exp['dq_dimension'],
                        'Severity': exp['severity'],
                        'Current_Value': exp['current_value'],
                        'Baseline_Value': exp['baseline'],
                        'Deviation_%': exp['deviation_pct'],
                        'Z_Score': exp['z_score'],
                        'Impact_Score': exp['importance'],
                        'Business_Impact': exp['explanation'],
                        'Recommended_Action': exp['recommendation']
                    })

        if detail_data:
            detail_df = pd.DataFrame(detail_data)
            detail_df.to_excel(writer, sheet_name='Feature_Analysis', index=False)

            # Apply severity coloring
            ws = writer.sheets['Feature_Analysis']
            for row in range(2, len(detail_df) + 2):
                severity = ws[f'E{row}'].value
                if severity in severity_colors:
                    ws[f'E{row}'].fill = severity_colors[severity]
                    ws[f'E{row}'].font = severity_fonts[severity]

        # Tab 5: Model Tuning & Data Quality Actions
        fix_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for fix in result['fixes']:
                    fix_data.append({
                        'Date': date,
                        'Feature': fix['feature'],
                        'DQ_Dimension': fix['dq_dimension'],
                        'Current_Value': fix['current_value'],
                        'Acceptable_Range': f"{fix['acceptable_range_min']} to {fix['acceptable_range_max']}",
                        'Baseline_Median': fix['baseline_median'],
                        'Deviation_Tolerance': fix['acceptable_deviation'],
                        'Z_Score': fix['z_score'],
                        'Model_Parameter_Tuning': fix['model_parameter_adjustment'],
                        'Data_Quality_Action': fix['data_quality_action']
                    })

        if fix_data:
            fix_df = pd.DataFrame(fix_data)
            fix_df.to_excel(writer, sheet_name='Model_Tuning_Actions', index=False)

    print(f"Comprehensive report exported to {filename}")
    return filename


# Wrapper function to maintain compatibility with old calling method
def analyze_anomaly_with_comprehensive_report(model_results: Dict, original_df: pd.DataFrame,
                                            target_date: str, top_n_features: Optional[int] = None,
                                            analyze_all_features: bool = False,
                                            export_to_excel: bool = True,
                                            excel_filename: Optional[str] = None) -> Dict[str, Any]:
    """Compatibility wrapper for existing calling pattern."""

    explainer = StreamlinedAnomalyExplainer(model_results, original_df)

    # Handle feature count
    top_n = len(explainer.feature_columns) if analyze_all_features else (top_n_features or 10)

    # Perform analysis
    analysis = explainer.analyze_anomaly(target_date, top_n)

    if 'error' in analysis:
        return analysis

    # Add compatibility fields
    analysis['analysis_scope'] = 'ALL_FEATURES' if analyze_all_features else f'TOP_{top_n}'
    analysis['features_analyzed'] = len(analysis['explanations'])
    analysis['top_contributing_features'] = analysis['explanations']
    analysis['dq_dimension_summary'] = analysis['dq_summary']
    analysis['counterfactual_fixpack'] = analysis['fixes']

    # Export if requested
    if export_to_excel:
        filename = excel_filename or f"anomaly_analysis_{target_date.replace('-', '_')}.xlsx"
        results_dict = {target_date: analysis}
        export_path = export_comprehensive_report(results_dict, filename)
        analysis['excel_report_path'] = export_path

    return analysis


def analyze_and_report(model_results: Dict, original_df: pd.DataFrame,
                      target_dates: List[str], top_features: int = 10,
                      export_excel: bool = True) -> Dict[str, Any]:
    """Main function for streamlined analysis and reporting."""

    explainer = StreamlinedAnomalyExplainer(model_results, original_df)
    results = {}

    for date in target_dates:
        print(f"Analyzing {date}...")
        analysis = explainer.analyze_anomaly(date, top_features)
        if 'error' not in analysis:
            results[date] = analysis

            # Print summary
            print(f"  Severity: {analysis['severity']}")
            print(f"  DQ Dimensions: {len(analysis['dq_summary'])}")
            print(f"  Critical/High Issues: {sum(1 for exp in analysis['explanations'] if exp['severity'] in ['CRITICAL', 'HIGH'])}")

    if export_excel and results:
        export_path = export_comprehensive_report(results)
        print(f"Report exported to: {export_path}")
        return {'results': results, 'report_path': export_path}

    return {'results': results}

In [ ]:
explanation = analyze_anomaly_with_comprehensive_report(
    model_results=lstm_pyod_results,
    original_df=transformed_cv_df,
    target_date='2023-02-27',
    analyze_all_features=True
)

Comprehensive report exported to anomaly_analysis_2023_02_27.xlsx


In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Tuple, Optional
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from openpyxl.styles import PatternFill, Font, Border, Side
import warnings
warnings.filterwarnings('ignore')

class StreamlinedAnomalyExplainerWithSeason:
    """Simplified anomaly explainer with DQ dimensions, enhanced reporting, and deseasonalization support."""

    def __init__(self, model_results: Dict[str, Any], original_df: pd.DataFrame, deseasonalizer=None):
        self.model_results = model_results
        self.original_df = original_df.copy()
        self.deseasonalizer = deseasonalizer

        if 'date' in self.original_df.columns:
            self.original_df['date'] = pd.to_datetime(self.original_df['date']).dt.strftime('%Y-%m-%d')

        self.feature_columns = [col for col in original_df.columns if col != 'date']
        self.dq_patterns = {
            'Completeness': ['completeness', 'missing', 'null', 'isnotnull'],
            'Accuracy': ['accuracy', 'entropy', 'zscore', 'outlier', 'anomaly', 'mean', 'sum', 'count'],
            'Consistency': ['consistency', 'standarddeviation', 'variance', 'correlation'],
            'Validity': ['validity', 'minimum', 'maximum', 'datatype', 'pattern', 'regex'],
            'Uniqueness': ['uniqueness', 'countdistinct', 'distinctness', 'duplicate'],
            'Timeliness': ['timeliness', 'freshness', 'lag', 'delay', 'timestamp']
        }

    def analyze_anomaly(self, target_date: str, top_n: int = 10) -> Dict[str, Any]:
        """Main analysis function."""
        target_date = pd.to_datetime(target_date).strftime('%Y-%m-%d')

        # Validate input
        anomaly_scores = self.model_results.get('anomaly_scores', {})
        if target_date not in anomaly_scores:
            return {'error': f"No anomaly detected for {target_date}"}

        target_row = self.original_df[self.original_df['date'] == target_date].iloc[0]
        historical_df = self.original_df[self.original_df['date'] < target_date]

        # Calculate feature contributions
        contributions = self._calculate_contributions(target_date, target_row, historical_df)
        top_features = sorted(contributions.items(), key=lambda x: x[1], reverse=True)[:top_n]

        # Generate explanations
        explanations = [self._explain_feature(f, score, target_row[f], historical_df[f].dropna(), target_date)
                       for f, score in top_features]

        # Create DQ summary
        dq_summary = self._summarize_dq_dimensions(explanations)

        return {
            'date': target_date,
            'severity': self.model_results.get('anomaly_severity', {}).get(target_date, 'UNKNOWN'),
            'consensus_score': round(anomaly_scores[target_date].get('consensus_score', 0), 4),
            'explanations': explanations,
            'dq_summary': dq_summary,
            'fixes': self._generate_fixes(target_date, target_row, [f for f, _ in top_features[:5]]),
            'executive_summary': self._create_summary(explanations, dq_summary, target_date)
        }

    def _calculate_contributions(self, target_date: str, target_row: pd.Series,
                               historical_df: pd.DataFrame) -> Dict[str, float]:
        """Calculate feature importance scores using ensemble approach."""
        contributions = {}

        for feature in self.feature_columns:
            current_val = target_row[feature]
            hist_values = historical_df[feature].dropna()

            if len(hist_values) < 3:
                contributions[feature] = 0
                continue

            # Statistical deviation score
            z_score = abs((current_val - hist_values.mean()) / (hist_values.std() + 1e-8))

            # Pattern deviation score
            recent_trend = hist_values.tail(7).mean()
            expected = 0.7 * recent_trend + 0.3 * hist_values.median()
            pattern_score = abs(current_val - expected) / (hist_values.std() + 1e-8)

            # Model perturbation score (if available)
            model_score = 0
            model = self.model_results.get('statistical_model')
            if model is not None:
                baseline_data = target_row[self.feature_columns].values.reshape(1, -1)
                baseline_anomaly = abs(model.decision_function(baseline_data)[0])

                perturbed_data = baseline_data.copy()
                perturbed_data[0, self.feature_columns.index(feature)] = hist_values.median()
                perturbed_anomaly = abs(model.decision_function(perturbed_data)[0])

                model_score = max(0, baseline_anomaly - perturbed_anomaly)

            # Ensemble score
            contributions[feature] = 0.4 * z_score + 0.4 * pattern_score + 0.2 * model_score

        return contributions

    def _recompose_original_value(self, feature: str, current_deseasonalized: float, target_date: str) -> float:
        """Reconstruct original value from deseasonalized value + seasonal component."""
        if not self.deseasonalizer:
            return current_deseasonalized

        # Get base column name (remove _deseasonalized suffix if present)
        base_column = feature.replace('_deseasonalized', '')

        try:
            # Get the row index for the target date
            target_row_idx = self.original_df[self.original_df['date'] == target_date].index[0]

            # Use recompose_for_explanation method
            recomposed_series = self.deseasonalizer.recompose_for_explanation(
                deseasonalized_values=pd.Series([current_deseasonalized], index=[target_row_idx]),
                original_column_name=base_column
            )

            return float(recomposed_series.iloc[0])

        except Exception as e:
            print(f"Warning: Could not recompose {feature}, using deseasonalized value: {e}")
            return current_deseasonalized

    def _explain_feature(self, feature: str, importance: float, current_val: float,
                        hist_values: pd.Series, target_date: str) -> Dict[str, Any]:
        """Generate explanation for a single feature."""
        if len(hist_values) == 0:
            return {
                'feature': feature,
                'dq_dimension': self._map_to_dq_dimension(feature),
                'importance': round(importance, 4),
                'severity': 'UNKNOWN',
                'explanation': f"No historical data for {feature}",
                'current_value': current_val,
                'baseline': None
            }

        hist_mean = hist_values.mean()
        hist_std = hist_values.std()
        z_score = (current_val - hist_mean) / (hist_std + 1e-8)
        deviation_pct = ((current_val - hist_mean) / (abs(hist_mean) + 1e-8)) * 100

        # Classify severity
        severity = ('CRITICAL' if abs(z_score) >= 4 or abs(deviation_pct) >= 300 else
                   'HIGH' if abs(z_score) >= 3 or abs(deviation_pct) >= 150 else
                   'MEDIUM' if abs(z_score) >= 2 or abs(deviation_pct) >= 75 else 'LOW')

        # Generate explanation
        direction = "increased" if current_val > hist_mean else "decreased"
        magnitude = ("drastically" if abs(deviation_pct) >= 200 else
                    "significantly" if abs(deviation_pct) >= 100 else
                    "moderately" if abs(deviation_pct) >= 50 else "slightly")

        explanation = f"{feature} has {magnitude} {direction} by {abs(deviation_pct):.1f}% from baseline"

        # Get original value (recomposed from deseasonalized if possible)
        original_current_value = self._recompose_original_value(feature, current_val, target_date)

        return {
            'feature': feature,
            'base_column': feature.split('_')[0],
            'dq_dimension': self._map_to_dq_dimension(feature),
            'importance': round(importance, 4),
            'severity': severity,
            'current_value': round(original_current_value, 6),  # Shows original value
            'current_deseasonalized_value': round(current_val, 6),  # Keep deseasonalized for reference
            'baseline': round(hist_mean, 6),
            'z_score': round(z_score, 3),
            'deviation_pct': round(deviation_pct, 2),
            'explanation': explanation,
            'recommendation': self._get_recommendation(severity, feature.split('_')[0])
        }

    def _map_to_dq_dimension(self, feature: str) -> str:
        """Map feature to DQ dimension."""
        feature_lower = feature.lower()
        for dimension, patterns in self.dq_patterns.items():
            if any(pattern in feature_lower for pattern in patterns):
                return dimension
        return 'Accuracy'

    def _summarize_dq_dimensions(self, explanations: List[Dict]) -> Dict[str, Dict]:
        """Summarize by DQ dimension."""
        summary = {}
        for exp in explanations:
            dq_dim = exp['dq_dimension']
            if dq_dim not in summary:
                summary[dq_dim] = {'count': 0, 'total_impact': 0, 'severities': []}

            summary[dq_dim]['count'] += 1
            summary[dq_dim]['total_impact'] += exp['importance']
            summary[dq_dim]['severities'].append(exp['severity'])

        # Calculate metrics
        for dq_dim in summary:
            summary[dq_dim]['avg_impact'] = summary[dq_dim]['total_impact'] / summary[dq_dim]['count']
            severity_counts = {s: summary[dq_dim]['severities'].count(s) for s in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']}
            summary[dq_dim]['severity_breakdown'] = severity_counts
            summary[dq_dim]['risk_level'] = self._assess_risk_level(severity_counts)

        return summary

    def _assess_risk_level(self, severity_counts: Dict[str, int]) -> str:
        """Assess overall risk level for DQ dimension."""
        if severity_counts.get('CRITICAL', 0) > 0:
            return 'EXTREME'
        elif severity_counts.get('HIGH', 0) >= 2:
            return 'HIGH'
        elif severity_counts.get('HIGH', 0) > 0 or severity_counts.get('MEDIUM', 0) >= 3:
            return 'MODERATE'
        return 'LOW'

    def _generate_fixes(self, target_date: str, target_row: pd.Series, features: List[str]) -> List[Dict]:
        """Generate actionable model parameter adjustments and acceptable ranges."""
        historical_df = self.original_df[self.original_df['date'] < target_date]
        fixes = []

        for feature in features:
            if feature in historical_df.columns:
                hist_values = historical_df[feature].dropna()
                current = target_row[feature]

                # Calculate acceptable range (median ± 2 std)
                median_val = hist_values.median()
                std_val = hist_values.std()
                acceptable_min = median_val - (2 * std_val)
                acceptable_max = median_val + (2 * std_val)

                # Determine severity and parameter adjustments
                z_score = abs((current - median_val) / (std_val + 1e-8))

                # Model parameter recommendations based on severity
                if z_score >= 4:
                    sensitivity = "Decrease contamination to 0.01, increase consensus_threshold to 0.7"
                elif z_score >= 3:
                    sensitivity = "Decrease contamination to 0.015, increase consensus_threshold to 0.6"
                elif z_score >= 2:
                    sensitivity = "Use default contamination 0.02, consensus_threshold 0.5"
                else:
                    sensitivity = "Increase contamination to 0.03, decrease consensus_threshold to 0.4"

                # Get original current value if deseasonalizer available
                original_current = self._recompose_original_value(feature, current, target_date)

                fixes.append({
                    'feature': feature,
                    'current_value': round(original_current, 6),
                    'acceptable_range_min': round(acceptable_min, 6),
                    'acceptable_range_max': round(acceptable_max, 6),
                    'baseline_median': round(median_val, 6),
                    'z_score': round(z_score, 3),
                    'dq_dimension': self._map_to_dq_dimension(feature),
                    'model_parameter_adjustment': sensitivity,
                    'data_quality_action': self._get_dq_action(feature, z_score),
                    'acceptable_deviation': f"±{round(2 * std_val, 3)} from baseline"
                })

        return fixes

    def _get_dq_action(self, feature: str, z_score: float) -> str:
        """Get data quality improvement action."""
        dq_dim = self._map_to_dq_dimension(feature)

        actions = {
            'Completeness': f"Review data ingestion pipeline for {feature.split('_')[0]}",
            'Accuracy': f"Validate calculation logic for {feature.split('_')[0]} metrics",
            'Consistency': f"Standardize data formats in {feature.split('_')[0]} processing",
            'Validity': f"Check business rules compliance for {feature.split('_')[0]}",
            'Uniqueness': f"Investigate duplicate handling in {feature.split('_')[0]}",
            'Timeliness': f"Review data refresh schedules for {feature.split('_')[0]}"
        }

        severity_prefix = "URGENT: " if z_score >= 3 else "Monitor: "
        return severity_prefix + actions.get(dq_dim, f"Review {feature.split('_')[0]} data quality")

    def _get_recommendation(self, severity: str, column: str) -> str:
        """Get actionable recommendation."""
        recommendations = {
            'CRITICAL': f"IMMEDIATE: Investigate {column} data pipeline and source systems",
            'HIGH': f"URGENT: Review {column} data collection within 4 hours",
            'MEDIUM': f"MONITOR: Schedule {column} review within 24 hours",
            'LOW': f"TRACK: Include {column} in next routine review"
        }
        return recommendations.get(severity, f"Review {column} data quality")

    def _create_summary(self, explanations: List[Dict], dq_summary: Dict, date: str) -> str:
        """Create executive summary."""
        if not explanations:
            return f"No significant anomalies detected for {date}"

        critical_count = sum(1 for exp in explanations if exp['severity'] == 'CRITICAL')
        high_count = sum(1 for exp in explanations if exp['severity'] == 'HIGH')

        summary = f"Anomaly Alert {date}: "

        if critical_count > 0:
            summary += f"{critical_count} critical issues require immediate attention. "
        if high_count > 0:
            summary += f"{high_count} high-priority issues need 4-hour review. "

        # Top DQ dimensions
        if dq_summary:
            top_dq = max(dq_summary.items(), key=lambda x: x[1]['total_impact'])
            summary += f"Primary concern: {top_dq[0]} dimension ({top_dq[1]['count']} features affected)."

        return summary


def export_comprehensive_report(analysis_results: Dict[str, Dict],
                              filename: str = 'anomaly_intelligence_report.xlsx') -> str:
    """Export enhanced report with better formatting and user-friendly tabs."""

    # Color schemes
    severity_colors = {
        'CRITICAL': PatternFill(start_color='DC143C', end_color='DC143C', fill_type='solid'),
        'HIGH': PatternFill(start_color='FF8C00', end_color='FF8C00', fill_type='solid'),
        'MEDIUM': PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid'),
        'LOW': PatternFill(start_color='98FB98', end_color='98FB98', fill_type='solid')
    }

    severity_fonts = {
        'CRITICAL': Font(color='FFFFFF', bold=True),
        'HIGH': Font(color='FFFFFF', bold=True),
        'MEDIUM': Font(color='000000', bold=True),
        'LOW': Font(color='000000', bold=False)
    }

    with pd.ExcelWriter(filename, engine='openpyxl') as writer:

        # Tab 1: Executive Dashboard
        dashboard_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                critical_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'CRITICAL')
                high_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'HIGH')

                dashboard_data.append({
                    'Alert_Date': date,
                    'Overall_Severity': result['severity'],
                    'Consensus_Score': result['consensus_score'],
                    'Critical_Issues': critical_count,
                    'High_Priority_Issues': high_count,
                    'DQ_Dimensions_Affected': len(result['dq_summary']),
                    'Action_Items': critical_count + high_count,
                    'Executive_Summary': result['executive_summary']
                })

        if dashboard_data:
            dashboard_df = pd.DataFrame(dashboard_data)
            dashboard_df.to_excel(writer, sheet_name='Executive_Dashboard', index=False)

            # Apply severity coloring
            ws = writer.sheets['Executive_Dashboard']
            for row in range(2, len(dashboard_df) + 2):
                severity = ws[f'B{row}'].value
                if severity in severity_colors:
                    ws[f'B{row}'].fill = severity_colors[severity]
                    ws[f'B{row}'].font = severity_fonts[severity]

        # Tab 2: Immediate Actions Required
        action_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    if exp['severity'] in ['CRITICAL', 'HIGH']:
                        sla = '2 hours' if exp['severity'] == 'CRITICAL' else '4 hours'
                        action_data.append({
                            'Date': date,
                            'Priority': exp['severity'],
                            'Feature': exp['feature'],
                            'Data_Source': exp['base_column'],
                            'DQ_Issue_Type': exp['dq_dimension'],
                            'Impact_Score': exp['importance'],
                            'Issue_Description': exp['explanation'],
                            'Action_Required': exp['recommendation'],
                            'SLA': sla,
                            'Owner': 'Data Engineering' if exp['dq_dimension'] == 'Completeness' else 'Data Quality Team'
                        })

        if action_data:
            action_df = pd.DataFrame(action_data)
            action_df = action_df.sort_values(['Priority', 'Impact_Score'], ascending=[True, False])
            action_df.to_excel(writer, sheet_name='Immediate_Actions', index=False)

            # Color code by priority
            ws = writer.sheets['Immediate_Actions']
            for row in range(2, len(action_df) + 2):
                priority = ws[f'B{row}'].value
                if priority in severity_colors:
                    for col in range(1, len(action_df.columns) + 1):
                        ws.cell(row=row, column=col).fill = severity_colors[priority]
                        if col == 2:  # Priority column
                            ws.cell(row=row, column=col).font = severity_fonts[priority]

        # Tab 3: Data Quality Heatmap
        heatmap_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for dq_dim, stats in result['dq_summary'].items():
                    risk_score = stats['total_impact']
                    heatmap_data.append({
                        'Date': date,
                        'DQ_Dimension': dq_dim,
                        'Risk_Score': round(risk_score, 2),
                        'Risk_Level': stats['risk_level'],
                        'Features_Affected': stats['count'],
                        'Critical_Issues': stats['severity_breakdown'].get('CRITICAL', 0),
                        'High_Issues': stats['severity_breakdown'].get('HIGH', 0),
                        'Average_Impact': round(stats['avg_impact'], 3)
                    })

        if heatmap_data:
            heatmap_df = pd.DataFrame(heatmap_data)
            heatmap_df.to_excel(writer, sheet_name='DQ_Risk_Heatmap', index=False)

            # Apply risk level coloring
            ws = writer.sheets['DQ_Risk_Heatmap']
            risk_colors = {
                'EXTREME': severity_colors['CRITICAL'],
                'HIGH': severity_colors['HIGH'],
                'MODERATE': severity_colors['MEDIUM'],
                'LOW': severity_colors['LOW']
            }

            for row in range(2, len(heatmap_df) + 2):
                risk_level = ws[f'D{row}'].value
                if risk_level in risk_colors:
                    ws[f'D{row}'].fill = risk_colors[risk_level]

        # Tab 4: Feature Analysis Details
        detail_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    detail_data.append({
                        'Date': date,
                        'Feature': exp['feature'],
                        'Data_Source': exp['base_column'],
                        'DQ_Dimension': exp['dq_dimension'],
                        'Severity': exp['severity'],
                        'Current_Value': exp['current_value'],
                        'Current_Deseasonalized': exp.get('current_deseasonalized_value', 'N/A'),
                        'Baseline_Value': exp['baseline'],
                        'Deviation_%': exp['deviation_pct'],
                        'Z_Score': exp['z_score'],
                        'Impact_Score': exp['importance'],
                        'Business_Impact': exp['explanation'],
                        'Recommended_Action': exp['recommendation']
                    })

        if detail_data:
            detail_df = pd.DataFrame(detail_data)
            detail_df.to_excel(writer, sheet_name='Feature_Analysis', index=False)

            # Apply severity coloring
            ws = writer.sheets['Feature_Analysis']
            for row in range(2, len(detail_df) + 2):
                severity = ws[f'E{row}'].value
                if severity in severity_colors:
                    ws[f'E{row}'].fill = severity_colors[severity]
                    ws[f'E{row}'].font = severity_fonts[severity]

        # Tab 5: Model Tuning & Data Quality Actions
        fix_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for fix in result['fixes']:
                    fix_data.append({
                        'Date': date,
                        'Feature': fix['feature'],
                        'DQ_Dimension': fix['dq_dimension'],
                        'Current_Value': fix['current_value'],
                        'Acceptable_Range': f"{fix['acceptable_range_min']} to {fix['acceptable_range_max']}",
                        'Baseline_Median': fix['baseline_median'],
                        'Deviation_Tolerance': fix['acceptable_deviation'],
                        'Z_Score': fix['z_score'],
                        'Model_Parameter_Tuning': fix['model_parameter_adjustment'],
                        'Data_Quality_Action': fix['data_quality_action']
                    })

        if fix_data:
            fix_df = pd.DataFrame(fix_data)
            fix_df.to_excel(writer, sheet_name='Model_Tuning_Actions', index=False)

    print(f"Comprehensive report exported to {filename}")
    return filename


# Wrapper function to maintain compatibility with old calling method
def analyze_anomaly_with_comprehensive_report(model_results: Dict, original_df: pd.DataFrame,
                                            target_date: str, top_n_features: Optional[int] = None,
                                            analyze_all_features: bool = False,
                                            export_to_excel: bool = True,
                                            excel_filename: Optional[str] = None,
                                            deseasonalizer=None) -> Dict[str, Any]:
    """Compatibility wrapper for existing calling pattern."""

    explainer = StreamlinedAnomalyExplainerWithSeason(model_results, original_df, deseasonalizer)

    # Handle feature count
    top_n = len(explainer.feature_columns) if analyze_all_features else (top_n_features or 10)

    # Perform analysis
    analysis = explainer.analyze_anomaly(target_date, top_n)

    if 'error' in analysis:
        return analysis

    # Add compatibility fields
    analysis['analysis_scope'] = 'ALL_FEATURES' if analyze_all_features else f'TOP_{top_n}'
    analysis['features_analyzed'] = len(analysis['explanations'])
    analysis['top_contributing_features'] = analysis['explanations']
    analysis['dq_dimension_summary'] = analysis['dq_summary']
    analysis['counterfactual_fixpack'] = analysis['fixes']

    # Export if requested
    if export_to_excel:
        filename = excel_filename or f"anomaly_analysis_{target_date.replace('-', '_')}.xlsx"
        results_dict = {target_date: analysis}
        export_path = export_comprehensive_report(results_dict, filename)
        analysis['excel_report_path'] = export_path

    return analysis

In [ ]:
stl_deseasonalizer = SmartSTLDeseasonalizer(frequency='daily')

Detecting system capabilities...
System specs - CPU: 8 cores, RAM: 51.0GB
cuDF (GPU DataFrame operations) detected
CuPy installed but GPU not accessible
PyTorch available but no GPU support
Selecting optimal execution mode...
Selected parallel_cpu mode - multi-core CPU processing
Initializing compute backend...
Parallel processing configured with 8 workers
Smart STL Deseasonalizer initialized successfully
Mode: parallel_cpu
Frequency: daily
System capabilities: CPU: 8 cores, RAM: 51.0GB
STL parameters: {'seasonal': 15, 'period': 7, 'min_periods': 21}


In [ ]:
explanation = analyze_anomaly_with_comprehensive_report(
    model_results=lstm_pyod_results,
    original_df=transformed_cv_df,
    target_date='2023-02-27',
    analyze_all_features=True,
    deseasonalizer=stl_deseasonalizer  # Pass your STL instance
)

Comprehensive report exported to anomaly_analysis_2023_02_27.xlsx


**deseasonlity testing**

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create sample data with clear seasonal pattern
np.random.seed(42)
dates = pd.date_range('2023-01-01', '2023-03-31', freq='D')

# Generate data with weekly seasonality (higher on weekends)
data = []
for i, date in enumerate(dates):
    base_value = 1000  # Base metric value

    # Weekly seasonality (higher on Friday=4, Saturday=5, Sunday=6)
    day_of_week = date.dayofweek
    seasonal_component = {0: 50, 1: 30, 2: 40, 3: 60, 4: 200, 5: 300, 6: 250}[day_of_week]

    # Trend component
    trend = i * 2

    # Random noise
    noise = np.random.normal(0, 20)

    # Total value
    total_value = base_value + seasonal_component + trend + noise

    data.append({
        'date': date.strftime('%Y-%m-%d'),
        'daily_records': total_value,
        'daily_records_Mean': total_value * 0.8 + np.random.normal(0, 5),
        'daily_records_Sum': total_value * 1.2 + np.random.normal(0, 10)
    })

# Create DataFrame
test_df = pd.DataFrame(data)
print("Original DataFrame:")
print(test_df.head(10))
print(f"Shape: {test_df.shape}")

# Initialize STL Deseasonalizer
# from SmartSTLDeseasonalizer import SmartSTLDeseasonalizer

deseasonalizer = SmartSTLDeseasonalizer(frequency='daily')

# Apply deseasonalization
print("\n" + "="*50)
print("APPLYING DESEASONALIZATION")
print("="*50)

deseasonalized_df = deseasonalizer.deseasonalize_dataframe(
    df=test_df,
    date_col='date',
    value_cols=['daily_records', 'daily_records_Mean', 'daily_records_Sum']
)

print("\nDeseasonalized DataFrame:")
print(deseasonalized_df[['date', 'daily_records', 'daily_records_deseasonalized']].head(10))

# Test recomposition for specific values
print("\n" + "="*50)
print("TESTING RECOMPOSITION")
print("="*50)

# Test dates
test_dates = ['2023-02-15', '2023-02-18', '2023-02-19']  # Wed, Sat, Sun

for test_date in test_dates:
    row = deseasonalized_df[deseasonalized_df['date'] == test_date].iloc[0]

    original_value = row['daily_records']
    deseasonalized_value = row['daily_records_deseasonalized']

    # Recompose back to original
    # recomposed_value = deseasonalizer.recompose_for_explanation(
    #     deseasonalized_values=pd.Series([deseasonalized_value]),
    #     original_column_name='daily_records'
    # ).iloc[0]

    test_row_index = deseasonalized_df[deseasonalized_df['date'] == test_date].index[0]
    recomposed_value = deseasonalizer.recompose_for_explanation(
        deseasonalized_values=pd.Series([deseasonalized_value], index=[test_row_index]),
        original_column_name='daily_records'
    ).iloc[0]

    print(f"\nDate: {test_date}")
    print(f"Original Value: {original_value:.2f}")
    print(f"Deseasonalized: {deseasonalized_value:.2f}")
    print(f"Recomposed: {recomposed_value:.2f}")
    print(f"Difference: {abs(original_value - recomposed_value):.6f}")

    # Verify they match (within floating point precision)
    assert abs(original_value - recomposed_value) < 1e-10, f"Values don't match for {test_date}"

print("\n" + "="*50)
print("TESTING WITH EXPLAINER CLASS")
print("="*50)

# Create mock model results for testing
mock_model_results = {
    'anomaly_scores': {
        '2023-02-18': {'consensus_score': 0.85, 'lstm_score': 0.9, 'pyod_score': 0.8},
        '2023-02-19': {'consensus_score': 0.75, 'lstm_score': 0.7, 'pyod_score': 0.8}
    },
    'anomaly_severity': {
        '2023-02-18': 'HIGH',
        '2023-02-19': 'MEDIUM'
    }
}

# Test with explainer
# from StreamlinedAnomalyExplainerWithSeason import analyze_anomaly_with_comprehensive_report

explanation = analyze_anomaly_with_comprehensive_report(
    model_results=mock_model_results,
    original_df=deseasonalized_df,
    target_date='2023-02-18',
    top_n_features=3,
    deseasonalizer=deseasonalizer
)

print(f"\nExplainer Results for 2023-02-18:")
print(f"Severity: {explanation['severity']}")
print(f"Features analyzed: {explanation['features_analyzed']}")

# Check if recomposition worked in explainer
for feature_exp in explanation['explanations'][:2]:
    print(f"\nFeature: {feature_exp['feature']}")
    print(f"Current Value (recomposed): {feature_exp['current_value']}")
    print(f"Deseasonalized Value: {feature_exp.get('current_deseasonalized_value', 'N/A')}")

print("\n" + "="*50)
print("TEST COMPLETED SUCCESSFULLY!")
print("✓ Deseasonalization working")
print("✓ Recomposition accurate")
print("✓ Explainer integration functional")
print("="*50)

Original DataFrame:
         date  daily_records  daily_records_Mean  daily_records_Sum
0  2023-01-01    1259.934283         1007.256105        1518.398025
1  2023-01-02    1082.460597          864.797711        1296.611347
2  2023-01-03    1065.584256          856.304579        1274.006364
3  2023-01-04    1056.851201          843.163872        1263.564144
4  2023-01-05    1072.839245          848.704995        1270.157916
5  2023-01-06    1198.754249          953.939244        1441.647573
6  2023-01-07    1293.839518         1028.010096        1567.263910
7  2023-01-08    1259.484474         1007.925220        1497.133887
8  2023-01-09    1055.112346          844.644489        1254.624879
9  2023-01-10    1055.513960          841.407975        1263.699815
Shape: (90, 4)
Detecting system capabilities...
System specs - CPU: 8 cores, RAM: 51.0GB
cuDF (GPU DataFrame operations) detected
CuPy installed but GPU not accessible
PyTorch available but no GPU support
Selecting optimal execution

AssertionError: Values don't match for 2023-02-15

**ANAMOLY with current value**

In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Tuple, Optional
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from openpyxl.styles import PatternFill, Font, Border, Side
import warnings
warnings.filterwarnings('ignore')

class StreamlinedAnomalyExplainer:
    """Simplified anomaly explainer with DQ dimensions and enhanced reporting."""

    def __init__(self, model_results: Dict[str, Any], original_df: pd.DataFrame, date_column: str = 'date'):
        self.model_results = model_results
        self.original_df = original_df.copy()
        self.date_column = date_column

        if date_column in self.original_df.columns:
            self.original_df[date_column] = pd.to_datetime(self.original_df[date_column]).dt.strftime('%Y-%m-%d')

        self.feature_columns = [col for col in original_df.columns if col != date_column]
        self.dq_patterns = {
            'Completeness': ['completeness', 'missing', 'null', 'isnotnull'],
            'Accuracy': ['accuracy', 'entropy', 'zscore', 'outlier', 'anomaly', 'mean', 'sum', 'count'],
            'Consistency': ['consistency', 'standarddeviation', 'variance', 'correlation'],
            'Validity': ['validity', 'minimum', 'maximum', 'datatype', 'pattern', 'regex'],
            'Uniqueness': ['uniqueness', 'countdistinct', 'distinctness', 'duplicate'],
            'Timeliness': ['timeliness', 'freshness', 'lag', 'delay', 'timestamp']
        }

    def analyze_anomaly(self, target_date: str, top_n: int = 10) -> Dict[str, Any]:
        """Main analysis function."""
        target_date = pd.to_datetime(target_date).strftime('%Y-%m-%d')

        # Validate input
        anomaly_scores = self.model_results.get('anomaly_scores', {})
        if target_date not in anomaly_scores:
            return {'error': f"No anomaly detected for {target_date}"}

        target_row = self.original_df[self.original_df[self.date_column] == target_date].iloc[0]
        historical_df = self.original_df[self.original_df[self.date_column] < target_date]

        # Calculate feature contributions
        contributions = self._calculate_contributions(target_date, target_row, historical_df)
        top_features = sorted(contributions.items(), key=lambda x: x[1], reverse=True)[:top_n]

        # Generate explanations
        explanations = [self._explain_feature(f, score, target_row[f], historical_df[f].dropna(), target_date)
                       for f, score in top_features]

        # Create DQ summary
        dq_summary = self._summarize_dq_dimensions(explanations)

        return {
            'date': target_date,
            'severity': self.model_results.get('anomaly_severity', {}).get(target_date, 'UNKNOWN'),
            'consensus_score': round(anomaly_scores[target_date].get('consensus_score', 0), 4),
            'explanations': explanations,
            'dq_summary': dq_summary,
            'fixes': self._generate_fixes(target_date, target_row, [f for f, _ in top_features[:5]]),
            'executive_summary': self._create_summary(explanations, dq_summary, target_date)
        }

    def _calculate_contributions(self, target_date: str, target_row: pd.Series,
                               historical_df: pd.DataFrame) -> Dict[str, float]:
        """Calculate feature importance scores using ensemble approach."""
        contributions = {}

        for feature in self.feature_columns:
            current_val = target_row[feature]
            hist_values = historical_df[feature].dropna()

            if len(hist_values) < 3:
                contributions[feature] = 0
                continue

            # Statistical deviation score
            z_score = abs((current_val - hist_values.mean()) / (hist_values.std() + 1e-8))

            # Pattern deviation score
            recent_trend = hist_values.tail(7).mean()
            expected = 0.7 * recent_trend + 0.3 * hist_values.median()
            pattern_score = abs(current_val - expected) / (hist_values.std() + 1e-8)

            # Model perturbation score (if available)
            model_score = 0
            model = self.model_results.get('statistical_model')
            if model is not None:
                baseline_data = target_row[self.feature_columns].values.reshape(1, -1)
                baseline_anomaly = abs(model.decision_function(baseline_data)[0])

                perturbed_data = baseline_data.copy()
                perturbed_data[0, self.feature_columns.index(feature)] = hist_values.median()
                perturbed_anomaly = abs(model.decision_function(perturbed_data)[0])

                model_score = max(0, baseline_anomaly - perturbed_anomaly)

            # Ensemble score
            contributions[feature] = 0.4 * z_score + 0.4 * pattern_score + 0.2 * model_score

        return contributions

    def _explain_feature(self, feature: str, importance: float, current_val: float,
                        hist_values: pd.Series, target_date: str) -> Dict[str, Any]:
        """Generate explanation for a single feature with raw value lookup for deseasonalized features."""
        if len(hist_values) == 0:
            return {
                'feature': feature,
                'dq_dimension': self._map_to_dq_dimension(feature),
                'importance': round(importance, 4),
                'severity': 'UNKNOWN',
                'explanation': f"No historical data for {feature}",
                'current_value': current_val,
                'baseline': None
            }

        hist_mean = hist_values.mean()
        hist_std = hist_values.std()
        z_score = (current_val - hist_mean) / (hist_std + 1e-8)
        deviation_pct = ((current_val - hist_mean) / (abs(hist_mean) + 1e-8)) * 100

        # Classify severity
        severity = ('CRITICAL' if abs(z_score) >= 4 or abs(deviation_pct) >= 300 else
                   'HIGH' if abs(z_score) >= 3 or abs(deviation_pct) >= 150 else
                   'MEDIUM' if abs(z_score) >= 2 or abs(deviation_pct) >= 75 else 'LOW')

        # Generate explanation
        direction = "increased" if current_val > hist_mean else "decreased"
        magnitude = ("drastically" if abs(deviation_pct) >= 200 else
                    "significantly" if abs(deviation_pct) >= 100 else
                    "moderately" if abs(deviation_pct) >= 50 else "slightly")

        explanation = f"{feature} has {magnitude} {direction} by {abs(deviation_pct):.1f}% from baseline"

        # Handle raw value lookup for deseasonalized features
        display_current_value = current_val  # Default to deseasonalized value

        if feature.endswith('_deseasonalized'):
            # Get base column name and look for raw value
            base_column = feature.replace('_deseasonalized', '')

            if base_column in self.original_df.columns:
                target_row = self.original_df[self.original_df[self.date_column] == target_date]
                if not target_row.empty:
                    display_current_value = target_row.iloc[0][base_column]

        return {
            'feature': feature,
            'base_column': feature.split('_')[0],
            'dq_dimension': self._map_to_dq_dimension(feature),
            'importance': round(importance, 4),
            'severity': severity,
            'current_value': round(display_current_value, 6),
            'baseline': round(hist_mean, 6),
            'z_score': round(z_score, 3),
            'deviation_pct': round(deviation_pct, 2),
            'explanation': explanation,
            'recommendation': self._get_recommendation(severity, feature.split('_')[0])
        }

    def _map_to_dq_dimension(self, feature: str) -> str:
        """Map feature to DQ dimension."""
        feature_lower = feature.lower()
        for dimension, patterns in self.dq_patterns.items():
            if any(pattern in feature_lower for pattern in patterns):
                return dimension
        return 'Accuracy'

    def _summarize_dq_dimensions(self, explanations: List[Dict]) -> Dict[str, Dict]:
        """Summarize by DQ dimension."""
        summary = {}
        for exp in explanations:
            dq_dim = exp['dq_dimension']
            if dq_dim not in summary:
                summary[dq_dim] = {'count': 0, 'total_impact': 0, 'severities': []}

            summary[dq_dim]['count'] += 1
            summary[dq_dim]['total_impact'] += exp['importance']
            summary[dq_dim]['severities'].append(exp['severity'])

        # Calculate metrics
        for dq_dim in summary:
            summary[dq_dim]['avg_impact'] = summary[dq_dim]['total_impact'] / summary[dq_dim]['count']
            severity_counts = {s: summary[dq_dim]['severities'].count(s) for s in ['CRITICAL', 'HIGH', 'MEDIUM', 'LOW']}
            summary[dq_dim]['severity_breakdown'] = severity_counts
            summary[dq_dim]['risk_level'] = self._assess_risk_level(severity_counts)

        return summary

    def _assess_risk_level(self, severity_counts: Dict[str, int]) -> str:
        """Assess overall risk level for DQ dimension."""
        if severity_counts.get('CRITICAL', 0) > 0:
            return 'EXTREME'
        elif severity_counts.get('HIGH', 0) >= 2:
            return 'HIGH'
        elif severity_counts.get('HIGH', 0) > 0 or severity_counts.get('MEDIUM', 0) >= 3:
            return 'MODERATE'
        return 'LOW'

    def _generate_fixes(self, target_date: str, target_row: pd.Series, features: List[str]) -> List[Dict]:
        """Generate actionable model parameter adjustments and acceptable ranges."""
        historical_df = self.original_df[self.original_df[self.date_column] < target_date]
        fixes = []

        for feature in features:
            if feature in historical_df.columns:
                hist_values = historical_df[feature].dropna()
                current = target_row[feature]

                # Calculate acceptable range (median ± 2 std)
                median_val = hist_values.median()
                std_val = hist_values.std()
                acceptable_min = median_val - (2 * std_val)
                acceptable_max = median_val + (2 * std_val)

                # Determine severity and parameter adjustments
                z_score = abs((current - median_val) / (std_val + 1e-8))

                # Model parameter recommendations based on severity
                if z_score >= 4:
                    sensitivity = "Decrease contamination to 0.01, increase consensus_threshold to 0.7"
                elif z_score >= 3:
                    sensitivity = "Decrease contamination to 0.015, increase consensus_threshold to 0.6"
                elif z_score >= 2:
                    sensitivity = "Use default contamination 0.02, consensus_threshold 0.5"
                else:
                    sensitivity = "Increase contamination to 0.03, decrease consensus_threshold to 0.4"

                # Get raw value for deseasonalized features
                display_current = current
                if feature.endswith('_deseasonalized'):
                    base_column = feature.replace('_deseasonalized', '')
                    if base_column in self.original_df.columns:
                        target_row_data = self.original_df[self.original_df[self.date_column] == target_date]
                        if not target_row_data.empty:
                            display_current = target_row_data.iloc[0][base_column]

                fixes.append({
                    'feature': feature,
                    'current_value': round(display_current, 6),
                    'acceptable_range_min': round(acceptable_min, 6),
                    'acceptable_range_max': round(acceptable_max, 6),
                    'baseline_median': round(median_val, 6),
                    'z_score': round(z_score, 3),
                    'dq_dimension': self._map_to_dq_dimension(feature),
                    'model_parameter_adjustment': sensitivity,
                    'data_quality_action': self._get_dq_action(feature, z_score),
                    'acceptable_deviation': f"±{round(2 * std_val, 3)} from baseline"
                })

        return fixes

    def _get_dq_action(self, feature: str, z_score: float) -> str:
        """Get data quality improvement action."""
        dq_dim = self._map_to_dq_dimension(feature)

        actions = {
            'Completeness': f"Review data ingestion pipeline for {feature.split('_')[0]}",
            'Accuracy': f"Validate calculation logic for {feature.split('_')[0]} metrics",
            'Consistency': f"Standardize data formats in {feature.split('_')[0]} processing",
            'Validity': f"Check business rules compliance for {feature.split('_')[0]}",
            'Uniqueness': f"Investigate duplicate handling in {feature.split('_')[0]}",
            'Timeliness': f"Review data refresh schedules for {feature.split('_')[0]}"
        }

        severity_prefix = "URGENT: " if z_score >= 3 else "Monitor: "
        return severity_prefix + actions.get(dq_dim, f"Review {feature.split('_')[0]} data quality")

    def _get_recommendation(self, severity: str, column: str) -> str:
        """Get actionable recommendation."""
        recommendations = {
            'CRITICAL': f"IMMEDIATE: Investigate {column} data pipeline and source systems",
            'HIGH': f"URGENT: Review {column} data collection within 4 hours",
            'MEDIUM': f"MONITOR: Schedule {column} review within 24 hours",
            'LOW': f"TRACK: Include {column} in next routine review"
        }
        return recommendations.get(severity, f"Review {column} data quality")

    def _create_summary(self, explanations: List[Dict], dq_summary: Dict, date: str) -> str:
        """Create executive summary."""
        if not explanations:
            return f"No significant anomalies detected for {date}"

        critical_count = sum(1 for exp in explanations if exp['severity'] == 'CRITICAL')
        high_count = sum(1 for exp in explanations if exp['severity'] == 'HIGH')

        summary = f"Anomaly Alert {date}: "

        if critical_count > 0:
            summary += f"{critical_count} critical issues require immediate attention. "
        if high_count > 0:
            summary += f"{high_count} high-priority issues need 4-hour review. "

        # Top DQ dimensions
        if dq_summary:
            top_dq = max(dq_summary.items(), key=lambda x: x[1]['total_impact'])
            summary += f"Primary concern: {top_dq[0]} dimension ({top_dq[1]['count']} features affected)."

        return summary


def export_comprehensive_report(analysis_results: Dict[str, Dict],
                              filename: str = 'anomaly_intelligence_report.xlsx') -> str:
    """Export enhanced report with better formatting and user-friendly tabs."""

    # Color schemes
    severity_colors = {
        'CRITICAL': PatternFill(start_color='DC143C', end_color='DC143C', fill_type='solid'),
        'HIGH': PatternFill(start_color='FF8C00', end_color='FF8C00', fill_type='solid'),
        'MEDIUM': PatternFill(start_color='FFD700', end_color='FFD700', fill_type='solid'),
        'LOW': PatternFill(start_color='98FB98', end_color='98FB98', fill_type='solid')
    }

    severity_fonts = {
        'CRITICAL': Font(color='FFFFFF', bold=True),
        'HIGH': Font(color='FFFFFF', bold=True),
        'MEDIUM': Font(color='000000', bold=True),
        'LOW': Font(color='000000', bold=False)
    }

    with pd.ExcelWriter(filename, engine='openpyxl') as writer:

        # Tab 1: Executive Dashboard
        dashboard_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                critical_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'CRITICAL')
                high_count = sum(1 for exp in result['explanations'] if exp['severity'] == 'HIGH')

                dashboard_data.append({
                    'Alert_Date': date,
                    'Overall_Severity': result['severity'],
                    'Consensus_Score': result['consensus_score'],
                    'Critical_Issues': critical_count,
                    'High_Priority_Issues': high_count,
                    'DQ_Dimensions_Affected': len(result['dq_summary']),
                    'Action_Items': critical_count + high_count,
                    'Executive_Summary': result['executive_summary']
                })

        if dashboard_data:
            dashboard_df = pd.DataFrame(dashboard_data)
            dashboard_df.to_excel(writer, sheet_name='Executive_Dashboard', index=False)

            # Apply severity coloring
            ws = writer.sheets['Executive_Dashboard']
            for row in range(2, len(dashboard_df) + 2):
                severity = ws[f'B{row}'].value
                if severity in severity_colors:
                    ws[f'B{row}'].fill = severity_colors[severity]
                    ws[f'B{row}'].font = severity_fonts[severity]

        # Tab 2: Immediate Actions Required
        action_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    if exp['severity'] in ['CRITICAL', 'HIGH']:
                        sla = '2 hours' if exp['severity'] == 'CRITICAL' else '4 hours'
                        action_data.append({
                            'Date': date,
                            'Priority': exp['severity'],
                            'Feature': exp['feature'],
                            'Data_Source': exp['base_column'],
                            'DQ_Issue_Type': exp['dq_dimension'],
                            'Impact_Score': exp['importance'],
                            'Issue_Description': exp['explanation'],
                            'Action_Required': exp['recommendation'],
                            'SLA': sla,
                            'Owner': 'Data Engineering' if exp['dq_dimension'] == 'Completeness' else 'Data Quality Team'
                        })

        if action_data:
            action_df = pd.DataFrame(action_data)
            action_df = action_df.sort_values(['Priority', 'Impact_Score'], ascending=[True, False])
            action_df.to_excel(writer, sheet_name='Immediate_Actions', index=False)

            # Color code by priority
            ws = writer.sheets['Immediate_Actions']
            for row in range(2, len(action_df) + 2):
                priority = ws[f'B{row}'].value
                if priority in severity_colors:
                    for col in range(1, len(action_df.columns) + 1):
                        ws.cell(row=row, column=col).fill = severity_colors[priority]
                        if col == 2:  # Priority column
                            ws.cell(row=row, column=col).font = severity_fonts[priority]

        # Tab 3: Data Quality Heatmap
        heatmap_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for dq_dim, stats in result['dq_summary'].items():
                    risk_score = stats['total_impact']
                    heatmap_data.append({
                        'Date': date,
                        'DQ_Dimension': dq_dim,
                        'Risk_Score': round(risk_score, 2),
                        'Risk_Level': stats['risk_level'],
                        'Features_Affected': stats['count'],
                        'Critical_Issues': stats['severity_breakdown'].get('CRITICAL', 0),
                        'High_Issues': stats['severity_breakdown'].get('HIGH', 0),
                        'Average_Impact': round(stats['avg_impact'], 3)
                    })

        if heatmap_data:
            heatmap_df = pd.DataFrame(heatmap_data)
            heatmap_df.to_excel(writer, sheet_name='DQ_Risk_Heatmap', index=False)

            # Apply risk level coloring
            ws = writer.sheets['DQ_Risk_Heatmap']
            risk_colors = {
                'EXTREME': severity_colors['CRITICAL'],
                'HIGH': severity_colors['HIGH'],
                'MODERATE': severity_colors['MEDIUM'],
                'LOW': severity_colors['LOW']
            }

            for row in range(2, len(heatmap_df) + 2):
                risk_level = ws[f'D{row}'].value
                if risk_level in risk_colors:
                    ws[f'D{row}'].fill = risk_colors[risk_level]

        # Tab 4: Feature Analysis Details
        detail_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for exp in result['explanations']:
                    detail_data.append({
                        'Date': date,
                        'Feature': exp['feature'],
                        'Data_Source': exp['base_column'],
                        'DQ_Dimension': exp['dq_dimension'],
                        'Severity': exp['severity'],
                        'Current_Value': exp['current_value'],
                        'Baseline_Value': exp['baseline'],
                        'Deviation_%': exp['deviation_pct'],
                        'Z_Score': exp['z_score'],
                        'Impact_Score': exp['importance'],
                        'Business_Impact': exp['explanation'],
                        'Recommended_Action': exp['recommendation']
                    })

        if detail_data:
            detail_df = pd.DataFrame(detail_data)
            detail_df.to_excel(writer, sheet_name='Feature_Analysis', index=False)

            # Apply severity coloring
            ws = writer.sheets['Feature_Analysis']
            for row in range(2, len(detail_df) + 2):
                severity = ws[f'E{row}'].value
                if severity in severity_colors:
                    ws[f'E{row}'].fill = severity_colors[severity]
                    ws[f'E{row}'].font = severity_fonts[severity]

        # Tab 5: Model Tuning & Data Quality Actions
        fix_data = []
        for date, result in analysis_results.items():
            if 'error' not in result:
                for fix in result['fixes']:
                    fix_data.append({
                        'Date': date,
                        'Feature': fix['feature'],
                        'DQ_Dimension': fix['dq_dimension'],
                        'Current_Value': fix['current_value'],
                        'Acceptable_Range': f"{fix['acceptable_range_min']} to {fix['acceptable_range_max']}",
                        'Baseline_Median': fix['baseline_median'],
                        'Deviation_Tolerance': fix['acceptable_deviation'],
                        'Z_Score': fix['z_score'],
                        'Model_Parameter_Tuning': fix['model_parameter_adjustment'],
                        'Data_Quality_Action': fix['data_quality_action']
                    })

        if fix_data:
            fix_df = pd.DataFrame(fix_data)
            fix_df.to_excel(writer, sheet_name='Model_Tuning_Actions', index=False)

    print(f"Comprehensive report exported to {filename}")
    return filename


def analyze_anomaly_with_comprehensive_report(model_results: Dict, original_df: pd.DataFrame,
                                            target_date: str, top_n_features: Optional[int] = None,
                                            analyze_all_features: bool = False,
                                            export_to_excel: bool = True,
                                            excel_filename: Optional[str] = None,
                                            date_column: str = 'date') -> Dict[str, Any]:
    """Compatibility wrapper for existing calling pattern."""

    explainer = StreamlinedAnomalyExplainer(model_results, original_df, date_column)

    # Handle feature count
    top_n = len(explainer.feature_columns) if analyze_all_features else (top_n_features or 10)

    # Perform analysis
    analysis = explainer.analyze_anomaly(target_date, top_n)

    if 'error' in analysis:
        return analysis

    # Add compatibility fields
    analysis['analysis_scope'] = 'ALL_FEATURES' if analyze_all_features else f'TOP_{top_n}'
    analysis['features_analyzed'] = len(analysis['explanations'])
    analysis['top_contributing_features'] = analysis['explanations']
    analysis['dq_dimension_summary'] = analysis['dq_summary']
    analysis['counterfactual_fixpack'] = analysis['fixes']

    # Export if requested
    if export_to_excel:
        filename = excel_filename or f"anomaly_analysis_{target_date.replace('-', '_')}.xlsx"
        results_dict = {target_date: analysis}
        export_path = export_comprehensive_report(results_dict, filename)
        analysis['excel_report_path'] = export_path

    return analysis